In [14]:
import json
import uuid
import gzip
import asyncio
import websockets
import numpy as np
import sounddevice as sd
import nest_asyncio
from unihiker import GUI
import time
from tkinter import END

# 配置参数
appid = "4166554764"    # 项目的 appid
token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"    # 项目的 token
cluster = "volcengine_streaming_common"  # 请求的集群

# 协议常量
PROTOCOL_VERSION = 0b0001
DEFAULT_HEADER_SIZE = 0b0001

PROTOCOL_VERSION_BITS = 4
HEADER_BITS = 4
MESSAGE_TYPE_BITS = 4
MESSAGE_TYPE_SPECIFIC_FLAGS_BITS = 4
MESSAGE_SERIALIZATION_BITS = 4
MESSAGE_COMPRESSION_BITS = 4
RESERVED_BITS = 8

# Message Type:
CLIENT_FULL_REQUEST = 0b0001
CLIENT_AUDIO_ONLY_REQUEST = 0b0010
SERVER_FULL_RESPONSE = 0b1001
SERVER_ACK = 0b1011
SERVER_ERROR_RESPONSE = 0b1111

# Message Type Specific Flags
NO_SEQUENCE = 0b0000
POS_SEQUENCE = 0b0001
NEG_SEQUENCE = 0b0010
NEG_SEQUENCE_1 = 0b0011

# Message Serialization
NO_SERIALIZATION = 0b0000
JSON = 0b0001
THRIFT = 0b0011
CUSTOM_TYPE = 0b1111

# Message Compression
NO_COMPRESSION = 0b0000
GZIP = 0b0001
CUSTOM_COMPRESSION = 0b1111




# 初始化GUI
gui = GUI()

# 创建标题
title = gui.draw_text(
    x=120,
    y=30,
    text="语音识别系统",
    font_size=24,
    origin='center'
)

# 创建文本框
text_box = gui.add_text_box(
    x=120,      # 中心x坐标
    y=160,      # 中心y坐标
    w=220,      # 宽度
    h=200,      # 高度
    origin='center',  # 居中对齐
    font_size=14
)

# 存储所有识别文本
all_texts = []

def update_recognition_text(new_text):
    """更新识别结果并保持滚动条在底部"""
    try:
        # 添加新文本到列表
        all_texts.append(new_text)
        
        # 更新文本框内容
        full_text = "\n".join(all_texts)
        text_box.config(text=full_text)
        
        # 滚动到底部
        text_box.text.see(END)
        
        # 更新GUI
        if gui.master.winfo_exists():
            gui.update()
            
    except Exception as e:
        print(f"更新文本错误: {e}")

def generate_header(
    version=PROTOCOL_VERSION,
    message_type=CLIENT_FULL_REQUEST,
    message_type_specific_flags=NO_SEQUENCE,
    serial_method=JSON,
    compression_type=GZIP,
    reserved_data=0x00,
    extension_header=bytes()
):
    header = bytearray()
    header_size = int(len(extension_header) / 4) + 1
    header.append((version << 4) | header_size)
    header.append((message_type << 4) | message_type_specific_flags)
    header.append((serial_method << 4) | compression_type)
    header.append(reserved_data)
    header.extend(extension_header)
    return header

def generate_full_default_header():
    return generate_header()

def generate_audio_default_header():
    return generate_header(
        message_type=CLIENT_AUDIO_ONLY_REQUEST
    )

def parse_response(res):
    """解析响应"""
    try:
        protocol_version = res[0] >> 4
        header_size = res[0] & 0x0f
        message_type = res[1] >> 4
        message_type_specific_flags = res[1] & 0x0f
        serialization_method = res[2] >> 4
        message_compression = res[2] & 0x0f
        reserved = res[3]
        header_extensions = res[4:header_size * 4]
        payload = res[header_size * 4:]
        result = {}
        payload_msg = None
        payload_size = 0
        
        if message_type == SERVER_FULL_RESPONSE:
            payload_size = int.from_bytes(payload[:4], "big", signed=True)
            payload_msg = payload[4:]
        elif message_type == SERVER_ACK:
            seq = int.from_bytes(payload[:4], "big", signed=True)
            result['seq'] = seq
            if len(payload) >= 8:
                payload_size = int.from_bytes(payload[4:8], "big", signed=False)
                payload_msg = payload[8:]
        elif message_type == SERVER_ERROR_RESPONSE:
            code = int.from_bytes(payload[:4], "big", signed=False)
            result['code'] = code
            payload_size = int.from_bytes(payload[4:8], "big", signed=False)
            payload_msg = payload[8:]
            
        if payload_msg is None:
            return result
            
        if message_compression == GZIP:
            payload_msg = gzip.decompress(payload_msg)
            
        if serialization_method == JSON:
            payload_msg = json.loads(str(payload_msg, "utf-8"))
        elif serialization_method != NO_SERIALIZATION:
            payload_msg = str(payload_msg, "utf-8")
            
        result['payload_msg'] = payload_msg
        result['payload_size'] = payload_size
        return result
    except Exception as e:
        return {"error": f"Failed to parse response: {str(e)}"}

class AsrWsClient:
    def __init__(self, appid, token, cluster):
        self.appid = appid
        self.token = token
        self.cluster = cluster
        self.ws_url = "wss://openspeech.bytedance.com/api/v2/asr"
        self.success_code = 1000
        self.uid = "streaming_asr_demo"
        self.workflow = "audio_in,resample,partition,vad,fe,decode,itn,nlu_punctuate"
        self.show_language = False
        self.show_utterances = True
        self.result_type = "single"
        self.format = "raw"
        self.rate = 16000
        self.language = "zh-CN"
        self.bits = 16
        self.channel = 1
        self.codec = "raw"
        self.auth_method = "token"

    def construct_request(self, reqid):
        return {
            'app': {
                'appid': self.appid,
                'cluster': self.cluster,
                'token': self.token,
            },
            'user': {
                'uid': self.uid
            },
            'request': {
                'reqid': reqid,
                'nbest': 1,
                'workflow': self.workflow,
                'show_language': self.show_language,
                'show_utterances': self.show_utterances,
                'result_type': self.result_type,
                'sequence': 1
            },
            'audio': {
                'format': self.format,
                'rate': self.rate,
                'language': self.language,
                'bits': self.bits,
                'channel': self.channel,
                'codec': self.codec
            }
        }

    def token_auth(self):
        return {'Authorization': f'Bearer; {self.token}'}

    async def process_microphone(self):
        """实时麦克风录音并识别"""
        reqid = str(uuid.uuid4())
        request_params = self.construct_request(reqid)
        
        # 构造初始请求
        payload_bytes = str.encode(json.dumps(request_params))
        payload_bytes = gzip.compress(payload_bytes)
        full_request = bytearray(generate_full_default_header())
        full_request.extend(len(payload_bytes).to_bytes(4, 'big'))
        full_request.extend(payload_bytes)

        print("建立WebSocket连接...")
        async with websockets.connect(
            self.ws_url, 
            extra_headers=self.token_auth(), 
            max_size=1000000000
        ) as ws:
            # 发送初始请求
            await ws.send(full_request)
            response = await ws.recv()
            result = parse_response(response)
            print(f"初始化响应: {result}")
            
            if 'payload_msg' in result and result['payload_msg']['code'] == self.success_code:
                print("初始化成功")
                print("录音任务已启动")
                chunk_size = 9600  # 每次读取的采样点数
                
                with sd.InputStream(
                    channels=1, 
                    samplerate=16000,
                    dtype=np.int16,
                    blocksize=chunk_size,
                    callback=None
                ) as stream:
                    print("开始录音...")
                    try:
                        while True:
                            # 读取音
                            audio_data, overflowed = stream.read(chunk_size)
                            if overflowed:
                                print("警告：音频缓冲区溢出")
                                
                            # 转换为字节
                            audio_bytes = audio_data.tobytes()
                            
                            # 压缩音频数据
                            compressed_audio = gzip.compress(audio_bytes)
                            
                            # 构造音频数据请求
                            audio_request = bytearray(generate_audio_default_header())
                            audio_request.extend(len(compressed_audio).to_bytes(4, 'big'))
                            audio_request.extend(compressed_audio)
                            
                            # 发送音频数据
                            await ws.send(audio_request)
                            
                            # 接收识别结果
                            response = await ws.recv()
                            result = parse_response(response)
                            
                            # 处理识别结果
                            if 'payload_msg' in result and 'result' in result['payload_msg']:
                                utterances = result['payload_msg']['result'][0].get('utterances', [])
                                for utterance in utterances:
                                    if not utterance['definite']:
                                        print(f"\r[识别中...] {utterance['text']}", end='', flush=True)
                                    else:
                                        print(f"\n[最终结果] {utterance['text']}")
                                        update_recognition_text(utterance['text'])
                    except KeyboardInterrupt:
                        # 发送最后一个音频包
                        last_request = bytearray(generate_last_audio_default_header())
                        last_request.extend(len(compressed_audio).to_bytes(4, 'big'))
                        last_request.extend(compressed_audio)
                        await ws.send(last_request)
                        print("\n录音已停止")
                    except Exception as e:
                        print(f"录音过程发生错误: {e}")
            else:
                print(f"初始化失败: {result['payload_msg'].get('message')}")

# 在notebook中运行异步代码的辅助函数
import nest_asyncio
nest_asyncio.apply()

# 创建客户端实例并运行
client = AsrWsClient(appid, token, cluster)
try:
    loop = asyncio.get_event_loop()
    loop.run_until_complete(client.process_microphone())
except KeyboardInterrupt:
    print("\n程序已被用户中断")
except Exception as e:
    print(f"\n程序异常: {e}")
finally:
    print("程序已退出")

def update_text_with_scroll(new_text):
    """更新文本并滚动到底部"""
    try:
        # 获取当前内容并追加新文本
        current_text = text_box.text.get("1.0", "end-1c")  # end-1c 去掉最后的换行符
        updated_text = current_text + new_text + "\n"
        
        # 更新文本内容
        text_box.text.delete("1.0", "end")
        text_box.text.insert("1.0", updated_text)
        
        # 强制滚动到底部
        text_box.text.yview_moveto(1.0)
        
        # 更新GUI
        if hasattr(gui, 'update'):
            gui.update()
            
    except Exception as e:
        print(f"更新文本错误: {e}")

# 测试文本更新
count = 1
while True:
    try:
        # 添加新文本
        new_text = f"这是第 {count} 行测试文本"
        update_text_with_scroll(new_text)
        
        count += 1
        time.sleep(1)
        
    except KeyboardInterrupt:
        break

def update_recognition_text(new_text):
    """更新识别结果并滚动到底部"""
    current_text = text_box.text.get("1.0", "end")  # 获取当前内容
    text_box.config(text=current_text + new_text + "\n")  # 追加新内容并换行
    text_box.text.see("end")  # 滚动到底部

# 在识别结果处理部分使用：
if result_data['result'][0].get('is_final'):
    text = result_data['result'][0].get('text', '')
    print(f"识别结果: {text}")
    update_recognition_text(text)



GUI is cleared because of reinit
建立WebSocket连接...
初始化响应: {'payload_msg': {'addition': {'duration': '0', 'logid': '20241031113334331098D2B276691EB789'}, 'code': 1000, 'message': 'Success', 'reqid': 'd8addb11-7bf3-45bb-9707-396664d9c039', 'sequence': 1}, 'payload_size': 157}
初始化成功
录音任务已启动
开始录音...

程序异常: name 'generate_last_audio_default_header' is not defined
程序已退出


NameError: name 'result_data' is not defined

In [1]:
from flask import Flask, request, send_file, jsonify

app = Flask(__name__)

@app.route('/')
def home():
    return send_file('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    try:
        data = request.get_json()
        message = data.get('message', '')
        print(f"收到消息: {message}")
        return jsonify({"status": "success"})
    except Exception as e:
        print(f"错误: {str(e)}")
        return jsonify({"status": "error", "message": str(e)}), 500

if __name__ == '__main__':
    print("服务器启动在 http://192.168.123.1:5000")
    app.run(host='0.0.0.0', port=5000)

服务器启动在 http://192.168.123.1:5000
 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.16.5.104:5000/ (Press CTRL+C to quit)
192.168.123.101 - - [31/Oct/2024 09:16:09] "GET / HTTP/1.1" 200 -
192.168.123.101 - - [31/Oct/2024 09:16:09] "GET /favicon.ico HTTP/1.1" 404 -


In [5]:
from flask import Flask, request, send_file, jsonify
from difflib import SequenceMatcher
import json
import uuid
import gzip
import asyncio
import websockets
import numpy as np
import sounddevice as sd
import nest_asyncio
from unihiker import GUI
import time
from tkinter import END
import threading

# Flask 应用
app = Flask(__name__)

# 共享数据存储
class SharedData:
    def __init__(self):
        self.speech_recognition_results = []
        self.latest_text = ""

shared_data = SharedData()

# Web 服务器路由
@app.route('/')
def home():
    return send_file('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    try:
        data = request.get_json()
        target_text = data.get('message', '')
        print(f"收到目标文本: {target_text}")

        # 获取最新的语音识别结果
        speech_text = shared_data.latest_text
        
        # 计算相似度和完整度
        similarity = calculate_similarity(target_text, speech_text)
        completeness = calculate_completeness(target_text, speech_text)
        
        response = {
            "status": "success",
            "target_text": target_text,
            "speech_text": speech_text,
            "similarity": round(similarity * 100, 2),
            "completeness": round(completeness * 100, 2),
            "analysis": {
                "similarity_comment": get_similarity_comment(similarity),
                "completeness_comment": get_completeness_comment(completeness)
            }
        }
        return jsonify(response)
    except Exception as e:
        print(f"错误: {str(e)}")
        return jsonify({"status": "error", "message": str(e)}), 500

# 相似度计算函数
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

def calculate_completeness(target_text, speech_text):
    target_words = set(target_text.split())
    speech_words = set(speech_text.split())
    common_words = target_words.intersection(speech_words)
    return len(common_words) / len(target_words) if target_words else 0

def get_similarity_comment(similarity):
    if similarity >= 0.9: return "非常接近"
    elif similarity >= 0.7: return "比较接近"
    elif similarity >= 0.5: return "部分接近"
    else: return "差异较大"

def get_completeness_comment(completeness):
    if completeness >= 0.9: return "内容完整"
    elif completeness >= 0.7: return "大部分完整"
    elif completeness >= 0.5: return "部分完整"
    else: return "内容缺失"

# 配置参数
appid = "4166554764"    # 项目的 appid
token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"    # 项目的 token
cluster = "volcengine_streaming_common"  # 请求的集群

# 协议常量
PROTOCOL_VERSION = 0b0001
DEFAULT_HEADER_SIZE = 0b0001

PROTOCOL_VERSION_BITS = 4
HEADER_BITS = 4
MESSAGE_TYPE_BITS = 4
MESSAGE_TYPE_SPECIFIC_FLAGS_BITS = 4
MESSAGE_SERIALIZATION_BITS = 4
MESSAGE_COMPRESSION_BITS = 4
RESERVED_BITS = 8

# Message Type:
CLIENT_FULL_REQUEST = 0b0001
CLIENT_AUDIO_ONLY_REQUEST = 0b0010
SERVER_FULL_RESPONSE = 0b1001
SERVER_ACK = 0b1011
SERVER_ERROR_RESPONSE = 0b1111

# Message Type Specific Flags
NO_SEQUENCE = 0b0000
POS_SEQUENCE = 0b0001
NEG_SEQUENCE = 0b0010
NEG_SEQUENCE_1 = 0b0011

# Message Serialization
NO_SERIALIZATION = 0b0000
JSON = 0b0001
THRIFT = 0b0011
CUSTOM_TYPE = 0b1111

# Message Compression
NO_COMPRESSION = 0b0000
GZIP = 0b0001
CUSTOM_COMPRESSION = 0b1111




# 初始化GUI
gui = GUI()

# 创建标题
title = gui.draw_text(
    x=120,
    y=30,
    text="语音识别系统",
    font_size=24,
    origin='center'
)

# 创建文本框
text_box = gui.add_text_box(
    x=120,      # 中心x坐标
    y=160,      # 中心y坐标
    w=220,      # 宽度
    h=200,      # 高度
    origin='center',  # 居中对齐
    font_size=14
)

# 存储所有识别文本
all_texts = []

def update_recognition_text(new_text):
    """更新识别结果并保持滚动条在底部"""
    try:
        # 更新共享数据
        shared_data.latest_text = new_text
        shared_data.speech_recognition_results.append(new_text)
        
        # 更新GUI
        all_texts.append(new_text)
        full_text = "\n".join(all_texts)
        text_box.config(text=full_text)
        text_box.text.see(END)
        
        if gui.master.winfo_exists():
            gui.update()
            
    except Exception as e:
        print(f"更新文本错误: {e}")

def generate_header(
    version=PROTOCOL_VERSION,
    message_type=CLIENT_FULL_REQUEST,
    message_type_specific_flags=NO_SEQUENCE,
    serial_method=JSON,
    compression_type=GZIP,
    reserved_data=0x00,
    extension_header=bytes()
):
    header = bytearray()
    header_size = int(len(extension_header) / 4) + 1
    header.append((version << 4) | header_size)
    header.append((message_type << 4) | message_type_specific_flags)
    header.append((serial_method << 4) | compression_type)
    header.append(reserved_data)
    header.extend(extension_header)
    return header

def generate_full_default_header():
    return generate_header()

def generate_audio_default_header():
    return generate_header(
        message_type=CLIENT_AUDIO_ONLY_REQUEST
    )

def parse_response(res):
    """解析响应"""
    try:
        protocol_version = res[0] >> 4
        header_size = res[0] & 0x0f
        message_type = res[1] >> 4
        message_type_specific_flags = res[1] & 0x0f
        serialization_method = res[2] >> 4
        message_compression = res[2] & 0x0f
        reserved = res[3]
        header_extensions = res[4:header_size * 4]
        payload = res[header_size * 4:]
        result = {}
        payload_msg = None
        payload_size = 0
        
        if message_type == SERVER_FULL_RESPONSE:
            payload_size = int.from_bytes(payload[:4], "big", signed=True)
            payload_msg = payload[4:]
        elif message_type == SERVER_ACK:
            seq = int.from_bytes(payload[:4], "big", signed=True)
            result['seq'] = seq
            if len(payload) >= 8:
                payload_size = int.from_bytes(payload[4:8], "big", signed=False)
                payload_msg = payload[8:]
        elif message_type == SERVER_ERROR_RESPONSE:
            code = int.from_bytes(payload[:4], "big", signed=False)
            result['code'] = code
            payload_size = int.from_bytes(payload[4:8], "big", signed=False)
            payload_msg = payload[8:]
            
        if payload_msg is None:
            return result
            
        if message_compression == GZIP:
            payload_msg = gzip.decompress(payload_msg)
            
        if serialization_method == JSON:
            payload_msg = json.loads(str(payload_msg, "utf-8"))
        elif serialization_method != NO_SERIALIZATION:
            payload_msg = str(payload_msg, "utf-8")
            
        result['payload_msg'] = payload_msg
        result['payload_size'] = payload_size
        return result
    except Exception as e:
        return {"error": f"Failed to parse response: {str(e)}"}

# 添加 AsrWsClient 类定义
class AsrWsClient:
    def __init__(self, appid, token, cluster):
        self.appid = appid
        self.token = token
        self.cluster = cluster
        self.ws_url = "wss://openspeech.bytedance.com/api/v2/asr"
        self.success_code = 1000
        self.uid = "streaming_asr_demo"
        self.workflow = "audio_in,resample,partition,vad,fe,decode,itn,nlu_punctuate"
        self.show_language = False
        self.show_utterances = True
        self.result_type = "single"
        self.format = "raw"
        self.rate = 16000
        self.language = "zh-CN"
        self.bits = 16
        self.channel = 1
        self.codec = "raw"
        self.auth_method = "token"

    def construct_request(self, reqid):
        return {
            'app': {
                'appid': self.appid,
                'cluster': self.cluster,
                'token': self.token,
            },
            'user': {
                'uid': self.uid
            },
            'request': {
                'reqid': reqid,
                'nbest': 1,
                'workflow': self.workflow,
                'show_language': self.show_language,
                'show_utterances': self.show_utterances,
                'result_type': self.result_type,
                'sequence': 1
            },
            'audio': {
                'format': self.format,
                'rate': self.rate,
                'language': self.language,
                'bits': self.bits,
                'channel': self.channel,
                'codec': self.codec
            }
        }

    def token_auth(self):
        return {'Authorization': f'Bearer; {self.token}'}

    async def process_microphone(self):
        """实时麦克风录音并识别"""
        reqid = str(uuid.uuid4())
        request_params = self.construct_request(reqid)
        
        # 构造初始请求
        payload_bytes = str.encode(json.dumps(request_params))
        payload_bytes = gzip.compress(payload_bytes)
        full_request = bytearray(generate_full_default_header())
        full_request.extend(len(payload_bytes).to_bytes(4, 'big'))
        full_request.extend(payload_bytes)

        print("建立WebSocket连接...")
        async with websockets.connect(
            self.ws_url, 
            extra_headers=self.token_auth(), 
            max_size=1000000000
        ) as ws:
            # 发送初始请求
            await ws.send(full_request)
            response = await ws.recv()
            result = parse_response(response)
            print(f"初始化响应: {result}")
            
            if 'payload_msg' in result and result['payload_msg']['code'] == self.success_code:
                print("初始化成功")
                print("录音任务已启动")
                chunk_size = 9600  # 每次读取的采样点数
                
                with sd.InputStream(
                    channels=1, 
                    samplerate=16000,
                    dtype=np.int16,
                    blocksize=chunk_size,
                    callback=None
                ) as stream:
                    print("开始录音...")
                    try:
                        while True:
                            # 读取音频
                            audio_data, overflowed = stream.read(chunk_size)
                            if overflowed:
                                print("警告：音频缓冲区溢出")
                                
                            # 转换为字节
                            audio_bytes = audio_data.tobytes()
                            
                            # 压缩音频数据
                            compressed_audio = gzip.compress(audio_bytes)
                            
                            # 构造音频数据请求
                            audio_request = bytearray(generate_audio_default_header())
                            audio_request.extend(len(compressed_audio).to_bytes(4, 'big'))
                            audio_request.extend(compressed_audio)
                            
                            # 发送音频数据
                            await ws.send(audio_request)
                            
                            # 接收识别结果
                            response = await ws.recv()
                            result = parse_response(response)
                            
                            # 处理识别结果
                            if 'payload_msg' in result and 'result' in result['payload_msg']:
                                utterances = result['payload_msg']['result'][0].get('utterances', [])
                                for utterance in utterances:
                                    if not utterance['definite']:
                                        print(f"\r[识别中...] {utterance['text']}", end='', flush=True)
                                    else:
                                        print(f"\n[最终结果] {utterance['text']}")
                                        update_recognition_text(utterance['text'])
                    except KeyboardInterrupt:
                        print("\n录音已停止")
                    except Exception as e:
                        print(f"录音过程发生错误: {e}")
            else:
                print(f"初始化失败: {result['payload_msg'].get('message')}")

# 在notebook中运行异步代码的辅助函数
import nest_asyncio
nest_asyncio.apply()

# 启动 Flask 服务器的函数
def run_flask():
    app.run(host='0.0.0.0', port=5000)

# 主程序
if __name__ == '__main__':
    # 在单独的线程中启动 Flask 服务器
    flask_thread = threading.Thread(target=run_flask)
    flask_thread.daemon = True  # 设置为守护线程
    flask_thread.start()
    
    print("Web服务器启动在 http://192.168.123.1:5000")
    
    # 创建语音识别客户端
    client = AsrWsClient(appid, token, cluster)
    
    try:
        # 启动语音识别
        loop = asyncio.get_event_loop()
        loop.run_until_complete(client.process_microphone())
    except KeyboardInterrupt:
        print("\n程序已被用户中断")
    except Exception as e:
        print(f"\n程序异常: {e}")
    finally:
        print("程序已退出")


GUI is cleared because of reinit
Web服务器启动在 http://192.168.123.1:5000
 * Serving Flask app '__main__' (lazy loading)
建立WebSocket连接...
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_1163/2647346817.py", line 383, in run_flask
    app.run(host='0.0.0.0', port=5000)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 920, in run
    run_simple(t.cast(str, host), port, self, **options)
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/serving.py", line 1010, in run_simple
    inner()
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/serving.py", line 959, in inner
    fd=fd,
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/serving.py", line 783, in make_server
    host, port, app, request_handler, passthrough_errors, ssl_context, fd=fd
  File "/usr/local/lib/python3.7/dist-packages/werkzeug/serving.py", line 688, in __init__
    super().__init__(server_addre

初始化响应: {'payload_msg': {'addition': {'duration': '0', 'logid': '20241031093415CC2B4FBA055BAE1B3789'}, 'code': 1000, 'message': 'Success', 'reqid': '221b3a71-f17c-462c-916a-bdb51ed0ac4d', 'sequence': 1}, 'payload_size': 159}
初始化成功
录音任务已启动
开始录音...
[识别中...] 大力又有人学又要人人心对是的所以我想的现在无论如何先我明天给明天不管怎么样我要要哈
[最终结果] 大力又有人学又有人人心。对，是的，所以我想着现在无论如何先我明天明天不管怎么样我要要考哈哈，不行。
[识别中...] 我现在就去检查了
[最终结果] 我现在就去检查了哦，嗯。
[识别中...] 
[最终结果] 原来是张老师！
[识别中...] 哈你今天啊你今天体验了一把掌鼓而且我现在知道什么他其实就是把他们嗯
[最终结果] 哈？你今天哦，你今天体验了一把掌控。而且我现在知道什么他其实就是把他们嗯当。
[识别中...] 试一下呀哈哈哈哦对了那个第四代原因的反面有点放心嗯我骑空
[最终结果] 试一下呀，哈哈，那男的连声都不敢，哦对了，那个第四代原因的反对，有点放心哦。
[识别中...] 前面没有是不是哦你一直会炒这个吧
[最终结果] 这个还有没有反应啊？这个就是前面没有是不哦，你一直会炒这个吧？
[识别中...] 我把这个通过应该挺好
[最终结果] 我把这个通过应该挺好。
[识别中...] 是考试干什么做啊
[最终结果] 这考试干什么呀？做啊！
[识别中...] 对不起哦姐
[最终结果] 对不起哦！
[识别中...] 不是我现在在想法因为是这样的没有别的没只有一份
[最终结果] 不是，我现在在想法，因为是这样的，没有别的美丽，只有一份。
[识别中...] 哦这是哦那完了那完那我发错了
[最终结果] 哦，这是哦那完了那我那我发错了。
[识别中...] 这个这个是叫我没有发我把这个格式做了但是
[最终结果] 这个这个机器叫我没有发，我把这个给他做了。
[识别中...] 
[最终结果] 
[识别中...] 下午的告诉我们
[最终结果] 下午的告诉我们干什么？
[识别中...] 啊

In [ ]:
from flask import Flask, request, send_file, jsonify
from difflib import SequenceMatcher
import json
import uuid
import gzip
import asyncio
import websockets
import numpy as np
import sounddevice as sd
import nest_asyncio
from unihiker import GUI
import time
from tkinter import END
import threading
import socket
import os
import signal

# Flask 应用
app = Flask(__name__)

# 共享数据存储
class SharedData:
    def __init__(self):
        self.speech_recognition_results = []
        self.latest_text = ""

shared_data = SharedData()

# Web 服务器路由
@app.route('/')
def home():
    return send_file('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    try:
        data = request.get_json()
        target_text = data.get('message', '')
        print(f"收到目标文本: {target_text}")

        # 获取最新的语音识别结果
        speech_text = shared_data.latest_text
        
        # 计算相似度和完整度
        similarity = calculate_similarity(target_text, speech_text)
        completeness = calculate_completeness(target_text, speech_text)
        
        response = {
            "status": "success",
            "target_text": target_text,
            "speech_text": speech_text,
            "similarity": round(similarity * 100, 2),
            "completeness": round(completeness * 100, 2),
            "analysis": {
                "similarity_comment": get_similarity_comment(similarity),
                "completeness_comment": get_completeness_comment(completeness)
            }
        }
        return jsonify(response)
    except Exception as e:
        print(f"错误: {str(e)}")
        return jsonify({"status": "error", "message": str(e)}), 500

# 相似度计算函数
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

def calculate_completeness(target_text, speech_text):
    target_words = set(target_text.split())
    speech_words = set(speech_text.split())
    common_words = target_words.intersection(speech_words)
    return len(common_words) / len(target_words) if target_words else 0

def get_similarity_comment(similarity):
    if similarity >= 0.9: return "非常接近"
    elif similarity >= 0.7: return "比较接近"
    elif similarity >= 0.5: return "部分接近"
    else: return "差异较大"

def get_completeness_comment(completeness):
    if completeness >= 0.9: return "内容完整"
    elif completeness >= 0.7: return "大部分完整"
    elif completeness >= 0.5: return "部分完整"
    else: return "内容缺失"

# 配置参数
appid = "4166554764"    # 项目的 appid
token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"    # 项目的 token
cluster = "volcengine_streaming_common"  # 请求的集群

# 协议常量
PROTOCOL_VERSION = 0b0001
DEFAULT_HEADER_SIZE = 0b0001

PROTOCOL_VERSION_BITS = 4
HEADER_BITS = 4
MESSAGE_TYPE_BITS = 4
MESSAGE_TYPE_SPECIFIC_FLAGS_BITS = 4
MESSAGE_SERIALIZATION_BITS = 4
MESSAGE_COMPRESSION_BITS = 4
RESERVED_BITS = 8

# Message Type:
CLIENT_FULL_REQUEST = 0b0001
CLIENT_AUDIO_ONLY_REQUEST = 0b0010
SERVER_FULL_RESPONSE = 0b1001
SERVER_ACK = 0b1011
SERVER_ERROR_RESPONSE = 0b1111

# Message Type Specific Flags
NO_SEQUENCE = 0b0000
POS_SEQUENCE = 0b0001
NEG_SEQUENCE = 0b0010
NEG_SEQUENCE_1 = 0b0011

# Message Serialization
NO_SERIALIZATION = 0b0000
JSON = 0b0001
THRIFT = 0b0011
CUSTOM_TYPE = 0b1111

# Message Compression
NO_COMPRESSION = 0b0000
GZIP = 0b0001
CUSTOM_COMPRESSION = 0b1111




# 初始化GUI
gui = GUI()

# 创建标题
title = gui.draw_text(
    x=120,
    y=30,
    text="语音识别系统",
    font_size=24,
    origin='center'
)

# 创建文本框
text_box = gui.add_text_box(
    x=120,      # 中心x坐标
    y=160,      # 中心y坐标
    w=220,      # 宽度
    h=200,      # 高度
    origin='center',  # 居中对齐
    font_size=14
)

# 存储所有识别文本
all_texts = []

def update_recognition_text(new_text):
    """更新识别结果并保持滚动条在底部"""
    try:
        # 更新共享数据
        shared_data.latest_text = new_text
        shared_data.speech_recognition_results.append(new_text)
        
        # 更新GUI
        all_texts.append(new_text)
        full_text = "\n".join(all_texts)
        text_box.config(text=full_text)
        text_box.text.see(END)
        
        if gui.master.winfo_exists():
            gui.update()
            
    except Exception as e:
        print(f"更新文本错误: {e}")

def generate_header(
    version=PROTOCOL_VERSION,
    message_type=CLIENT_FULL_REQUEST,
    message_type_specific_flags=NO_SEQUENCE,
    serial_method=JSON,
    compression_type=GZIP,
    reserved_data=0x00,
    extension_header=bytes()
):
    header = bytearray()
    header_size = int(len(extension_header) / 4) + 1
    header.append((version << 4) | header_size)
    header.append((message_type << 4) | message_type_specific_flags)
    header.append((serial_method << 4) | compression_type)
    header.append(reserved_data)
    header.extend(extension_header)
    return header

def generate_full_default_header():
    return generate_header()

def generate_audio_default_header():
    return generate_header(
        message_type=CLIENT_AUDIO_ONLY_REQUEST
    )

def parse_response(res):
    """解析响应"""
    try:
        protocol_version = res[0] >> 4
        header_size = res[0] & 0x0f
        message_type = res[1] >> 4
        message_type_specific_flags = res[1] & 0x0f
        serialization_method = res[2] >> 4
        message_compression = res[2] & 0x0f
        reserved = res[3]
        header_extensions = res[4:header_size * 4]
        payload = res[header_size * 4:]
        result = {}
        payload_msg = None
        payload_size = 0
        
        if message_type == SERVER_FULL_RESPONSE:
            payload_size = int.from_bytes(payload[:4], "big", signed=True)
            payload_msg = payload[4:]
        elif message_type == SERVER_ACK:
            seq = int.from_bytes(payload[:4], "big", signed=True)
            result['seq'] = seq
            if len(payload) >= 8:
                payload_size = int.from_bytes(payload[4:8], "big", signed=False)
                payload_msg = payload[8:]
        elif message_type == SERVER_ERROR_RESPONSE:
            code = int.from_bytes(payload[:4], "big", signed=False)
            result['code'] = code
            payload_size = int.from_bytes(payload[4:8], "big", signed=False)
            payload_msg = payload[8:]
            
        if payload_msg is None:
            return result
            
        if message_compression == GZIP:
            payload_msg = gzip.decompress(payload_msg)
            
        if serialization_method == JSON:
            payload_msg = json.loads(str(payload_msg, "utf-8"))
        elif serialization_method != NO_SERIALIZATION:
            payload_msg = str(payload_msg, "utf-8")
            
        result['payload_msg'] = payload_msg
        result['payload_size'] = payload_size
        return result
    except Exception as e:
        return {"error": f"Failed to parse response: {str(e)}"}

# 添加 AsrWsClient 类定义
class AsrWsClient:
    def __init__(self, appid, token, cluster):
        self.appid = appid
        self.token = token
        self.cluster = cluster
        self.ws_url = "wss://openspeech.bytedance.com/api/v2/asr"
        self.success_code = 1000
        self.uid = "streaming_asr_demo"
        self.workflow = "audio_in,resample,partition,vad,fe,decode,itn,nlu_punctuate"
        self.show_language = False
        self.show_utterances = True
        self.result_type = "single"
        self.format = "raw"
        self.rate = 16000
        self.language = "zh-CN"
        self.bits = 16
        self.channel = 1
        self.codec = "raw"
        self.auth_method = "token"

    def construct_request(self, reqid):
        return {
            'app': {
                'appid': self.appid,
                'cluster': self.cluster,
                'token': self.token,
            },
            'user': {
                'uid': self.uid
            },
            'request': {
                'reqid': reqid,
                'nbest': 1,
                'workflow': self.workflow,
                'show_language': self.show_language,
                'show_utterances': self.show_utterances,
                'result_type': self.result_type,
                'sequence': 1
            },
            'audio': {
                'format': self.format,
                'rate': self.rate,
                'language': self.language,
                'bits': self.bits,
                'channel': self.channel,
                'codec': self.codec
            }
        }

    def token_auth(self):
        return {'Authorization': f'Bearer; {self.token}'}

    async def process_microphone(self):
        """实时麦克风录音并识别"""
        reqid = str(uuid.uuid4())
        request_params = self.construct_request(reqid)
        
        # 构造初始请求
        payload_bytes = str.encode(json.dumps(request_params))
        payload_bytes = gzip.compress(payload_bytes)
        full_request = bytearray(generate_full_default_header())
        full_request.extend(len(payload_bytes).to_bytes(4, 'big'))
        full_request.extend(payload_bytes)

        print("建立WebSocket连接...")
        async with websockets.connect(
            self.ws_url, 
            extra_headers=self.token_auth(), 
            max_size=1000000000
        ) as ws:
            # 发送初始请求
            await ws.send(full_request)
            response = await ws.recv()
            result = parse_response(response)
            print(f"初始化响应: {result}")
            
            if 'payload_msg' in result and result['payload_msg']['code'] == self.success_code:
                print("初始化成功")
                print("录音任务已启动")
                chunk_size = 9600  # 每次读取的采样点数
                
                with sd.InputStream(
                    channels=1, 
                    samplerate=16000,
                    dtype=np.int16,
                    blocksize=chunk_size,
                    callback=None
                ) as stream:
                    print("开始录音...")
                    try:
                        while True:
                            # 读取音频
                            audio_data, overflowed = stream.read(chunk_size)
                            if overflowed:
                                print("警告：音频缓冲区溢出")
                                
                            # 转换为字节
                            audio_bytes = audio_data.tobytes()
                            
                            # 压缩音频数据
                            compressed_audio = gzip.compress(audio_bytes)
                            
                            # 构造音频数据请求
                            audio_request = bytearray(generate_audio_default_header())
                            audio_request.extend(len(compressed_audio).to_bytes(4, 'big'))
                            audio_request.extend(compressed_audio)
                            
                            # 发送音频数据
                            await ws.send(audio_request)
                            
                            # 接收识别结果
                            response = await ws.recv()
                            result = parse_response(response)
                            
                            # 处理识别结果
                            if 'payload_msg' in result and 'result' in result['payload_msg']:
                                utterances = result['payload_msg']['result'][0].get('utterances', [])
                                for utterance in utterances:
                                    if not utterance['definite']:
                                        print(f"\r[识别中...] {utterance['text']}", end='', flush=True)
                                    else:
                                        print(f"\n[最终结果] {utterance['text']}")
                                        update_recognition_text(utterance['text'])
                    except KeyboardInterrupt:
                        print("\n录音已停止")
                    except Exception as e:
                        print(f"录音过程发生错误: {e}")
            else:
                print(f"初始化失败: {result['payload_msg'].get('message')}")

# 在notebook中运行异步代码的辅助函数
import nest_asyncio
nest_asyncio.apply()

def check_and_kill_process_on_port(port):
    try:
        # 尝试获取使用该端口的进程 PID
        cmd = f"lsof -i :{port} -t"
        pid = os.popen(cmd).read().strip()
        
        if pid:
            print(f"正在释放端口 {port}...")
            os.system(f"kill -9 {pid}")
            time.sleep(1)  # 等待进程结束
            return True
    except:
        pass
    return False

# 启动 Flask 服务器的函数
def run_flask():
    port = 5000
    max_retries = 3
    retry_count = 0
    
    while retry_count < max_retries:
        try:
            app.run(host='0.0.0.0', port=port)
            break
        except OSError as e:
            if e.errno == 98:  # Address already in use
                print(f"端口 {port} 已被占用")
                if check_and_kill_process_on_port(port):
                    print(f"端口 {port} 已释放")
                    retry_count += 1
                    continue
                else:
                    print(f"无法释放端口 {port}")
                    port += 1  # 尝试下一个端口
            else:
                raise e

# 主程序
if __name__ == '__main__':
    try:
        # 初始化GUI
        gui = GUI()
        
        # 创建标题和文本框
        title = gui.draw_text(
            x=120,
            y=30,
            text="语音识别系统",
            font_size=24,
            origin='center'
        )
        
        text_box = gui.add_text_box(
            x=120,
            y=160,
            w=220,
            h=200,
            origin='center',
            font_size=14
        )
        
        # 存储所有识别文本
        all_texts = []
        
        # 在单独的线程中启动 Flask 服务器
        flask_thread = threading.Thread(target=run_flask)
        flask_thread.daemon = True
        flask_thread.start()
        
        # 创建语音识别客户端
        client = AsrWsClient(appid, token, cluster)
        
        # 启动语音识别
        loop = asyncio.get_event_loop()
        loop.run_until_complete(client.process_microphone())
        
    except KeyboardInterrupt:
        print("\n程序已被用户中断")
    except Exception as e:
        print(f"\n程序异常: {e}")
    finally:
        # 清理资源
        try:
            loop.close()
        except:
            pass
        print("程序已退出")


In [ ]:
上面的代码实现了话筒拾音，流式API语音识别，大模型校对语音识别结果，在unihiker端实时显示文本，教师端显示发送范文。问题：用的是DIFFLIB比对范文和识别结果。

In [12]:

                            # 读取音频
                            audio_data, overflowed = stream.read(chunk_size)
                            if overflowed:
                                print("警告：音频缓冲区溢出")
                                
                            # 转换为字节
                            audio_bytes = audio_data.tobytes()
                            
                            # 压缩音频数据
                            compressed_audio = gzip.compress(audio_bytes)
                            
                            # 构造音频数据请求
                            audio_request = bytearray(generate_audio_default_header())
                            audio_request.extend(len(compressed_audio).to_bytes(4, 'big'))
                            audio_request.extend(compressed_audio)
                            
                            # 发送音频数据
                            await ws.send(audio_request)
                            
                            # 接收识别结果
                            response = await ws.recv()
                            result = parse_response(response)
                            
                            # 处理识别结果
                            if 'payload_msg' in result and 'result' in result['payload_msg']:
                                utterances = result['payload_msg']['result'][0].get('utterances', [])
                                for utterance in utterances:
                                    if not utterance['definite']:
                                        print(f"\r[识别中...] {utterance['text']}", end='', flush=True)
                                    else:
                                        print(f"\n[最终结果] {utterance['text']}")
                                        update_recognition_text(utterance['text'])
                    except KeyboardInterrupt:
                        print("\n录音已停止")
                    except Exception as e:
                        print(f"录音过程发生错误: {e}")
            else:
                print(f"初始化失败: {result['payload_msg'].get('message')}")

# 在notebook中运行异步代码的辅助函数
import nest_asyncio
nest_asyncio.apply()

def check_and_kill_process_on_port(port):
    try:
        # 尝试获取使用该端口的进程 PID
        cmd = f"lsof -i :{port} -t"
        pid = os.popen(cmd).read().strip()
        
        if pid:
            print(f"正在释放端口 {port}...")
            os.system(f"kill -9 {pid}")
            time.sleep(1)  # 等待进程结束
            return True
    except:
        pass
    return False

# 启动 Flask 服务器的函数
def run_flask():
    port = 5000
    max_retries = 3
    retry_count = 0
    
    while retry_count < max_retries:
        try:
            app.run(host='0.0.0.0', port=port)
            break
        except OSError as e:
            if e.errno == 98:  # Address already in use
                print(f"端口 {port} 已被占用")
                if check_and_kill_process_on_port(port):
                    print(f"端口 {port} 已释放")
                    retry_count += 1
                    continue
                else:
                    print(f"无法释放端口 {port}")
                    port += 1  # 尝试下一个端口
            else:
                raise e

# 添加语言模型请求函数
async def correct_text_with_llm(text):
    try:
        # 构造 WebSocket 请求
        ws_url = "wss://openspeech.bytedance.com/api/v2/chat"
        
        async with websockets.connect(ws_url, extra_headers={'Authorization': f'Bearer; {token}'}) as ws:
            # 构造请求
            request = {
                'text': text,
                'task': 'text_correction',
                'prompt': '请纠正这段语音识别文本中可能的错误，包括错别字、标点符号等，保持原意。'
            }
            
            # 发送请求
            await ws.send(json.dumps(request))
            
            # 接收响应
            response = await ws.recv()
            result = json.loads(response)
            
            return result.get('corrected_text', text)
    except Exception as e:
        print(f"文本纠正错误: {e}")
        return text

# 添加按钮点击处理函数
def on_correct_button_pressed():
    try:
        # 获取最新的识别文本
        current_text = all_texts[-1] if all_texts else ""
        if not current_text:
            print("没有可纠正的文本")
            return

        print("正在纠正文本...")
        
        # 添加提示信息
        update_recognition_text("[正在进行文本校验...]")
        
        # 这里可以添加实际的文本纠正逻辑
        corrected_text = f"[校验结果] {current_text}"
        
        # 显示纠正后的文本
        update_recognition_text(corrected_text)
        
    except Exception as e:
        print(f"纠正按钮错误: {e}")
        update_recognition_text(f"[错误] 文本校验失败: {str(e)}")

# 主程序
if __name__ == '__main__':
    try:
        # 初始化GUI
        gui = GUI()
        
        # 创建标题
        title = gui.draw_text(
            x=120,
            y=30,
            text="语音识别系统",
            font_size=24,
            origin='center'
        )
        
        # 创建文本框
        text_box = gui.add_text_box(
            x=120,
            y=160,
            w=220,
            h=200,
            origin='center',
            font_size=14
        )
        
        # 添加校验按钮
        correct_button = gui.add_button(
            x=120,
            y=300,
            w=100,
            h=40,
            text="文本校验",
            origin='center'
        )
        # 使用 when_pressed 绑定事件
        correct_button.when_pressed(on_correct_button_pressed)
        
        # 存储所有识别文本
        all_texts = []
        
        # 在单独的线程中启动 Flask 服务器
        flask_thread = threading.Thread(target=run_flask)
        flask_thread.daemon = True
        flask_thread.start()
        
        print("Web服务器启动在 http://192.168.123.1:5000")
        
        # 创建语音识别客户端
        client = AsrWsClient(appid, token, cluster)
        
        # 启动语音识别
        loop = asyncio.get_event_loop()
        loop.run_until_complete(client.process_microphone())
        
    except KeyboardInterrupt:
        print("\n程序已被用户中断")
    except Exception as e:
        print(f"\n程序异常: {e}")
    finally:
        print("程序已退出")


GUI is cleared because of reinit


AttributeError: 'WidgetButton' object has no attribute 'when_pressed'

In [15]:
import json
import uuid
import gzip
import asyncio
import websockets
import numpy as np
import sounddevice as sd
import nest_asyncio
from unihiker import GUI
import time
from tkinter import END
import requests
import os
import dashscope
from flask import Flask, request, jsonify, send_file

# 配置参数
appid = "4166554764"    # 项目的 appid
token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"    # 项目的 token
cluster = "volcengine_streaming_common"  # 请求的集群

# 协议常量
PROTOCOL_VERSION = 0b0001
DEFAULT_HEADER_SIZE = 0b0001

PROTOCOL_VERSION_BITS = 4
HEADER_BITS = 4
MESSAGE_TYPE_BITS = 4
MESSAGE_TYPE_SPECIFIC_FLAGS_BITS = 4
MESSAGE_SERIALIZATION_BITS = 4
MESSAGE_COMPRESSION_BITS = 4
RESERVED_BITS = 8

# Message Type:
CLIENT_FULL_REQUEST = 0b0001
CLIENT_AUDIO_ONLY_REQUEST = 0b0010
SERVER_FULL_RESPONSE = 0b1001
SERVER_ACK = 0b1011
SERVER_ERROR_RESPONSE = 0b1111

# Message Type Specific Flags
NO_SEQUENCE = 0b0000
POS_SEQUENCE = 0b0001
NEG_SEQUENCE = 0b0010
NEG_SEQUENCE_1 = 0b0011

# Message Serialization
NO_SERIALIZATION = 0b0000
JSON = 0b0001
THRIFT = 0b0011
CUSTOM_TYPE = 0b1111

# Message Compression
NO_COMPRESSION = 0b0000
GZIP = 0b0001
CUSTOM_COMPRESSION = 0b1111

# 添加一个全局标志来控制语音识别
is_recording = True

# 初始化GUI
gui = GUI()

# 创建标题
title = gui.draw_text(
    x=120,
    y=20,
    text="背诵助手",
    font_size=20,
    origin='center'
)

# 创建文本框
text_box = gui.add_text_box(
    x=120,
    y=140,
    w=220,
    h=180,
    origin='center',
    font_size=12
)

# 创建纠正文本按钮
correction_button = gui.add_button(
    x=120,
    y=270,
    w=160,
    h=36,
    text="纠正文本",
    origin='center'
)

def generate_header(
    version=PROTOCOL_VERSION,
    message_type=CLIENT_FULL_REQUEST,
    message_type_specific_flags=NO_SEQUENCE,
    serial_method=JSON,
    compression_type=GZIP,
    reserved_data=0x00,
    extension_header=bytes()
):
    header = bytearray()
    header_size = int(len(extension_header) / 4) + 1
    header.append((version << 4) | header_size)
    header.append((message_type << 4) | message_type_specific_flags)
    header.append((serial_method << 4) | compression_type)
    header.append(reserved_data)
    header.extend(extension_header)
    return header

def generate_full_default_header():
    return generate_header()

def generate_audio_default_header():
    return generate_header(
        message_type=CLIENT_AUDIO_ONLY_REQUEST
    )

def parse_response(res):
    """解析响应"""
    try:
        protocol_version = res[0] >> 4
        header_size = res[0] & 0x0f
        message_type = res[1] >> 4
        message_type_specific_flags = res[1] & 0x0f
        serialization_method = res[2] >> 4
        message_compression = res[2] & 0x0f
        reserved = res[3]
        header_extensions = res[4:header_size * 4]
        payload = res[header_size * 4:]
        result = {}
        payload_msg = None
        payload_size = 0
        
        if message_type == SERVER_FULL_RESPONSE:
            payload_size = int.from_bytes(payload[:4], "big", signed=True)
            payload_msg = payload[4:]
        elif message_type == SERVER_ACK:
            seq = int.from_bytes(payload[:4], "big", signed=True)
            result['seq'] = seq
            if len(payload) >= 8:
                payload_size = int.from_bytes(payload[4:8], "big", signed=False)
                payload_msg = payload[8:]
        elif message_type == SERVER_ERROR_RESPONSE:
            code = int.from_bytes(payload[:4], "big", signed=False)
            result['code'] = code
            payload_size = int.from_bytes(payload[4:8], "big", signed=False)
            payload_msg = payload[8:]
            
        if payload_msg is None:
            return result
            
        if message_compression == GZIP:
            payload_msg = gzip.decompress(payload_msg)
            
        if serialization_method == JSON:
            payload_msg = json.loads(str(payload_msg, "utf-8"))
        elif serialization_method != NO_SERIALIZATION:
            payload_msg = str(payload_msg, "utf-8")
            
        result['payload_msg'] = payload_msg
        result['payload_size'] = payload_size
        return result
    except Exception as e:
        return {"error": f"Failed to parse response: {str(e)}"}

class AsrWsClient:
    def __init__(self, appid, token, cluster):
        self.appid = appid
        self.token = token
        self.cluster = cluster
        self.ws_url = "wss://openspeech.bytedance.com/api/v2/asr"
        self.success_code = 1000
        self.uid = "streaming_asr_demo"
        self.workflow = "audio_in,resample,partition,vad,fe,decode,itn,nlu_punctuate"
        self.show_language = False
        self.show_utterances = True
        self.result_type = "single"
        self.format = "raw"
        self.rate = 16000
        self.language = "zh-CN"
        self.bits = 16
        self.channel = 1
        self.codec = "raw"
        self.auth_method = "token"

    def construct_request(self, reqid):
        return {
            'app': {
                'appid': self.appid,
                'cluster': self.cluster,
                'token': self.token,
            },
            'user': {
                'uid': self.uid
            },
            'request': {
                'reqid': reqid,
                'nbest': 1,
                'workflow': self.workflow,
                'show_language': self.show_language,
                'show_utterances': self.show_utterances,
                'result_type': self.result_type,
                'sequence': 1
            },
            'audio': {
                'format': self.format,
                'rate': self.rate,
                'language': self.language,
                'bits': self.bits,
                'channel': self.channel,
                'codec': self.codec
            }
        }

    def token_auth(self):
        return {'Authorization': f'Bearer; {self.token}'}

    async def process_microphone(self):
        """实时麦克风录音并识别"""
        global is_recording
        is_recording = True
        
        reqid = str(uuid.uuid4())
        request_params = self.construct_request(reqid)
        
        # 构造初始请求
        payload_bytes = str.encode(json.dumps(request_params))
        payload_bytes = gzip.compress(payload_bytes)
        full_request = bytearray(generate_full_default_header())
        full_request.extend(len(payload_bytes).to_bytes(4, 'big'))
        full_request.extend(payload_bytes)

        print("建立WebSocket连接...")
        async with websockets.connect(
            self.ws_url, 
            extra_headers=self.token_auth(), 
            max_size=1000000000
        ) as ws:
            # 发送初始请求
            await ws.send(full_request)
            response = await ws.recv()
            result = parse_response(response)
            print(f"初始化响应: {result}")
            
            if 'payload_msg' in result and result['payload_msg']['code'] == self.success_code:
                print("初始化成功")
                print("录音任务已启动")
                chunk_size = 9600
                
                with sd.InputStream(channels=1, samplerate=16000, dtype=np.int16, blocksize=chunk_size) as stream:
                    print("开始录音...")
                    try:
                        while is_recording:
                            audio_data, overflowed = stream.read(chunk_size)
                            if overflowed:
                                print("警告：音频缓冲区溢出")
                                
                            # 转换为字节
                            audio_bytes = audio_data.tobytes()
                            
                            # 压缩音频数据
                            compressed_audio = gzip.compress(audio_bytes)
                            
                            # 构造音频数据请求
                            audio_request = bytearray(generate_audio_default_header())
                            audio_request.extend(len(compressed_audio).to_bytes(4, 'big'))
                            audio_request.extend(compressed_audio)
                            
                            # 发送音频数据
                            await ws.send(audio_request)
                            
                            # 接收识别结果
                            response = await ws.recv()
                            result = parse_response(response)
                            
                            # 处理识别结果
                            if 'payload_msg' in result and 'result' in result['payload_msg']:
                                utterances = result['payload_msg']['result'][0].get('utterances', [])
                                for utterance in utterances:
                                    if not utterance['definite']:
                                        print(f"\r[识别中...] {utterance['text']}", end='', flush=True)
                                    else:
                                        print(f"\n[最终结果] {utterance['text']}")
                                        update_recognition_text(utterance['text'])
                    except KeyboardInterrupt:
                        print("\n录音已停止")
                    except Exception as e:
                        print(f"录音过程发生错误: {e}")
            else:
                print(f"初始化失败: {result['payload_msg'].get('message')}")

def update_recognition_text(new_text, is_correction=False):
    """更新识别结果并保持滚动条在底部"""
    try:
        prefix = "[纠正结果] " if is_correction else ""
        text_box.text.insert(END, prefix + new_text + "\n")
        text_box.text.see(END)
        gui.update()
    except Exception as e:
        print(f"更新文本错误: {e}")

def call_qwen_api(text):
    """调用通义千问API进行文本纠正"""
    try:
        print(f"开始调用API，输入文本：{text}")
        messages = [
            {'role': 'system', 'content': 'You are a helpful assistant.'},
            {'role': 'user', 'content': f'下面的文字是语音识别后的结果，请剔除不必要的标点符号，替换错误的文字，最终返回正确的结果。语音识别文字如下：{text}'}
        ]
        
        response = dashscope.Generation.call(
            api_key=os.getenv('DASHSCOPE_API_KEY'),
            model="qwen-plus",
            messages=messages,
            result_format='message'
        )
        
        if response.status_code == 200:
            result = response.output.choices[0].message.content
            print(f"API调用成功，返回结果：{result}")
            return result
        else:
            error_msg = f"API调用失败: {response.message}"
            print(error_msg)
            return error_msg
    except Exception as e:
        error_msg = f"API调用错误: {str(e)}"
        print(error_msg)
        return error_msg

def on_correction_click():
    """处理纠正按钮点击事件"""
    global is_recording
    try:
        print("纠正按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)  # 给一点时间让录音线程完全停止
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的当前文本：{current_text}")
        
        if current_text.strip():
            # 3. 调用API进行纠正
            print("开始调用API进行纠正...")
            corrected_text = call_qwen_api(current_text)
            print(f"API返回的纠正结果：{corrected_text}")
            
            # 4. 显示纠正结果
            update_recognition_text(corrected_text, is_correction=True)
            print("更新显示完成")
        else:
            print("文本框为空，不进行API调用")
            
    except Exception as e:
        error_msg = f"纠正文本错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg, is_correction=True)

# 绑定纠正按钮点击事件
correction_button.config(command=on_correction_click)

# 设置API密钥
os.environ['DASHSCOPE_API_KEY'] = 'sk-7c04ee6f9432492bb344baa7a5c0162f'

# 在notebook中运行异步代码的辅助函数
import nest_asyncio
nest_asyncio.apply()

# 创建客户端实例并运行
client = AsrWsClient(appid, token, cluster)
try:
    # 创建一个新的事件循环
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    
    # 启动语音识别任务
    recognition_task = loop.create_task(client.process_microphone())
    
    # 主循环
    while True:
        try:
            loop.run_until_complete(asyncio.sleep(0.1))
            gui.update()
            
        except KeyboardInterrupt:
            break
            
except Exception as e:
    print(f"\n程序异常: {e}")
finally:
    loop.close()
    print("程序已退出")

# 添加 Flask 应用
app = Flask(__name__)

# Web 服务器路由
@app.route('/')
def home():
    return send_file('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    try:
        data = request.get_json()
        message = data.get('message', '')
        print(f"收到目标文本: {message}")
        
        # 获取最新的语音识别结果
        speech_text = shared_data.latest_text
        
        # 计算相似度和完整度
        similarity = calculate_similarity(message, speech_text)
        completeness = calculate_completeness(message, speech_text)
        
        response = {
            "status": "success",
            "target_text": message,
            "speech_text": speech_text,
            "similarity": round(similarity * 100, 2),
            "completeness": round(completeness * 100, 2),
            "analysis": {
                "similarity_comment": get_similarity_comment(similarity),
                "completeness_comment": get_completeness_comment(completeness)
            }
        }
        return jsonify(response)
    except Exception as e:
        print(f"错误: {str(e)}")
        return jsonify({"status": "error", "message": str(e)}), 500

GUI is cleared because of reinit
建立WebSocket连接...
初始化响应: {'payload_msg': {'addition': {'duration': '0', 'logid': '202410311138527178A5213B4F601F65F6'}, 'code': 1000, 'message': 'Success', 'reqid': 'b45f675b-270e-43cf-ad99-edde703ecb36', 'sequence': 1}, 'payload_size': 158}
初始化成功
录音任务已启动
开始录音...
[识别中...] 开始我是啊
[最终结果] 开始啊！
[识别中...] 刚回来吃饭就回来了我说吧哈哈
[最终结果] 什么回来了？回来了，刚回来吃饭就回来了。我说吧，哈哈。
[识别中...] 那你要吃饭呢饭是要吃
[最终结果] 肯定要吃饭呢，饭是要吃。
[识别中...] 因为我昨天在班主任前面问了一下他说早上吃饭吃点然后回来就是说就是再怎么样也得让他们吃饱
[最终结果] 因为我昨天在班主任群里问了一下，他说早上吃饭时间，然后回来就是说就是再怎么样也得让他们吃饱。


10.1.2.101 - - [31/Oct/2024 11:39:57] "GET / HTTP/1.1" 200 -


[识别中...] 
[最终结果] 
[识别中...] do
[最终结果] 你们看看啊！
[识别中...] 有没有黑的没有哎它飞出来了没有
[最终结果] 有没有黑的？没有哎。它飞出来了没有？好了。
[识别中...] 我的昨天也喝完然后喝完了
[最终结果] 我的昨天也喝完好像喝完了。喝什？
[识别中...] 这是热火
[最终结果] 这是热火场是不是？
[识别中...] 不是我不喜欢吃没了
[最终结果] 不吃，我不喜欢吃，没了。
[识别中...] 快点吃了我昨天跟着那个香喷发的

10.1.2.101 - - [31/Oct/2024 11:40:28] "POST /submit HTTP/1.1" 200 -


收到目标文本: 再别康桥
轻轻的我走了，
正如我轻轻的来；
我轻轻的招手，
作别西天的云彩。
那河畔的金柳，
是夕阳中的新娘；
波光里的艳影，
在我的心头荡漾。
软泥上的青荇⑴，
油油的在水底招摇⑵；
在康河的柔波里，
我甘心做一条水草！
那榆荫下的一潭，
不是清泉，是天上虹；
揉碎在浮藻间，
沉淀着彩虹似的梦。
寻梦？撑一支长篙⑶，
向青草更青处漫溯⑷；
满载一船星辉，
在星辉斑斓里放歌。
但我不能放歌，
悄悄是别离的笙箫；
夏虫也为我沉默，
沉默是今晚的康桥！
悄悄的我走了，
正如我悄悄的来；
我挥一挥衣袖，
不带走一片云彩。
[识别中...] 快点吃了我昨天跟着那个香喷发的瓶子搞了三包都是用了三包这么多太太淡了那个杯子挺大的你这里只有那种我只有这种奶茶
[最终结果] 我昨天跟着那个香喷衫的瓶子搞了3包，都是用了3包这么多，这个太太淡了，那个杯子挺大的，你这里只有那种？我只有这种奶茶了没有。
[识别中...] 算了我说
[最终结果] 算了，我知道。
[识别中...] 加这就是黑巧啊不那个不是是不是黑巧是那个叫我这边
[最终结果] 这这就是黑巧啊不，那个不是是不是黑巧是那个脚步这边。
[识别中...] 你是人间的
[最终结果] 你是人间的！
[识别中...] 岁月天对啊林徽因可粉这可不是甜的呀它是什么我说
[最终结果] 四月天对啊林徽因可粉这个不是甜的呀他是什么？我说。
[识别中...] 你是人间的四月天
[最终结果] 你是人间的四月天。
[识别中...] 小巷点亮了四面风清零在春的光阴中
[最终结果] 想象点亮了四面风，清零在春的光阴中。
[识别中...] 再别康桥
[最终结果] 再别康桥！
[识别中...] 徐志摩
[最终结果] 许志摩。
[识别中...] 轻轻的
[最终结果] 轻轻的。
[识别中...] 我走了
[最终结果] 我走了。
[识别中...] 正如我
[最终结果] 正如我。
[识别中...] 轻轻的来
[最终结果] 轻轻的来。
[识别中...] 我轻轻地招手
[最终结果] 我轻轻的招手。
[识别中...] 作别
[最终结果] 作别。
[识别中...] 西天的云彩
[最终结果] 西天的云彩！
[识别中...] 那河畔的金柳
[最终结果] 那河畔的金柳，
[识别中...] 是夕阳中的
录音已停止
程序已退出


In [ ]:
import json
import uuid
import gzip
import asyncio
import websockets
import numpy as np
import sounddevice as sd
import nest_asyncio
from unihiker import GUI
import time
from tkinter import END
import requests
import os
import dashscope
from flask import Flask, request, send_file, jsonify
import threading
import socket

# 配置参数
appid = "4166554764"    # 项目的 appid
token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"    # 项目的 token
cluster = "volcengine_streaming_common"  # 请求的集群

# 协议常量
PROTOCOL_VERSION = 0b0001
DEFAULT_HEADER_SIZE = 0b0001

PROTOCOL_VERSION_BITS = 4
HEADER_BITS = 4
MESSAGE_TYPE_BITS = 4
MESSAGE_TYPE_SPECIFIC_FLAGS_BITS = 4
MESSAGE_SERIALIZATION_BITS = 4
MESSAGE_COMPRESSION_BITS = 4
RESERVED_BITS = 8

# Message Type:
CLIENT_FULL_REQUEST = 0b0001
CLIENT_AUDIO_ONLY_REQUEST = 0b0010
SERVER_FULL_RESPONSE = 0b1001
SERVER_ACK = 0b1011
SERVER_ERROR_RESPONSE = 0b1111

# Message Type Specific Flags
NO_SEQUENCE = 0b0000
POS_SEQUENCE = 0b0001
NEG_SEQUENCE = 0b0010
NEG_SEQUENCE_1 = 0b0011

# Message Serialization
NO_SERIALIZATION = 0b0000
JSON = 0b0001
THRIFT = 0b0011
CUSTOM_TYPE = 0b1111

# Message Compression
NO_COMPRESSION = 0b0000
GZIP = 0b0001
CUSTOM_COMPRESSION = 0b1111

# 添加一个全局标志来控制语音识别
is_recording = True

# 初始化GUI
gui = GUI()

# 创建标题
title = gui.draw_text(
    x=120,
    y=20,
    text="背诵助手",
    font_size=20,
    origin='center'
)

# 创建文本框
text_box = gui.add_text_box(
    x=120,
    y=140,
    w=220,
    h=180,
    origin='center',
    font_size=12
)

# 创建纠正文本按钮
correction_button = gui.add_button(
    x=120,
    y=270,
    w=160,
    h=36,
    text="纠正文本",
    origin='center'
)

# 添加打分按钮
score_button = gui.add_button(
    x=120,
    y=350,  # 位于校验按钮下方
    w=100,
    h=40,
    text="文本打分",
    origin='center'
)

# 共享数据类
class SharedData:
    def __init__(self):
        self.latest_text = ""  # 最新的识别文本
        self.target_text = ""  # 范文
        self.latest_score = None  # 最新的评分结果

shared_data = SharedData()

app = Flask(__name__)

# Flask路由
@app.route('/')
def home():
    return send_file('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    try:
        data = request.get_json()
        message = data.get('message', '')
        print(f"收到范文: {message}")
        shared_data.target_text = message  # 保存范文
        return jsonify({"status": "success"})
    except Exception as e:
        print(f"错误: {str(e)}")
        return jsonify({"status": "error", "message": str(e)}), 500

@app.route('/get_score', methods=['GET'])
def get_score():
    try:
        if shared_data.latest_score:
            return jsonify({
                "status": "success",
                "data": shared_data.latest_score
            })
        return jsonify({
            "status": "waiting",
            "message": "等待评分结果..."
        })
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

# 打分按钮处理函数
def on_score_button_pressed():
    try:
        target_text = shared_data.target_text
        current_text = shared_data.latest_text
        
        if not target_text:
            print("未找到范文")
            update_recognition_text("[错误] 请先在网页端输入范文")
            return
            
        if not current_text:
            print("未找到识别文本")
            update_recognition_text("[错误] 请先进行语音识别")
            return
        
        print("正在评分...")
        update_recognition_text("[正在进行评分分析...]")
        
        # 构造提示词
        prompt = f"""
请对比以下两段文本，从准确度、完整度、流畅度三个维度进行评分（满分100分），并给出详细分析：

范文：
{target_text}

朗读文本：
{current_text}

请按以下格式输出：
准确度：XX分
完整度：XX分
流畅度：XX分
总分：XX分

详细分析：
1. 准确度分析：...
2. 完整度分析：...
3. 流畅度分析：...
4. 改进建议：...
"""
        
        # 调用大模型API
        response = dashscope.Generation.call(
            api_key=os.getenv('DASHSCOPE_API_KEY'),
            model="qwen-plus",
            messages=[
                {'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': prompt}
            ],
            result_format='message'
        )
        
        if response.status_code == 200:
            result = response.output.choices[0].message.content
            print(f"评分结果：{result}")
            
            # 更新UNIHIKER显示
            update_recognition_text(f"[评分结果]\n{result}")
            
            # 更新共享数据供网页显示
            shared_data.latest_score = {
                "target_text": target_text,
                "speech_text": current_text,
                "score_result": result
            }
        else:
            error_msg = f"评分失败: {response.message}"
            print(error_msg)
            update_recognition_text(f"[错误] {error_msg}")
            
    except Exception as e:
        error_msg = f"评分错误: {str(e)}"
        print(error_msg)
        update_recognition_text(f"[错误] {error_msg}")

# 绑定纠正按钮点击事件
correction_button.config(command=on_correction_click)

# 设置API密钥
os.environ['DASHSCOPE_API_KEY'] = 'sk-7c04ee6f9432492bb344baa7a5c0162f'

# 在notebook中运行异步代码的辅助函数
import nest_asyncio
nest_asyncio.apply()

# 创建客户端实例并运行
client = AsrWsClient(appid, token, cluster)
try:
    # 创建一个新的事件循环
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    
    # 启动语音识别任务
    recognition_task = loop.create_task(client.process_microphone())
    
    # 主循环
    while True:
        try:
            loop.run_until_complete(asyncio.sleep(0.1))
            gui.update()
            
        except KeyboardInterrupt:
            break
            
except Exception as e:
    print(f"\n程序异常: {e}")
finally:
    loop.close()
    print("程序已退出")

# Flask 服务器运行函数
def get_ip_address():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "192.168.123.1"

def check_port(port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        sock.bind(('0.0.0.0', port))
        sock.close()
        return True
    except:
        return False

def run_flask():
    try:
        ip = get_ip_address()
        # 使用明显的分隔线和格式化输出
        print("\n" + "="*50)
        print("【Web服务器启动信息】")
        print("="*50)
        print(f"服务器状态: 正在启动...")
        print(f"访问地址: http://{ip}:5000")
        print(f"本地地址: http://localhost:5000")
        print(f"监听端口: 5000")
        print("="*50)
        print("请在浏览器中访问以上地址来输入范文")
        print("="*50 + "\n")
        
        # 启动服务器
        app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
    except Exception as e:
        print("\n" + "="*50)
        print("【服务器启动失败】")
        print(f"错误信息: {str(e)}")
        print("="*50 + "\n")

if __name__ == '__main__':
    try:
        print("\n=== 程序启动顺序 ===")
        print("1. 启动Web服务器")
        print("2. 初始化GUI界面")
        print("3. 启动语音识别\n")
        
        # 检查端口
        if not check_port(5000):
            print("\n警告: 端口 5000 已被占用!")
            print("请尝试关闭占用该端口的程序，或者使用其他端口\n")
            exit(1)
        
        # 启动Web服务器
        flask_thread = threading.Thread(target=run_flask)
        flask_thread.daemon = True
        flask_thread.start()
        
        # 等待服务器启动
        time.sleep(2)
        print("Web服务器启动完成\n")
        
        # 继续执行其他初始化...
        print("正在初始化GUI...")
        
        # 保持主线程运行
        while True:
            time.sleep(1)
            
    except KeyboardInterrupt:
        print("\n程序被用户中断")
    except Exception as e:
        print(f"\n程序异常: {e}")
    finally:
        print("\n程序已退出")

GUI is cleared because of reinit
建立WebSocket连接...
初始化响应: {'payload_msg': {'addition': {'duration': '0', 'logid': '202410311245336E0F1C8B485E81207F66'}, 'code': 1000, 'message': 'Success', 'reqid': '0ab8028b-06d0-49dd-8d05-4583a8373942', 'sequence': 1}, 'payload_size': 157}
初始化成功
录音任务已启动
开始录音...


10.1.2.101 - - [31/Oct/2024 12:49:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:49:08] "GET / HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:49:09] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:49:09] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:49:10] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:49:10] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:49:11] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:49:11] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:49:12] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:49:12] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:49:13] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:49:13] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:49:14] "GET /get_analysis HTTP/1.

[识别中...] 

10.1.2.101 - - [31/Oct/2024 12:49:19] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:49:19] "GET /get_analysis HTTP/1.1" 200 -


[识别中...] 

...

10.1.2.101 - - [31/Oct/2024 12:49:46] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:49:46] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:49:47] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:49:47] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:49:48] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:49:48] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:49:49] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:49:49] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:49:50] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:49:50] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:49:51] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:49:51] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:49:52] "G

程序已退出

=== 程序启动顺序 ===
1. 启动Web服务器
2. 初始化GUI界面
3. 启动语音识别


警告: 端口 5000 已被占用!
请尝试关闭占用该端口的程序，或者使用其他端口


【Web服务器启动信息】
服务器状态: 正在启动...
访问地址: http://172.16.5.104:5000
本地地址: http://localhost:5000
监听端口: 5000
请在浏览器中访问以上地址来输入范文

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off

【服务器启动失败】
错误信息: [Errno 98] Address already in use



10.1.2.101 - - [31/Oct/2024 12:50:10] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:50:10] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:50:11] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:50:11] "GET /get_analysis HTTP/1.1" 200 -


Web服务器启动完成

正在初始化GUI...


10.1.2.101 - - [31/Oct/2024 12:50:12] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:50:12] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:50:13] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:50:13] "GET /get_analysis HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 12:50:14] "GET /get_analysis HTTP/1.1" 200 -
INFO:werkzeug:10.1.2.101 - - [31/Oct/2024 12:50:14] "GET /get_analysis HTTP/1.1" 200 -


In [6]:
from flask import Flask, request, send_file, jsonify
from difflib import SequenceMatcher
import json
import uuid
import gzip
import asyncio
import websockets
import numpy as np
import sounddevice as sd
import nest_asyncio
from unihiker import GUI
import time
from tkinter import END
import requests
import os
import dashscope
from pinpong.board import Board
from pinpong.extension.unihiker import *

# Flask 应用
app = Flask(__name__)

# 共享数据存储
class SharedData:
    def __init__(self):
        self.speech_recognition_results = []
        self.latest_text = ""
        self.target_text = ""
        self.latest_score = None

shared_data = SharedData()

# Web 服务器路由
@app.route('/')
def home():
    return send_file('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    try:
        data = request.get_json()
        target_text = data.get('message', '')
        print(f"收到目标文本: {target_text}")

        # 获取最新的语音识别结果
        speech_text = shared_data.latest_text
        
        # 计算相似度和完整度
        similarity = calculate_similarity(target_text, speech_text)
        completeness = calculate_completeness(target_text, speech_text)
        
        response = {
            "status": "success",
            "target_text": target_text,
            "speech_text": speech_text,
            "similarity": round(similarity * 100, 2),
            "completeness": round(completeness * 100, 2),
            "analysis": {
                "similarity_comment": get_similarity_comment(similarity),
                "completeness_comment": get_completeness_comment(completeness)
            }
        }
        return jsonify(response)
    except Exception as e:
        print(f"错误: {str(e)}")
        return jsonify({"status": "error", "message": str(e)}), 500

# 相似度计算函数
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

def calculate_completeness(target_text, speech_text):
    target_words = set(target_text.split())
    speech_words = set(speech_text.split())
    common_words = target_words.intersection(speech_words)
    return len(common_words) / len(target_words) if target_words else 0

def get_similarity_comment(similarity):
    if similarity >= 0.9: return "非常接近"
    elif similarity >= 0.7: return "比较接近"
    elif similarity >= 0.5: return "部分接近"
    else: return "差异较大"

def get_completeness_comment(completeness):
    if completeness >= 0.9: return "内容完整"
    elif completeness >= 0.7: return "大部分完整"
    elif completeness >= 0.5: return "部分完整"
    else: return "内容缺失"

# 配置参数
appid = "4166554764"    # 项目的 appid
token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"    # 项目的 token
cluster = "volcengine_streaming_common"  # 请求的集群

# 协议常量
PROTOCOL_VERSION = 0b0001
DEFAULT_HEADER_SIZE = 0b0001

PROTOCOL_VERSION_BITS = 4
HEADER_BITS = 4
MESSAGE_TYPE_BITS = 4
MESSAGE_TYPE_SPECIFIC_FLAGS_BITS = 4
MESSAGE_SERIALIZATION_BITS = 4
MESSAGE_COMPRESSION_BITS = 4
RESERVED_BITS = 8

# Message Type:
CLIENT_FULL_REQUEST = 0b0001
CLIENT_AUDIO_ONLY_REQUEST = 0b0010
SERVER_FULL_RESPONSE = 0b1001
SERVER_ACK = 0b1011
SERVER_ERROR_RESPONSE = 0b1111

# Message Type Specific Flags
NO_SEQUENCE = 0b0000
POS_SEQUENCE = 0b0001
NEG_SEQUENCE = 0b0010
NEG_SEQUENCE_1 = 0b0011

# Message Serialization
NO_SERIALIZATION = 0b0000
JSON = 0b0001
THRIFT = 0b0011
CUSTOM_TYPE = 0b1111

# Message Compression
NO_COMPRESSION = 0b0000
GZIP = 0b0001
CUSTOM_COMPRESSION = 0b1111

# 初始化 nest_asyncio
nest_asyncio.apply()

# 全局变量
is_recording = True
all_texts = []
gui = None
loop = None  # 添加全局 loop 变量

def on_correction_click():
    """处理纠正按钮点击事件"""
    global is_recording
    try:
        print("纠正按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)  # 给一点时间让录音线程完全停止
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的当前文本：{current_text}")
        
        if current_text.strip():
            # 3. 调用API进行纠正
            print("开始调用API进行纠正...")
            corrected_text = call_qwen_api(current_text)
            print(f"API返回的纠正结果：{corrected_text}")
            
            # 4. 显示纠正结果
            update_recognition_text(corrected_text, is_correction=True)
            print("更新显示完成")
        else:
            print("文本框为空，不进行API调用")
            
    except Exception as e:
        error_msg = f"纠正文本错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg, is_correction=True)

def on_score_click():
    """处理打分按钮点击事件"""
    global is_recording
    try:
        print("打分按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的朗读文本：{current_text}")
        
        # 3. 获取范文
        target_text = shared_data.target_text
        if not target_text:
            error_msg = "[错误] 请先在网页端输入范文"
            print(error_msg)
            update_recognition_text(error_msg)
            return
            
        if not current_text.strip():
            error_msg = "[错误] 请先进行语音识别"
            print(error_msg)
            update_recognition_text(error_msg)
            return
        
        print("开始调用API进行评分...")
        # 4. 构造提示词
        prompt = f"""
请对比以下两段文本，从准确度、完整度、流畅度三个维度进行评分（满分100分），并给出详细分析：

范文：
{target_text}

朗读文本：
{current_text}

请按以下格式输出：
准确度：XX分
完整度：XX分
流畅度：XX分
总分：XX分

详细分析：
1. 准确度分析：...
2. 完整度分析：...
3. 流畅度分析：...
4. 改进建议：...
"""
        
        # 5. 调用API进行评分
        response = dashscope.Generation.call(
            api_key=os.getenv('DASHSCOPE_API_KEY'),
            model="qwen-plus",
            messages=[
                {'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': prompt}
            ],
            result_format='message'
        )
        
        if response.status_code == 200:
            result = response.output.choices[0].message.content
            print(f"评分结果：{result}")
            
            # 更新UNIHIKER显示
            update_recognition_text(f"[评分结果]\n{result}")
            
            # 更新共享数据供网页显示
            shared_data.latest_score = {
                "target_text": target_text,
                "speech_text": current_text,
                "score_result": result
            }
            
        else:
            error_msg = f"评分失败: {response.message}"
            print(error_msg)
            update_recognition_text(f"[错误] {error_msg}")
            
    except Exception as e:
        error_msg = f"评分错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg)

def init_gui():
    """初始化GUI，只调用一次"""
    global gui
    
    # 初始化GUI
    gui = GUI()
    
    # 创建标题
    title = gui.draw_text(
        x=120,
        y=20,
        text="背诵助手",
        font_size=20,
        origin='center'
    )
    
    # 创建文本框
    text_box = gui.add_text_box(
        x=120,
        y=140,
        w=220,
        h=180,
        origin='center',
        font_size=12
    )
    
    # 创建纠正按钮
    correction_button = gui.add_button(
        x=120,
        y=270,
        w=160,
        h=36,
        text="纠正文本",
        origin='center',
        onclick=on_correction_click
    )
    
    # 创建打分按钮
    score_button = gui.add_button(
        x=120,
        y=320,  # 位于纠正按钮下方
        w=160,
        h=36,
        text="文本打分",
        origin='center',
        onclick=on_score_click
    )
    
    return gui, text_box

if __name__ == '__main__':
    try:
        print("\n=== 程序启动 ===")
        
        # 1. 初始化板子
        Board().begin()
        
        # 2. 只初始化一次GUI
        gui, text_box = init_gui()
        
        # 3. 创建客户端实例
        client = AsrWsClient(appid, token, cluster)
        
        # 4. 创建事件循环
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        
        # 5. 启动语音识别任务
        recognition_task = loop.create_task(client.process_microphone())
        
        # 6. 主循环
        while True:
            try:
                loop.run_until_complete(asyncio.sleep(0.1))
                
                # 检测按钮A（向上滚动）
                if button_a.is_pressed():
                    print("按钮A按下")
                    text_box.text.yview_scroll(-1, "units")
                    gui.update()
                    time.sleep(0.2)
                
                # 检测按钮B（向下滚动）
                if button_b.is_pressed():
                    print("按钮B按下")
                    text_box.text.yview_scroll(1, "units")
                    gui.update()
                    time.sleep(0.2)
                
                gui.update()
                
            except KeyboardInterrupt:
                print("\n程序被用户中断")
                break
            except Exception as e:
                print(f"\n循环中出现错误: {e}")
                break
                
    except Exception as e:
        print(f"\n程序异常: {e}")
    finally:
        try:
            if loop and not loop.is_closed():
                loop.close()
        except Exception as e:
            print(f"关闭事件循环时出错: {e}")
        print("程序已退出")



=== 程序启动 ===

  ___________________________
 |                           |
 |      PinPong v0.5.2       |
 |    Designed by DFRobot    |
 |___________________________|
 
[01] Python3.7.3 Linux-4.4.143-67-rockchip-g01bbbc5d1312-aarch64-with-debian-10.11 Board: UNIHIKER
selected -> board: UNIHIKER serial: /dev/ttyS3
[10] Opening /dev/ttyS3
[32] Firmata ID: 3.7
[35] Burning firmware...
initialize
/usr/local/lib/python3.7/dist-packages/pinpong/base/FirmataExpress.UNIHIKER.3.8.bin
stm32flash -w /usr/local/lib/python3.7/dist-packages/pinpong/base/FirmataExpress.UNIHIKER.3.8.bin -v -g 0x08000000 /dev/ttyS3


GET returns unknown commands (0x 6)


stm32flash 0.5

http://stm32flash.sourceforge.net/

Using Parser : Raw BINARY
Interface serial_posix: 57600 8E1
Version      : 0x30
Option 1     : 0x00
Option 2     : 0x00
Device ID    : 0x0410 (STM32F10xxx Medium-density)
- RAM        : 20KiB  (512b reserved by bootloader)
- Flash      : 128KiB (size first sector: 4x1024)
- Option RAM : 16b
- System RAM : 2KiB
Write to memory
Erasing memory
Wrote and verified address 0x0800bd64 (100.00%) Done.

Starting execution at address 0x08000000... done.

[37] Burn done
selected -> board: UNIHIKER serial: /dev/ttyS3
[10] Opening /dev/ttyS3
[22] Arduino compatible device found and connected to /dev/ttyS3
[40] Retrieving analog map...
[42] Auto-discovery complete. Found 30 Digital Pins and 30 Analog Pins
------------------------------
All right. PinPong go...
------------------------------
GUI is cleared because of reinit
建立WebSocket连接...
初始化响应: {'payload_msg': {'addition': {'duration': '0', 'logid': '202410311357033626A5C580775126003D'}, 'code': 

SystemExit: 0

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
from flask import Flask, request, send_file, jsonify
from difflib import SequenceMatcher
import json
import uuid
import gzip
import asyncio
import websockets
import numpy as np
import sounddevice as sd
import nest_asyncio
from unihiker import GUI
import time
from tkinter import END
import requests
import os
import dashscope
from pinpong.board import Board
from pinpong.extension.unihiker import *
import threading
import socket

# Flask 应用
app = Flask(__name__)

# 共享数据存储
class SharedData:
    def __init__(self):
        self.speech_recognition_results = []
        self.latest_text = ""
        self.target_text = ""
        self.latest_score = None

shared_data = SharedData()

# Web 服务器路由
@app.route('/')
def index():
    return send_file('index.html')

@app.route('/submit_text', methods=['POST'])
def submit_text():
    data = request.get_json()
    text = data.get('text', '')
    shared_data.target_text = text
    return jsonify({"status": "success"})

@app.route('/get_score')
def get_score():
    if shared_data.latest_score:
        return jsonify({"status": "success", "data": shared_data.latest_score})
    return jsonify({"status": "waiting"})

# 相似度计算函数
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

def calculate_completeness(target_text, speech_text):
    target_words = set(target_text.split())
    speech_words = set(speech_text.split())
    common_words = target_words.intersection(speech_words)
    return len(common_words) / len(target_words) if target_words else 0

def get_similarity_comment(similarity):
    if similarity >= 0.9: return "非常接近"
    elif similarity >= 0.7: return "比较接近"
    elif similarity >= 0.5: return "部分接近"
    else: return "差异较大"

def get_completeness_comment(completeness):
    if completeness >= 0.9: return "内容完整"
    elif completeness >= 0.7: return "大部分完整"
    elif completeness >= 0.5: return "部分完整"
    else: return "内容缺失"

# 配置参数
appid = "4166554764"    # 项目的 appid
token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"    # 项目的 token
cluster = "volcengine_streaming_common"  # 请求的集群

# 协议常量
PROTOCOL_VERSION = 0b0001
DEFAULT_HEADER_SIZE = 0b0001

PROTOCOL_VERSION_BITS = 4
HEADER_BITS = 4
MESSAGE_TYPE_BITS = 4
MESSAGE_TYPE_SPECIFIC_FLAGS_BITS = 4
MESSAGE_SERIALIZATION_BITS = 4
MESSAGE_COMPRESSION_BITS = 4
RESERVED_BITS = 8

# Message Type:
CLIENT_FULL_REQUEST = 0b0001
CLIENT_AUDIO_ONLY_REQUEST = 0b0010
SERVER_FULL_RESPONSE = 0b1001
SERVER_ACK = 0b1011
SERVER_ERROR_RESPONSE = 0b1111

# Message Type Specific Flags
NO_SEQUENCE = 0b0000
POS_SEQUENCE = 0b0001
NEG_SEQUENCE = 0b0010
NEG_SEQUENCE_1 = 0b0011

# Message Serialization
NO_SERIALIZATION = 0b0000
JSON = 0b0001
THRIFT = 0b0011
CUSTOM_TYPE = 0b1111

# Message Compression
NO_COMPRESSION = 0b0000
GZIP = 0b0001
CUSTOM_COMPRESSION = 0b1111

# 初始化 nest_asyncio
nest_asyncio.apply()

# 全局变量
is_recording = True
all_texts = []
gui = None
loop = None  # 添加全局 loop 变量

def on_correction_click():
    """处理纠正按钮点击事件"""
    global is_recording
    try:
        print("纠正按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的当前文本：{current_text}")
        
        if current_text.strip():
            # 3. 调用API进行纠正
            print("开始调用API进行纠正...")
            corrected_text = call_qwen_api(current_text)
            print(f"API返回的纠正结果：{corrected_text}")
            
            # 4. 显示纠正结果
            update_recognition_text(corrected_text, is_correction=True)
            print("更新显示完成")
        else:
            print("文本框为空，不进行API调用")
            
    except Exception as e:
        error_msg = f"纠正文本错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg, is_correction=True)

def on_score_click():
    """处理打分按钮点击事件"""
    global is_recording
    try:
        print("打分按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的朗读文本：{current_text}")
        
        # 3. 获取范文
        target_text = shared_data.target_text
        if not target_text:
            error_msg = "[错误] 请先在网页端输入范文"
            print(error_msg)
            update_recognition_text(error_msg)
            return
            
        if not current_text.strip():
            error_msg = "[错误] 请先进行语音识别"
            print(error_msg)
            update_recognition_text(error_msg)
            return
        
        print("开始调用API进行评分...")
        # 4. 构造提示词
        prompt = f"""
请对比以下两段文本，从准确度、完整度、流畅度三个维度进行评分（满分100分），并给出详细分析：

范文：
{target_text}

朗读文本：
{current_text}

请按以下格式输出：
准确度：XX分
完整度：XX分
流畅度：XX分
总分：XX分

详细分析：
1. 准确度分析：...
2. 完整度分析：...
3. 流畅度分析：...
4. 改进建议：...
"""
        
        # 5. 调用API进行评分
        response = dashscope.Generation.call(
            api_key=os.getenv('DASHSCOPE_API_KEY'),
            model="qwen-plus",
            messages=[
                {'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': prompt}
            ],
            result_format='message'
        )
        
        if response.status_code == 200:
            result = response.output.choices[0].message.content
            print(f"评分结果：{result}")
            
            # 更新UNIHIKER显示
            update_recognition_text(f"[评分结果]\n{result}")
            
            # 更新共享数据供网页显示
            shared_data.latest_score = {
                "target_text": target_text,
                "speech_text": current_text,
                "score_result": result
            }
            
        else:
            error_msg = f"评分失败: {response.message}"
            print(error_msg)
            update_recognition_text(f"[错误] {error_msg}")
            
    except Exception as e:
        error_msg = f"评分错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg)

def init_gui():
    """初始化GUI，只调用一次"""
    global gui, text_box
    
    # 初始化GUI
    gui = GUI()
    
    # 创建标题
    title = gui.draw_text(
        x=120,
        y=20,
        text="背诵助手",
        font_size=20,
        origin='center'
    )
    
    # 创建文本框
    text_box = gui.add_text_box(
        x=120,
        y=140,
        w=220,
        h=180,
        origin='center',
        font_size=12
    )
    
    # 创建纠正按钮 - 左边
    correction_button = gui.add_button(
        x=70,  # 移到左边
        y=290,  # 两个按钮在同一水平线上
        w=100,  # 减小按钮宽度
        h=36,
        text="纠正文本",
        origin='center',
        onclick=on_correction_click
    )
    
    # 创建打分按钮 - 右边
    score_button = gui.add_button(
        x=180,  # 移到右边
        y=290,  # 与纠正按钮在同一水平线上
        w=100,  # 减小按钮宽度
        h=36,
        text="文本打分",
        origin='center',
        onclick=on_score_click
    )
    
    return gui, text_box

def run_flask():
    """运行Flask服务器"""
    try:
        ip = get_ip_address()
        server_info = """
╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://{ip}:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000                               ║
║                                                ║
║  请在浏览器中访问以上地址来输入范文          ║
║                                                ║
╚════════════════════════════════════════════════╝
""".format(ip=ip)
        
        for _ in range(3):
            print("\n" + server_info, flush=True)
            time.sleep(1)
        
        app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
    except Exception as e:
        print("\n" + "="*50)
        print("【服务器启动失败】")
        print(f"错误信息: {str(e)}")
        print("="*50 + "\n")

def get_ip_address():
    """获取本机IP地址"""
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "127.0.0.1"

class AsrWsClient:
    def __init__(self, appid, token, cluster):
        self.appid = appid
        self.token = token
        self.cluster = cluster
        self.ws_url = "wss://openspeech.bytedance.com/api/v2/asr"
        self.success_code = 1000
        self.uid = "streaming_asr_demo"
        self.workflow = "audio_in,resample,partition,vad,fe,decode,itn,nlu_punctuate"
        self.show_language = False
        self.show_utterances = True
        self.result_type = "single"
        self.format = "raw"
        self.rate = 16000
        self.language = "zh-CN"
        self.bits = 16
        self.channel = 1
        self.codec = "raw"
        self.auth_method = "token"

    def construct_request(self, reqid):
        return {
            'app': {
                'appid': self.appid,
                'cluster': self.cluster,
                'token': self.token,
            },
            'user': {
                'uid': self.uid
            },
            'request': {
                'reqid': reqid,
                'nbest': 1,
                'workflow': self.workflow,
                'show_language': self.show_language,
                'show_utterances': self.show_utterances,
                'result_type': self.result_type,
                'sequence': 1
            },
            'audio': {
                'format': self.format,
                'rate': self.rate,
                'language': self.language,
                'bits': self.bits,
                'channel': self.channel,
                'codec': self.codec
            }
        }

    def token_auth(self):
        return {'Authorization': f'Bearer; {self.token}'}

    async def process_microphone(self):
        """实时麦克风录音并识别"""
        global is_recording
        is_recording = True
        
        reqid = str(uuid.uuid4())
        request_params = self.construct_request(reqid)
        
        payload_bytes = str.encode(json.dumps(request_params))
        payload_bytes = gzip.compress(payload_bytes)
        full_request = bytearray(generate_full_default_header())
        full_request.extend(len(payload_bytes).to_bytes(4, 'big'))
        full_request.extend(payload_bytes)

        print("建立WebSocket连接...")
        async with websockets.connect(
            self.ws_url, 
            extra_headers=self.token_auth(), 
            max_size=1000000000
        ) as ws:
            await ws.send(full_request)
            response = await ws.recv()
            result = parse_response(response)
            print(f"初始化响应: {result}")
            
            if 'payload_msg' in result and result['payload_msg']['code'] == self.success_code:
                print("初始化成功")
                print("录音任务已启动")
                chunk_size = 9600
                
                with sd.InputStream(channels=1, samplerate=16000, dtype=np.int16, blocksize=chunk_size) as stream:
                    print("开始录音...")
                    while is_recording:
                        audio_data, overflowed = stream.read(chunk_size)
                        if overflowed:
                            print("警告：音频缓冲区溢出")
                            
                        audio_bytes = audio_data.tobytes()
                        compressed_audio = gzip.compress(audio_bytes)
                        
                        audio_request = bytearray(generate_audio_default_header())
                        audio_request.extend(len(compressed_audio).to_bytes(4, 'big'))
                        audio_request.extend(compressed_audio)
                        
                        await ws.send(audio_request)
                        response = await ws.recv()
                        result = parse_response(response)
                        
                        if 'payload_msg' in result and 'result' in result['payload_msg']:
                            utterances = result['payload_msg']['result'][0].get('utterances', [])
                            for utterance in utterances:
                                if not utterance['definite']:
                                    print(f"\r[识别中...] {utterance['text']}", end='', flush=True)
                                else:
                                    print(f"\n[最终结果] {utterance['text']}")
                                    update_recognition_text(utterance['text'])

# 添加其他必要的函数
def generate_full_default_header():
    return generate_header()

def generate_header(
    version=PROTOCOL_VERSION,
    message_type=CLIENT_FULL_REQUEST,
    message_type_specific_flags=NO_SEQUENCE,
    serial_method=JSON,
    compression_type=GZIP,
    reserved_data=0x00,
    extension_header=bytes()
):
    """生成请求头"""
    header = bytearray()
    header_size = int(len(extension_header) / 4) + 1
    header.append((version << 4) | header_size)
    header.append((message_type << 4) | message_type_specific_flags)
    header.append((serial_method << 4) | compression_type)
    header.append(reserved_data)
    header.extend(extension_header)
    return header

def generate_audio_default_header():
    """生成音频数据请求头"""
    return generate_header(message_type=CLIENT_AUDIO_ONLY_REQUEST)

def parse_response(res):
    """解析响应"""
    try:
        protocol_version = res[0] >> 4
        header_size = res[0] & 0x0f
        message_type = res[1] >> 4
        message_type_specific_flags = res[1] & 0x0f
        serialization_method = res[2] >> 4
        message_compression = res[2] & 0x0f
        reserved = res[3]
        header_extensions = res[4:header_size * 4]
        payload = res[header_size * 4:]
        result = {}
        payload_msg = None
        payload_size = 0
        
        if message_type == SERVER_FULL_RESPONSE:
            payload_size = int.from_bytes(payload[:4], "big", signed=True)
            payload_msg = payload[4:]
        elif message_type == SERVER_ACK:
            seq = int.from_bytes(payload[:4], "big", signed=True)
            result['seq'] = seq
            if len(payload) >= 8:
                payload_size = int.from_bytes(payload[4:8], "big", signed=False)
                payload_msg = payload[8:]
        elif message_type == SERVER_ERROR_RESPONSE:
            code = int.from_bytes(payload[:4], "big", signed=False)
            result['code'] = code
            payload_size = int.from_bytes(payload[4:8], "big", signed=False)
            payload_msg = payload[8:]
            
        if payload_msg is None:
            return result
            
        if message_compression == GZIP:
            payload_msg = gzip.decompress(payload_msg)
            
        if serialization_method == JSON:
            payload_msg = json.loads(str(payload_msg, "utf-8"))
        elif serialization_method != NO_SERIALIZATION:
            payload_msg = str(payload_msg, "utf-8")
            
        result['payload_msg'] = payload_msg
        result['payload_size'] = payload_size
        return result
    except Exception as e:
        return {"error": f"Failed to parse response: {str(e)}"}

def update_recognition_text(new_text, is_correction=False):
    """更新识别结果并保持滚动条在底部"""
    try:
        prefix = "[纠正结果] " if is_correction else ""
        all_texts.append(prefix + new_text)
        
        full_text = "\n".join(all_texts)
        text_box.config(text=full_text)
        text_box.text.see(END)
        
        if gui.master.winfo_exists():
            gui.update()
            
    except Exception as e:
        print(f"更新文本错误: {e}")

if __name__ == '__main__':
    try:
        print("\n=== 程序启动顺序 ===")
        print("1. 启动Web服务器")
        print("2. 初始化GUI界面")
        print("3. 启动语音识别\n")
        
        # 1. 先启动Web服务器
        print("正在启动Web服务器...\n")
        flask_thread = threading.Thread(target=run_flask)
        flask_thread.daemon = True
        flask_thread.start()
        
        # 等待服务器启动
        time.sleep(3)
        
        # 2. 初始化板子
        print("\n正在初始化硬件...\n")
        Board().begin()
        
        # 3. 初始化GUI
        print("\n正在初始化GUI...\n")
        gui, text_box = init_gui()
        
        # 4. 创建客户端实例并启动语音识别
        print("\n正在启动语音识别...\n")
        client = AsrWsClient(appid, token, cluster)
        
        # 5. 创建事件循环
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        recognition_task = loop.create_task(client.process_microphone())
        
        # 6. 主循环
        while True:
            try:
                loop.run_until_complete(asyncio.sleep(0.1))
                
                # 检测按钮A（向上滚动）
                if button_a.is_pressed():
                    text_box.text.yview_scroll(-1, "units")
                    gui.update()
                    time.sleep(0.2)
                
                # 检测按钮B（向下滚动）
                if button_b.is_pressed():
                    text_box.text.yview_scroll(1, "units")
                    gui.update()
                    time.sleep(0.2)
                
                gui.update()
                
            except KeyboardInterrupt:
                print("\n程序被用户中断")
                break
            except Exception as e:
                print(f"\n循环中出现错误: {e}")
                break
                
    except Exception as e:
        print(f"\n程序异常: {e}")
    finally:
        try:
            if loop and not loop.is_closed():
                loop.close()
        except Exception as e:
            print(f"关闭事件循环时出错: {e}")
        print("程序已退出")

# 设置API密钥
os.environ['DASHSCOPE_API_KEY'] = 'sk-7c04ee6f9432492bb344baa7a5c0162f'



=== 程序启动顺序 ===
1. 启动Web服务器
2. 初始化GUI界面
3. 启动语音识别

正在启动Web服务器...



╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://172.16.5.104:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000                               ║
║                                                ║
║  请在浏览器中访问以上地址来输入范文          ║
║                                                ║
╚════════════════════════════════════════════════╝



╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://172.16.5.104:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000   

 * Running on all addresses.
 * Running on http://172.16.5.104:5000/ (Press CTRL+C to quit)



  ___________________________
 |                           |
 |      PinPong v0.5.2       |
 |    Designed by DFRobot    |
 |___________________________|
 
[01] Python3.7.3 Linux-4.4.143-67-rockchip-g01bbbc5d1312-aarch64-with-debian-10.11 Board: UNIHIKER
selected -> board: UNIHIKER serial: /dev/ttyS3
[10] Opening /dev/ttyS3
[32] Firmata ID: 3.8
[22] Arduino compatible device found and connected to /dev/ttyS3
[40] Retrieving analog map...
[42] Auto-discovery complete. Found 30 Digital Pins and 30 Analog Pins
------------------------------
All right. PinPong go...
------------------------------

正在初始化GUI...

GUI is cleared because of reinit

正在启动语音识别...

建立WebSocket连接...
初始化响应: {'payload_msg': {'addition': {'duration': '0', 'logid': '20241031142247A77DEB13908E9B240276'}, 'code': 1000, 'message': 'Success', 'reqid': '0d05acc5-6426-4c7b-908e-b270644ffd3e', 'sequence': 1}, 'payload_size': 158}
初始化成功
录音任务已启动
开始录音...
[识别中...] 作别
[最终结果] 作别。
[识别中...] 西天的云在
[最终结果] 西天的云在？
[识别中...] 那河畔的金柳
[最终结果] 

10.1.2.101 - - [31/Oct/2024 14:23:42] "GET / HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:23:43] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:23:44] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:23:45] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:23:46] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:23:47] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:23:48] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:23:49] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:23:50] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:23:51] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:23:52] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:23:53] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:23:54] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:23:55] "POST /submit HTTP/1.1" 404 -
10.1.2.101 - - [31/Oct/2024 14:

user quit process
程序已退出


SystemExit: 0

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
from flask import Flask, request, send_file, jsonify
from difflib import SequenceMatcher
import json
import uuid
import gzip
import asyncio
import websockets
import numpy as np
import sounddevice as sd
import nest_asyncio
from unihiker import GUI
import time
from tkinter import END
import requests
import os
import dashscope
from pinpong.board import Board
from pinpong.extension.unihiker import *
import threading
import socket

# Flask 应用
app = Flask(__name__)

# 共享数据存储
class SharedData:
    def __init__(self):
        self.speech_recognition_results = []
        self.latest_text = ""
        self.target_text = ""
        self.latest_score = None

shared_data = SharedData()

# Web 服务器路由
@app.route('/')
def index():
    return send_file('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    try:
        data = request.get_json()
        text = data.get('text', '')
        shared_data.target_text = text
        return jsonify({"status": "success", "message": "范文提交成功"})
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

@app.route('/get_score')
def get_score():
    if shared_data.latest_score:
        score_data = shared_data.latest_score
        # 返回后清除分数，避免重复获取
        shared_data.latest_score = None
        return jsonify({"status": "success", "data": score_data})
    return jsonify({"status": "waiting"})

# 相似度计算函数
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

def calculate_completeness(target_text, speech_text):
    target_words = set(target_text.split())
    speech_words = set(speech_text.split())
    common_words = target_words.intersection(speech_words)
    return len(common_words) / len(target_words) if target_words else 0

def get_similarity_comment(similarity):
    if similarity >= 0.9: return "非常接近"
    elif similarity >= 0.7: return "比较接近"
    elif similarity >= 0.5: return "部分接近"
    else: return "差异较大"

def get_completeness_comment(completeness):
    if completeness >= 0.9: return "内容完整"
    elif completeness >= 0.7: return "大部分完整"
    elif completeness >= 0.5: return "部分完整"
    else: return "内容缺失"

# 配置参数
appid = "4166554764"    # 项目的 appid
token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"    # 项目的 token
cluster = "volcengine_streaming_common"  # 请求的集群

# 协议常量
PROTOCOL_VERSION = 0b0001
DEFAULT_HEADER_SIZE = 0b0001

PROTOCOL_VERSION_BITS = 4
HEADER_BITS = 4
MESSAGE_TYPE_BITS = 4
MESSAGE_TYPE_SPECIFIC_FLAGS_BITS = 4
MESSAGE_SERIALIZATION_BITS = 4
MESSAGE_COMPRESSION_BITS = 4
RESERVED_BITS = 8

# Message Type:
CLIENT_FULL_REQUEST = 0b0001
CLIENT_AUDIO_ONLY_REQUEST = 0b0010
SERVER_FULL_RESPONSE = 0b1001
SERVER_ACK = 0b1011
SERVER_ERROR_RESPONSE = 0b1111

# Message Type Specific Flags
NO_SEQUENCE = 0b0000
POS_SEQUENCE = 0b0001
NEG_SEQUENCE = 0b0010
NEG_SEQUENCE_1 = 0b0011

# Message Serialization
NO_SERIALIZATION = 0b0000
JSON = 0b0001
THRIFT = 0b0011
CUSTOM_TYPE = 0b1111

# Message Compression
NO_COMPRESSION = 0b0000
GZIP = 0b0001
CUSTOM_COMPRESSION = 0b1111

# 初始化 nest_asyncio
nest_asyncio.apply()

# 全局变量
is_recording = True
all_texts = []
gui = None
loop = None  # 添加全局 loop 变量

def on_correction_click():
    """处理纠正按钮点击事件"""
    global is_recording
    try:
        print("纠正按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的当前文本：{current_text}")
        
        if current_text.strip():
            # 3. 调用API进行纠正
            print("开始调用API进行纠正...")
            corrected_text = call_qwen_api(current_text)
            print(f"API返回的纠正结果：{corrected_text}")
            
            # 4. 显示纠正结果
            update_recognition_text(corrected_text, is_correction=True)
            print("更新显示完成")
        else:
            print("文本框为空，不进行API调用")
            
    except Exception as e:
        error_msg = f"纠正文本错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg, is_correction=True)

def on_score_click():
    """处理打分按钮点击事件"""
    global is_recording
    try:
        print("打分按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的朗读文本：{current_text}")
        
        # 3. 获取范文
        target_text = shared_data.target_text
        if not target_text:
            error_msg = "[错误] 请先在网页端输入范文"
            print(error_msg)
            update_recognition_text(error_msg)
            return
            
        if not current_text.strip():
            error_msg = "[错误] 请先进行语音识别"
            print(error_msg)
            update_recognition_text(error_msg)
            return
        
        print("开始调用API进行评分...")
        # 4. 构造提示词
        prompt = f"""
请对比以下两段文本，从准确度、完整度、流畅度三个维度进行评分（满分100分），并给出详细分析：

范文：
{target_text}

朗读文本：
{current_text}

请按以下格式输出：
准确度：XX分
完整度：XX分
流畅度：XX分
总分：XX分

详细分析：
1. 准确度分析：...
2. 完整度分析：...
3. 流畅度分析：...
4. 改进建议：...
"""
        
        # 5. 调用API进行评分
        response = dashscope.Generation.call(
            api_key=os.getenv('DASHSCOPE_API_KEY'),
            model="qwen-plus",
            messages=[
                {'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': prompt}
            ],
            result_format='message'
        )
        
        if response.status_code == 200:
            result = response.output.choices[0].message.content
            print(f"评分结果：{result}")
            
            # 更新UNIHIKER显示
            update_recognition_text(f"[评分结果]\n{result}")
            
            # 更新共享数据供网页显示
            shared_data.latest_score = {
                "target_text": target_text,
                "speech_text": current_text,
                "score_result": result
            }
            
        else:
            error_msg = f"评分失败: {response.message}"
            print(error_msg)
            update_recognition_text(f"[错误] {error_msg}")
            
    except Exception as e:
        error_msg = f"评分错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg)

def init_gui():
    """初始化GUI，只调用一次"""
    global gui, text_box
    
    if gui is not None:
        print("GUI 已经初始化，跳过重复初始化")
        return gui, text_box
        
    # 初始化GUI
    gui = GUI()
    
    # 创建标题
    title = gui.draw_text(
        x=120,
        y=20,
        text="背诵助手",
        font_size=20,
        origin='center'
    )
    
    # 创建文本框
    text_box = gui.add_text_box(
        x=120,
        y=140,
        w=220,
        h=180,
        origin='center',
        font_size=12
    )
    
    # 创建纠正按钮 - 左边
    correction_button = gui.add_button(
        x=70,  # 移到左边
        y=290,  # 两个按钮在同一水平线上
        w=100,  # 减小按钮宽度
        h=36,
        text="纠正文本",
        origin='center',
        onclick=on_correction_click
    )
    
    # 创建打分按钮 - 右边
    score_button = gui.add_button(
        x=180,  # 移到右边
        y=290,  # 与纠正按钮在同一水平线上
        w=100,  # 减小按钮宽度
        h=36,
        text="文本打分",
        origin='center',
        onclick=on_score_click
    )
    
    return gui, text_box

def run_flask():
    """运行Flask服务器"""
    try:
        ip = get_ip_address()
        server_info = """
╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://{ip}:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000                               ║
║                                                ║
║  请在浏览器中访问以上地址来输入范文          ║
║                                                ║
╚════════════════════════════════════════════════╝
""".format(ip=ip)
        
        for _ in range(3):
            print("\n" + server_info, flush=True)
            time.sleep(1)
        
        app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
    except Exception as e:
        print("\n" + "="*50)
        print("【服务器启动失败】")
        print(f"错误信息: {str(e)}")
        print("="*50 + "\n")

def get_ip_address():
    """获取本机IP地址"""
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "127.0.0.1"

class AsrWsClient:
    def __init__(self, appid, token, cluster):
        self.appid = appid
        self.token = token
        self.cluster = cluster
        self.ws_url = "wss://openspeech.bytedance.com/api/v2/asr"
        self.success_code = 1000
        self.uid = "streaming_asr_demo"
        self.workflow = "audio_in,resample,partition,vad,fe,decode,itn,nlu_punctuate"
        self.show_language = False
        self.show_utterances = True
        self.result_type = "single"
        self.format = "raw"
        self.rate = 16000
        self.language = "zh-CN"
        self.bits = 16
        self.channel = 1
        self.codec = "raw"
        self.auth_method = "token"

    def construct_request(self, reqid):
        return {
            'app': {
                'appid': self.appid,
                'cluster': self.cluster,
                'token': self.token,
            },
            'user': {
                'uid': self.uid
            },
            'request': {
                'reqid': reqid,
                'nbest': 1,
                'workflow': self.workflow,
                'show_language': self.show_language,
                'show_utterances': self.show_utterances,
                'result_type': self.result_type,
                'sequence': 1
            },
            'audio': {
                'format': self.format,
                'rate': self.rate,
                'language': self.language,
                'bits': self.bits,
                'channel': self.channel,
                'codec': self.codec
            }
        }

    def token_auth(self):
        return {'Authorization': f'Bearer; {self.token}'}

    async def process_microphone(self):
        """实时麦克风录音并识别"""
        global is_recording
        is_recording = True
        
        reqid = str(uuid.uuid4())
        request_params = self.construct_request(reqid)
        
        payload_bytes = str.encode(json.dumps(request_params))
        payload_bytes = gzip.compress(payload_bytes)
        full_request = bytearray(generate_full_default_header())
        full_request.extend(len(payload_bytes).to_bytes(4, 'big'))
        full_request.extend(payload_bytes)

        print("建立WebSocket连接...")
        async with websockets.connect(
            self.ws_url, 
            extra_headers=self.token_auth(), 
            max_size=1000000000
        ) as ws:
            await ws.send(full_request)
            response = await ws.recv()
            result = parse_response(response)
            print(f"初始化响应: {result}")
            
            if 'payload_msg' in result and result['payload_msg']['code'] == self.success_code:
                print("初始化成功")
                print("录音任务已启动")
                chunk_size = 9600
                
                with sd.InputStream(channels=1, samplerate=16000, dtype=np.int16, blocksize=chunk_size) as stream:
                    print("开始录音...")
                    while is_recording:
                        audio_data, overflowed = stream.read(chunk_size)
                        if overflowed:
                            print("警告：音频缓冲区溢出")
                            
                        audio_bytes = audio_data.tobytes()
                        compressed_audio = gzip.compress(audio_bytes)
                        
                        audio_request = bytearray(generate_audio_default_header())
                        audio_request.extend(len(compressed_audio).to_bytes(4, 'big'))
                        audio_request.extend(compressed_audio)
                        
                        await ws.send(audio_request)
                        response = await ws.recv()
                        result = parse_response(response)
                        
                        if 'payload_msg' in result and 'result' in result['payload_msg']:
                            utterances = result['payload_msg']['result'][0].get('utterances', [])
                            for utterance in utterances:
                                if not utterance['definite']:
                                    print(f"\r[识别中...] {utterance['text']}", end='', flush=True)
                                else:
                                    print(f"\n[最终结果] {utterance['text']}")
                                    update_recognition_text(utterance['text'])

# 添加其他必要的函数
def generate_full_default_header():
    return generate_header()

def generate_header(
    version=PROTOCOL_VERSION,
    message_type=CLIENT_FULL_REQUEST,
    message_type_specific_flags=NO_SEQUENCE,
    serial_method=JSON,
    compression_type=GZIP,
    reserved_data=0x00,
    extension_header=bytes()
):
    """生成请求头"""
    header = bytearray()
    header_size = int(len(extension_header) / 4) + 1
    header.append((version << 4) | header_size)
    header.append((message_type << 4) | message_type_specific_flags)
    header.append((serial_method << 4) | compression_type)
    header.append(reserved_data)
    header.extend(extension_header)
    return header

def generate_audio_default_header():
    """生成音频数据请求头"""
    return generate_header(message_type=CLIENT_AUDIO_ONLY_REQUEST)

def parse_response(res):
    """解析响应"""
    try:
        protocol_version = res[0] >> 4
        header_size = res[0] & 0x0f
        message_type = res[1] >> 4
        message_type_specific_flags = res[1] & 0x0f
        serialization_method = res[2] >> 4
        message_compression = res[2] & 0x0f
        reserved = res[3]
        header_extensions = res[4:header_size * 4]
        payload = res[header_size * 4:]
        result = {}
        payload_msg = None
        payload_size = 0
        
        if message_type == SERVER_FULL_RESPONSE:
            payload_size = int.from_bytes(payload[:4], "big", signed=True)
            payload_msg = payload[4:]
        elif message_type == SERVER_ACK:
            seq = int.from_bytes(payload[:4], "big", signed=True)
            result['seq'] = seq
            if len(payload) >= 8:
                payload_size = int.from_bytes(payload[4:8], "big", signed=False)
                payload_msg = payload[8:]
        elif message_type == SERVER_ERROR_RESPONSE:
            code = int.from_bytes(payload[:4], "big", signed=False)
            result['code'] = code
            payload_size = int.from_bytes(payload[4:8], "big", signed=False)
            payload_msg = payload[8:]
            
        if payload_msg is None:
            return result
            
        if message_compression == GZIP:
            payload_msg = gzip.decompress(payload_msg)
            
        if serialization_method == JSON:
            payload_msg = json.loads(str(payload_msg, "utf-8"))
        elif serialization_method != NO_SERIALIZATION:
            payload_msg = str(payload_msg, "utf-8")
            
        result['payload_msg'] = payload_msg
        result['payload_size'] = payload_size
        return result
    except Exception as e:
        return {"error": f"Failed to parse response: {str(e)}"}

def update_recognition_text(new_text, is_correction=False):
    """更新识别结果并保持滚动条在底部"""
    try:
        prefix = "[纠正结果] " if is_correction else ""
        all_texts.append(prefix + new_text)
        
        full_text = "\n".join(all_texts)
        text_box.config(text=full_text)
        text_box.text.see(END)
        
        if gui.master.winfo_exists():
            gui.update()
            
    except Exception as e:
        print(f"更新文本错误: {e}")

@app.errorhandler(404)
def not_found(error):
    return jsonify({"status": "error", "message": "接口不存在"}), 404

@app.errorhandler(500)
def server_error(error):
    return jsonify({"status": "error", "message": "服务器内部错误"}), 500

def cleanup():
    """程序退出时的清理工作"""
    global is_recording, loop
    try:
        is_recording = False
        if loop and not loop.is_closed():
            loop.close()
    except Exception as e:
        print(f"清理时出错: {e}")

if __name__ == '__main__':
    try:
        print("\n=== 程序启动顺序 ===")
        print("1. 启动Web服务器")
        print("2. 初始化GUI界面")
        print("3. 启动语音识别\n")
        
        # 1. 先启动Web服务器
        print("正在启动Web服务器...\n")
        flask_thread = threading.Thread(target=run_flask)
        flask_thread.daemon = True
        flask_thread.start()
        
        # 等待服务器启动
        time.sleep(3)
        
        # 2. 初始化板子
        print("\n正在初始化硬件...\n")
        Board().begin()
        
        # 3. 初始化GUI
        print("\n正在初始化GUI...\n")
        gui, text_box = init_gui()
        
        # 4. 创建客户端实例并启动语音识别
        print("\n正在启动语音识别...\n")
        client = AsrWsClient(appid, token, cluster)
        
        # 5. 创建事件循环
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        recognition_task = loop.create_task(client.process_microphone())
        
        # 6. 主循环
        while True:
            try:
                loop.run_until_complete(asyncio.sleep(0.1))
                
                # 检测按钮A（向上滚动）
                if button_a.is_pressed():
                    text_box.text.yview_scroll(-1, "units")
                    gui.update()
                    time.sleep(0.2)
                
                # 检测按钮B（向下滚动）
                if button_b.is_pressed():
                    text_box.text.yview_scroll(1, "units")
                    gui.update()
                    time.sleep(0.2)
                
                gui.update()
                
            except KeyboardInterrupt:
                print("\n程序被用户中断")
                break
            except Exception as e:
                print(f"\n循环中出现错误: {e}")
                break
                
    except Exception as e:
        print(f"\n程序异常: {e}")
    finally:
        cleanup()
        print("程序已正常退出")

# 设置API密钥
os.environ['DASHSCOPE_API_KEY'] = 'sk-7c04ee6f9432492bb344baa7a5c0162f'


10.1.2.101 - - [31/Oct/2024 14:26:00] "GET /get_score HTTP/1.1" 200 -



=== 程序启动顺序 ===
1. 启动Web服务器
2. 初始化GUI界面
3. 启动语音识别

正在启动Web服务器...



╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://172.16.5.104:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000                               ║
║                                                ║
║  请在浏览器中访问以上地址来输入范文          ║
║                                                ║
╚════════════════════════════════════════════════╝



10.1.2.101 - - [31/Oct/2024 14:26:01] "GET /get_score HTTP/1.1" 200 -




╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://172.16.5.104:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000                               ║
║                                                ║
║  请在浏览器中访问以上地址来输入范文          ║
║                                                ║
╚════════════════════════════════════════════════╝



10.1.2.101 - - [31/Oct/2024 14:26:02] "GET /get_score HTTP/1.1" 200 -


...


【服务器启动失败】
错误信息: [Errno 98] Address already in use


  ___________________________
 |                           |
 |      PinPong v0.5.2       |
 |    Designed by DFRobot    |
 |___________________________|
 
[01] Python3.7.3 Linux-4.4.143-67-rockchip-g01bbbc5d1312-aarch64-with-debian-10.11 Board: UNIHIKER
selected -> board: UNIHIKER serial: /dev/ttyS3
[10] Opening /dev/ttyS3
[32] Firmata ID: 3.8
[22] Arduino compatible device found and connected to /dev/ttyS3
[40] Retrieving analog map...


10.1.2.101 - - [31/Oct/2024 14:26:05] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:06] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:07] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:08] "GET /get_score HTTP/1.1" 200 -



程序异常: *** Analog map retrieval timed out. ***
Do you have Arduino connectivity and do you have the correct Firmata sketch uploaded to the board?
程序已正常退出


10.1.2.101 - - [31/Oct/2024 14:26:09] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:10] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:11] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:12] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:13] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:14] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:15] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:16] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:17] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:18] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:19] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:20] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:21] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:26:22] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/O

10.1.2.101 - - [31/Oct/2024 14:28:04] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:28:05] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:28:06] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:28:07] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:28:08] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:28:09] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:28:10] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:28:11] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:28:12] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:28:13] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:28:14] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:28:15] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:28:16] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 14:28:17] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/O

In [1]:
from flask import Flask, request, send_file, jsonify
from difflib import SequenceMatcher
import json
import uuid
import gzip
import asyncio
import websockets
import numpy as np
import sounddevice as sd
import nest_asyncio
from unihiker import GUI
import time
from tkinter import END
import requests
import os
import dashscope
import threading
import socket
import psutil
import sys

# Flask 应用
app = Flask(__name__)

# 共享数据存储
class SharedData:
    def __init__(self):
        self.speech_recognition_results = []
        self.latest_text = ""
        self.target_text = ""
        self.latest_score = None

shared_data = SharedData()

# Web 服务器路由
@app.route('/')
def index():
    return send_file('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    try:
        data = request.get_json()
        text = data.get('text', '')
        shared_data.target_text = text
        return jsonify({"status": "success", "message": "范文提交成功"})
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

@app.route('/get_score')
def get_score():
    if shared_data.latest_score:
        score_data = shared_data.latest_score
        # 返回后清除分数，避免重复获取
        shared_data.latest_score = None
        return jsonify({"status": "success", "data": score_data})
    return jsonify({"status": "waiting"})

# 相似度计算函数
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

def calculate_completeness(target_text, speech_text):
    target_words = set(target_text.split())
    speech_words = set(speech_text.split())
    common_words = target_words.intersection(speech_words)
    return len(common_words) / len(target_words) if target_words else 0

def get_similarity_comment(similarity):
    if similarity >= 0.9: return "非常接近"
    elif similarity >= 0.7: return "比较接近"
    elif similarity >= 0.5: return "部分接近"
    else: return "差异较大"

def get_completeness_comment(completeness):
    if completeness >= 0.9: return "内容完整"
    elif completeness >= 0.7: return "大部分完整"
    elif completeness >= 0.5: return "部分完整"
    else: return "内容缺失"

# 配置参数
appid = "4166554764"    # 项目的 appid
token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"    # 项目的 token
cluster = "volcengine_streaming_common"  # 请求的集群

# 协议常量
PROTOCOL_VERSION = 0b0001
DEFAULT_HEADER_SIZE = 0b0001

PROTOCOL_VERSION_BITS = 4
HEADER_BITS = 4
MESSAGE_TYPE_BITS = 4
MESSAGE_TYPE_SPECIFIC_FLAGS_BITS = 4
MESSAGE_SERIALIZATION_BITS = 4
MESSAGE_COMPRESSION_BITS = 4
RESERVED_BITS = 8

# Message Type:
CLIENT_FULL_REQUEST = 0b0001
CLIENT_AUDIO_ONLY_REQUEST = 0b0010
SERVER_FULL_RESPONSE = 0b1001
SERVER_ACK = 0b1011
SERVER_ERROR_RESPONSE = 0b1111

# Message Type Specific Flags
NO_SEQUENCE = 0b0000
POS_SEQUENCE = 0b0001
NEG_SEQUENCE = 0b0010
NEG_SEQUENCE_1 = 0b0011

# Message Serialization
NO_SERIALIZATION = 0b0000
JSON = 0b0001
THRIFT = 0b0011
CUSTOM_TYPE = 0b1111

# Message Compression
NO_COMPRESSION = 0b0000
GZIP = 0b0001
CUSTOM_COMPRESSION = 0b1111

# 初始化 nest_asyncio
nest_asyncio.apply()

# 全局变量
is_recording = True
all_texts = []
gui = None
loop = None  # 添加全局 loop 变量

def on_correction_click():
    """处理纠正按钮点击事件"""
    global is_recording
    try:
        print("纠正按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的当前文本：{current_text}")
        
        if current_text.strip():
            # 3. 调用API进行纠正
            print("开始调用API进行纠正...")
            corrected_text = call_qwen_api(current_text)
            print(f"API返回的纠正结果：{corrected_text}")
            
            # 4. 显示纠正结果
            update_recognition_text(corrected_text, is_correction=True)
            print("更新显示完成")
        else:
            print("文本框为空，不进行API调用")
            
    except Exception as e:
        error_msg = f"纠正文本错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg, is_correction=True)

def on_score_click():
    """处理打分按钮点击事件"""
    global is_recording
    try:
        print("打分按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的朗读文本：{current_text}")
        
        # 3. 获取范文
        target_text = shared_data.target_text
        if not target_text:
            error_msg = "[错误] 请先在网页端输入范文"
            print(error_msg)
            update_recognition_text(error_msg)
            return
            
        if not current_text.strip():
            error_msg = "[错误] 请先进行语音识别"
            print(error_msg)
            update_recognition_text(error_msg)
            return
        
        print("开始调用API进行评分...")
        # 4. 构造提示词
        prompt = f"""
请对比以下两段文本，从准确度、完整度、流畅度三维度进行评分（满100分），并给出详细分析：

范文：
{target_text}

朗读文本：
{current_text}

请按以下格式输出：
准确度：XX分
完整度：XX分
流畅度：XX分
总分：XX分

详细分析：
1. 准确度分析：...
2. 完整度分析：...
3. 流畅度分析：...
4. 改进建议：...
"""
        
        # 5. 调用API进行评分
        response = dashscope.Generation.call(
            api_key=os.getenv('DASHSCOPE_API_KEY'),
            model="qwen-plus",
            messages=[
                {'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': prompt}
            ],
            result_format='message'
        )
        
        if response.status_code == 200:
            result = response.output.choices[0].message.content
            print(f"评分结果：{result}")
            
            # 更新UNIHIKER显示
            update_recognition_text(f"[评分结果]\n{result}")
            
            # 更新共享数据供网页显示
            shared_data.latest_score = {
                "target_text": target_text,
                "speech_text": current_text,
                "score_result": result
            }
            
        else:
            error_msg = f"评分失败: {response.message}"
            print(error_msg)
            update_recognition_text(f"[错误] {error_msg}")
            
    except Exception as e:
        error_msg = f"评分错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg)

def init_gui():
    """初始化GUI，只调用一次"""
    global gui, text_box
    
    if gui is not None:
        print("GUI 已经初始化，跳过重复初始化")
        return gui, text_box
        
    # 初始化GUI
    gui = GUI()
    
    # 创建标题
    title = gui.draw_text(
        x=120,
        y=20,
        text="背诵助手",
        font_size=20,
        origin='center'
    )
    
    # 创建文本框
    text_box = gui.add_text_box(
        x=120,
        y=140,
        w=220,
        h=180,
        origin='center',
        font_size=12
    )
    
    # 创建纠正按钮 - 左边
    correction_button = gui.add_button(
        x=70,  # 移到左边
        y=290,  # 两个按钮在同一水平线上
        w=100,  # 减小按钮宽度
        h=36,
        text="纠正文本",
        origin='center',
        onclick=on_correction_click
    )
    
    # 创建打分按钮 - 右边
    score_button = gui.add_button(
        x=180,  # 移到右边
        y=290,  # 与纠正按钮在同一水平线上
        w=100,  # 减小按钮宽度
        h=36,
        text="文本打分",
        origin='center',
        onclick=on_score_click
    )
    
    return gui, text_box

def kill_existing_flask():
    """杀死已存在的Flask进程"""
    try:
        current_pid = os.getpid()
        
        for proc in psutil.process_iter(['pid', 'name', 'connections']):
            try:
                # 跳过当前进程
                if proc.pid == current_pid:
                    continue
                    
                for conn in proc.connections():
                    if conn.laddr.port == 5000:
                        print(f"发现端口5000被进程占用 (PID: {proc.pid})")
                        proc.kill()
                        print(f"已终止进程 {proc.pid}")
                        time.sleep(1)  # 等待进程完全终止
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                continue
    except Exception as e:
        print(f"清理进程时出错: {e}")

def run_flask():
    """运行Flask服务器"""
    try:
        # 先尝试释放端口
        try:
            import socket
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            sock.bind(('0.0.0.0', 5000))
            sock.close()
        except Exception as e:
            print(f"端口绑定测试失败: {e}")
            # 尝试结束占用端口的进程
            try:
                import os
                os.system("fuser -k 5000/tcp")
                print("已尝试释放端口5000")
                time.sleep(2)
            except:
                print("无法释放端口，请手动结束占用端口的进程")
                return False

        ip = get_ip_address()
        server_info = """
╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://{ip}:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000                               ║
║                                                ║
║  请在浏览器中访问以上地址来输入范文          ║
║                                                ║
╚════════════════════════════════════════════════╝
""".format(ip=ip)
        
        print("\n" + server_info, flush=True)
        
        app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
        return True
    except Exception as e:
        print("\n" + "="*50)
        print("【服务器启动失败】")
        print(f"错误信息: {str(e)}")
        print("="*50 + "\n")
        return False

def get_ip_address():
    """获取本机IP地址"""
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "127.0.0.1"

class AsrWsClient:
    def __init__(self, appid, token, cluster):
        self.appid = appid
        self.token = token
        self.cluster = cluster
        self.ws_url = "wss://openspeech.bytedance.com/api/v2/asr"
        self.success_code = 1000
        self.uid = "streaming_asr_demo"
        self.workflow = "audio_in,resample,partition,vad,fe,decode,itn,nlu_punctuate"
        self.show_language = False
        self.show_utterances = True
        self.result_type = "single"
        self.format = "raw"
        self.rate = 16000
        self.language = "zh-CN"
        self.bits = 16
        self.channel = 1
        self.codec = "raw"
        self.auth_method = "token"

    def construct_request(self, reqid):
        return {
            'app': {
                'appid': self.appid,
                'cluster': self.cluster,
                'token': self.token,
            },
            'user': {
                'uid': self.uid
            },
            'request': {
                'reqid': reqid,
                'nbest': 1,
                'workflow': self.workflow,
                'show_language': self.show_language,
                'show_utterances': self.show_utterances,
                'result_type': self.result_type,
                'sequence': 1
            },
            'audio': {
                'format': self.format,
                'rate': self.rate,
                'language': self.language,
                'bits': self.bits,
                'channel': self.channel,
                'codec': self.codec
            }
        }

    def token_auth(self):
        return {'Authorization': f'Bearer; {self.token}'}

    async def process_microphone(self):
        """实时麦克风录音并识别"""
        global is_recording
        is_recording = True
        
        reqid = str(uuid.uuid4())
        request_params = self.construct_request(reqid)
        
        payload_bytes = str.encode(json.dumps(request_params))
        payload_bytes = gzip.compress(payload_bytes)
        full_request = bytearray(generate_full_default_header())
        full_request.extend(len(payload_bytes).to_bytes(4, 'big'))
        full_request.extend(payload_bytes)

        print("建立WebSocket连接...")
        async with websockets.connect(
            self.ws_url, 
            extra_headers=self.token_auth(), 
            max_size=1000000000
        ) as ws:
            await ws.send(full_request)
            response = await ws.recv()
            result = parse_response(response)
            print(f"初始化响应: {result}")
            
            if 'payload_msg' in result and result['payload_msg']['code'] == self.success_code:
                print("初始化成功")
                print("录音任务已启动")
                chunk_size = 9600
                
                with sd.InputStream(channels=1, samplerate=16000, dtype=np.int16, blocksize=chunk_size) as stream:
                    print("开始录音...")
                    while is_recording:
                        audio_data, overflowed = stream.read(chunk_size)
                        if overflowed:
                            print("警告：音频缓冲区溢出")
                            
                        audio_bytes = audio_data.tobytes()
                        compressed_audio = gzip.compress(audio_bytes)
                        
                        audio_request = bytearray(generate_audio_default_header())
                        audio_request.extend(len(compressed_audio).to_bytes(4, 'big'))
                        audio_request.extend(compressed_audio)
                        
                        await ws.send(audio_request)
                        response = await ws.recv()
                        result = parse_response(response)
                        
                        if 'payload_msg' in result and 'result' in result['payload_msg']:
                            utterances = result['payload_msg']['result'][0].get('utterances', [])
                            for utterance in utterances:
                                if not utterance['definite']:
                                    print(f"\r[识别中...] {utterance['text']}", end='', flush=True)
                                else:
                                    print(f"\n[最终结果] {utterance['text']}")
                                    update_recognition_text(utterance['text'])

# 添加其他必要的函数
def generate_full_default_header():
    return generate_header()

def generate_header(
    version=PROTOCOL_VERSION,
    message_type=CLIENT_FULL_REQUEST,
    message_type_specific_flags=NO_SEQUENCE,
    serial_method=JSON,
    compression_type=GZIP,
    reserved_data=0x00,
    extension_header=bytes()
):
    """生成请求头"""
    header = bytearray()
    header_size = int(len(extension_header) / 4) + 1
    header.append((version << 4) | header_size)
    header.append((message_type << 4) | message_type_specific_flags)
    header.append((serial_method << 4) | compression_type)
    header.append(reserved_data)
    header.extend(extension_header)
    return header

def generate_audio_default_header():
    """生成音频数据请求头"""
    return generate_header(message_type=CLIENT_AUDIO_ONLY_REQUEST)

def parse_response(res):
    """解析响应"""
    try:
        protocol_version = res[0] >> 4
        header_size = res[0] & 0x0f
        message_type = res[1] >> 4
        message_type_specific_flags = res[1] & 0x0f
        serialization_method = res[2] >> 4
        message_compression = res[2] & 0x0f
        reserved = res[3]
        header_extensions = res[4:header_size * 4]
        payload = res[header_size * 4:]
        result = {}
        payload_msg = None
        payload_size = 0
        
        if message_type == SERVER_FULL_RESPONSE:
            payload_size = int.from_bytes(payload[:4], "big", signed=True)
            payload_msg = payload[4:]
        elif message_type == SERVER_ACK:
            seq = int.from_bytes(payload[:4], "big", signed=True)
            result['seq'] = seq
            if len(payload) >= 8:
                payload_size = int.from_bytes(payload[4:8], "big", signed=False)
                payload_msg = payload[8:]
        elif message_type == SERVER_ERROR_RESPONSE:
            code = int.from_bytes(payload[:4], "big", signed=False)
            result['code'] = code
            payload_size = int.from_bytes(payload[4:8], "big", signed=False)
            payload_msg = payload[8:]
            
        if payload_msg is None:
            return result
            
        if message_compression == GZIP:
            payload_msg = gzip.decompress(payload_msg)
            
        if serialization_method == JSON:
            payload_msg = json.loads(str(payload_msg, "utf-8"))
        elif serialization_method != NO_SERIALIZATION:
            payload_msg = str(payload_msg, "utf-8")
            
        result['payload_msg'] = payload_msg
        result['payload_size'] = payload_size
        return result
    except Exception as e:
        return {"error": f"Failed to parse response: {str(e)}"}

def update_recognition_text(new_text, is_correction=False):
    """更新识别结果并保持滚动条在底部"""
    try:
        prefix = "[纠正结果] " if is_correction else ""
        all_texts.append(prefix + new_text)
        
        full_text = "\n".join(all_texts)
        text_box.config(text=full_text)
        text_box.text.see(END)
        
        if gui.master.winfo_exists():
            gui.update()
            
    except Exception as e:
        print(f"更新文本错误: {e}")

@app.errorhandler(404)
def not_found(error):
    return jsonify({"status": "error", "message": "接口不存在"}), 404

@app.errorhandler(500)
def server_error(error):
    return jsonify({"status": "error", "message": "服务器内部错误"}), 500

def cleanup():
    """程序退出时的清理工作"""
    global is_recording, loop
    try:
        is_recording = False
        if loop and not loop.is_closed():
            loop.close()
    except Exception as e:
        print(f"清理时出错: {e}")

if __name__ == '__main__':
    try:
        print("\n=== 程序启动顺序 ===")
        print("1. 启动Web服务器")
        print("2. 初始化GUI界面")
        print("3. 启动语音识别\n")
        
        # 1. 先启动Web服务器
        print("正在启动Web服务器...\n")
        flask_thread = threading.Thread(target=run_flask)
        flask_thread.daemon = True
        flask_thread.start()
        
        # 等待确保服务器启动
        time.sleep(5)  # 增加等待时间
        
        # 检查服务器是否成功启动
        try:
            response = requests.get('http://localhost:5000', timeout=3)
            if response.status_code != 200:
                raise Exception("服务器响应异常")
        except Exception as e:
            print(f"服务器启动检查失败: {e}")
            print("请确保端口5000未被占用")
            sys.exit(1)
        
        # 2. 初始化GUI
        print("\n正在初始化GUI...\n")
        gui, text_box = init_gui()
        
        # 3. 创建客户端实例并启动语音识别
        print("\n正在启动语音识别...\n")
        client = AsrWsClient(appid, token, cluster)
        
        # 4. 创建事件循环
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        recognition_task = loop.create_task(client.process_microphone())
        
        # 5. 主循环
        while True:
            try:
                loop.run_until_complete(asyncio.sleep(0.1))
                gui.update()
                
            except KeyboardInterrupt:
                print("\n程序被用户中断")
                break
            except Exception as e:
                print(f"\n循环中出现错误: {e}")
                break
                
    except Exception as e:
        print(f"\n程序异常: {e}")
    finally:
        try:
            if 'loop' in locals() and loop and not loop.is_closed():
                loop.close()
        except Exception as e:
            print(f"关闭事件循环时出错: {e}")
        print("程序已正常退出")

# 设置API密钥
os.environ['DASHSCOPE_API_KEY'] = 'sk-7c04ee6f9432492bb344baa7a5c0162f'



=== 程序启动顺序 ===
1. 启动Web服务器
2. 初始化GUI界面
3. 启动语音识别

正在启动Web服务器...



╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://172.16.5.104:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000                               ║
║                                                ║
║  请在浏览器中访问以上地址来输入范文          ║
║                                                ║
╚════════════════════════════════════════════════╝

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.16.5.104:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Oct/2024 16:22:08] "GET / HTTP/1.1" 200 -



正在初始化GUI...


正在启动语音识别...

建立WebSocket连接...
初始化响应: {'payload_msg': {'addition': {'duration': '0', 'logid': '202410311622095D68DFCD2EA408272ED2'}, 'code': 1000, 'message': 'Success', 'reqid': '5ac060b8-be4c-4380-8172-8d85fb179273', 'sequence': 1}, 'payload_size': 160}
初始化成功
录音任务已启动
开始录音...


10.1.2.101 - - [31/Oct/2024 16:22:24] "GET /get_score HTTP/1.1" 200 -


[识别中...] 我轻轻地招手
[最终结果] 我轻轻的招手。
[识别中...] 作别
[最终结果] 作别。
[识别中...] 西天的云在
[最终结果] 西天的云在？
[识别中...] 那河畔的金柳
[最终结果] 那河畔的金柳，
[识别中...] 是夕阳中的新娘
[最终结果] 是夕阳中的新娘。
[识别中...] 波光里的艳影在我的心头荡漾

...

获取到的当前文本：我轻轻的招手。
作别。
西天的云在？
那河畔的金柳，
是夕阳中的新娘。
波光里的艳影在我的心头荡漾。
软泥上的清醒，油油的在水底招摇。
在康河的柔波里，我甘心做一条水草。
那榆荫下的一潭，
不是清泉，是天上红。
揉碎在浮藻间，沉淀着彩虹似的梦。
漩涡。
成一支长高像青草，更清楚曼索。
满载一船星辉，
在星辉斑斓里放歌！
但我。
放歌！
悄悄是别离的笙箫，夏虫也为我沉默沉默。
是今晚的康桥！
悄悄的。

开始调用API进行纠正...
纠正文本错误: name 'call_qwen_api' is not defined


10.1.2.101 - - [31/Oct/2024 16:24:17] "GET /get_score HTTP/1.1" 200 -


打分按钮被点击
正在停止语音识别...
获取到的朗读文本：我轻轻的招手。
作别。
西天的云在？
那河畔的金柳，
是夕阳中的新娘。
波光里的艳影在我的心头荡漾。
软泥上的清醒，油油的在水底招摇。
在康河的柔波里，我甘心做一条水草。
那榆荫下的一潭，
不是清泉，是天上红。
揉碎在浮藻间，沉淀着彩虹似的梦。
漩涡。
成一支长高像青草，更清楚曼索。
满载一船星辉，
在星辉斑斓里放歌！
但我。
放歌！
悄悄是别离的笙箫，夏虫也为我沉默沉默。
是今晚的康桥！
悄悄的。
我走。
[纠正结果] 纠正文本错误: name 'call_qwen_api' is not defined

[错误] 请先在网页端输入范文


10.1.2.101 - - [31/Oct/2024 16:24:18] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:24:20] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:24:22] "GET /get_score HTTP/1.1" 200 -



程序被用户中断
程序已正常退出


In [ ]:
能正常语音识别，无法纠错，网页访问正常，打分按钮按下没反应

In [1]:
from flask import Flask, request, send_file, jsonify
from difflib import SequenceMatcher
import json
import uuid
import gzip
import asyncio
import websockets
import numpy as np
import sounddevice as sd
import nest_asyncio
from unihiker import GUI
import time
from tkinter import END
import requests
import os
import dashscope
import threading
import socket
import psutil
import sys

# Flask 应用
app = Flask(__name__)

# 共享数据存储
class SharedData:
    def __init__(self):
        self.speech_recognition_results = []
        self.latest_text = ""
        self.target_text = ""
        self.latest_score = None

shared_data = SharedData()

# Web 服务器路由
@app.route('/')
def index():
    return send_file('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    try:
        data = request.get_json()
        text = data.get('text', '')
        shared_data.target_text = text
        return jsonify({"status": "success", "message": "范文提交成功"})
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

@app.route('/get_score')
def get_score():
    if shared_data.latest_score:
        score_data = shared_data.latest_score
        # 返回后清除分数，避免重复获取
        shared_data.latest_score = None
        return jsonify({"status": "success", "data": score_data})
    return jsonify({"status": "waiting"})

# 相似度计算函数
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

def calculate_completeness(target_text, speech_text):
    target_words = set(target_text.split())
    speech_words = set(speech_text.split())
    common_words = target_words.intersection(speech_words)
    return len(common_words) / len(target_words) if target_words else 0

def get_similarity_comment(similarity):
    if similarity >= 0.9: return "非常接近"
    elif similarity >= 0.7: return "比较接近"
    elif similarity >= 0.5: return "部分接近"
    else: return "差异较大"

def get_completeness_comment(completeness):
    if completeness >= 0.9: return "内容完整"
    elif completeness >= 0.7: return "大部分完整"
    elif completeness >= 0.5: return "部分完整"
    else: return "内容缺失"

# 配置参数
appid = "4166554764"    # 项目的 appid
token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"    # 项目的 token
cluster = "volcengine_streaming_common"  # 请求的集群

# 协议常量
PROTOCOL_VERSION = 0b0001
DEFAULT_HEADER_SIZE = 0b0001

PROTOCOL_VERSION_BITS = 4
HEADER_BITS = 4
MESSAGE_TYPE_BITS = 4
MESSAGE_TYPE_SPECIFIC_FLAGS_BITS = 4
MESSAGE_SERIALIZATION_BITS = 4
MESSAGE_COMPRESSION_BITS = 4
RESERVED_BITS = 8

# Message Type:
CLIENT_FULL_REQUEST = 0b0001
CLIENT_AUDIO_ONLY_REQUEST = 0b0010
SERVER_FULL_RESPONSE = 0b1001
SERVER_ACK = 0b1011
SERVER_ERROR_RESPONSE = 0b1111

# Message Type Specific Flags
NO_SEQUENCE = 0b0000
POS_SEQUENCE = 0b0001
NEG_SEQUENCE = 0b0010
NEG_SEQUENCE_1 = 0b0011

# Message Serialization
NO_SERIALIZATION = 0b0000
JSON = 0b0001
THRIFT = 0b0011
CUSTOM_TYPE = 0b1111

# Message Compression
NO_COMPRESSION = 0b0000
GZIP = 0b0001
CUSTOM_COMPRESSION = 0b1111

# 初始化 nest_asyncio
nest_asyncio.apply()

# 全局变量
is_recording = True
all_texts = []
gui = None
loop = None  # 添加全局 loop 变量

# 在程序最开始处（所有 import 语句之后）设置 API key
def init_api_keys():
    """初始化API密钥"""
    try:
        # 语音识别配置
        global appid, token, cluster
        appid = "4166554764"
        token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"
        cluster = "volcengine_streaming_common"
        
        # 千问API配置
        dashscope_key = 'sk-7c04ee6f9432492bb344baa7a5c0162f'
        os.environ['DASHSCOPE_API_KEY'] = dashscope_key
        dashscope.api_key = dashscope_key
        
        # 验证千问API key是否设置成功
        print(f"当前 DashScope API Key: {dashscope.api_key}")
        return True
        
    except Exception as e:
        print(f"API密钥初始化失败: {e}")
        return False

def call_qwen_api(text):
    """调用千问API进行文本纠错"""
    try:
        if not dashscope.api_key:
            print("[错误] DashScope API Key未设置")
            return None
            
        print(f"使用的API Key: {dashscope.api_key}")
        
        messages = [
            {
                'role': 'system',
                'content': '你是一个专业的文本纠错助手。请对输入的文本进行标点符号和错别字的修正，保持原文的意思不变。'
            },
            {
                'role': 'user',
                'content': f'请修正以下文本的标点符号和错别字：\n{text}'
            }
        ]
        
        response = dashscope.Generation.call(
            model='qwen-plus',
            messages=messages,
            result_format='message',
            api_key=dashscope.api_key  # 显式传递API key
        )
        
        if response.status_code == 200:
            corrected_text = response.output.choices[0].message.content.strip()
            print(f"[纠正结果] {corrected_text}")
            return corrected_text
        else:
            error_msg = f"API调用失败: {response.code} - {response.message}"
            print(f"[错误] {error_msg}")
            return None
            
    except Exception as e:
        error_msg = f"纠错API调用失败: {str(e)}"
        print(f"[错误] {error_msg}")
        return None

def on_correction_click():
    """处理纠正按钮点击事件"""
    global is_recording
    try:
        print("纠正按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的当前文本：{current_text}")
        
        if current_text.strip():
            # 3. 调用API进行纠正
            print("开始调用API进行纠正...")
            corrected_text = call_qwen_api(current_text)
            print(f"API返回的纠正结果：{corrected_text}")
            
            # 4. 显示纠正结果
            update_recognition_text(corrected_text, is_correction=True)
            print("更新显示完成")
        else:
            print("文本框为空，不进行API调用")
            
    except Exception as e:
        error_msg = f"纠正文本错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg, is_correction=True)

def on_score_click():
    """处理打分按钮点击事件"""
    global is_recording
    try:
        print("打分按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的朗读文本：{current_text}")
        
        # 3. 获取范文
        target_text = shared_data.target_text
        if not target_text:
            error_msg = "[错误] 请先在网页端输入范文"
            print(error_msg)
            update_recognition_text(error_msg)
            return
            
        if not current_text.strip():
            error_msg = "[错误] 请先进行语音识别"
            print(error_msg)
            update_recognition_text(error_msg)
            return
        
        print("开始调用API进行评分...")
        # 4. 构造提示词
        prompt = f"""
请对比以下两段文本，从准确度、完整度、流畅度三维度进行评分（满100分），并给出详细分析：

范文：
{target_text}

朗读文本：
{current_text}

请按以下格式输出：
准确度：XX分
完整度：XX分
流畅度：XX分
总分：XX分

详细分析：
1. 准确度分析：...
2. 完整度分析：...
3. 流畅度分析：...
4. 改进建议：...
"""
        
        # 5. 调用API进行评分
        response = dashscope.Generation.call(
            api_key=os.getenv('DASHSCOPE_API_KEY'),
            model="qwen-plus",
            messages=[
                {'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': prompt}
            ],
            result_format='message'
        )
        
        if response.status_code == 200:
            result = response.output.choices[0].message.content
            print(f"评分结果：{result}")
            
            # 更新UNIHIKER显示
            update_recognition_text(f"[评分结果]\n{result}")
            
            # 更新共享数据供网页显示
            shared_data.latest_score = {
                "target_text": target_text,
                "speech_text": current_text,
                "score_result": result
            }
            
        else:
            error_msg = f"评分失败: {response.message}"
            print(error_msg)
            update_recognition_text(f"[错误] {error_msg}")
            
    except Exception as e:
        error_msg = f"评分错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg)

def init_gui():
    """初始化GUI，只调用一次"""
    global gui, text_box
    
    if gui is not None:
        print("GUI 已经初始化，跳过重复初始化")
        return gui, text_box
        
    # 初始化GUI
    gui = GUI()
    
    # 创建标题
    title = gui.draw_text(
        x=120,
        y=20,
        text="背诵助手",
        font_size=20,
        origin='center'
    )
    
    # 创建文本框
    text_box = gui.add_text_box(
        x=120,
        y=140,
        w=220,
        h=180,
        origin='center',
        font_size=12
    )
    
    # 创建纠正按钮 - 左边
    correction_button = gui.add_button(
        x=70,  # 移到左边
        y=290,  # 两个按钮在同一水平线上
        w=100,  # 减小按钮宽度
        h=36,
        text="纠正文本",
        origin='center',
        onclick=on_correction_click
    )
    
    # 创建打分按钮 - 右边
    score_button = gui.add_button(
        x=180,  # 移到右边
        y=290,  # 与纠正按钮在同一水平线上
        w=100,  # 减小按钮宽度
        h=36,
        text="文本打分",
        origin='center',
        onclick=on_score_click
    )
    
    return gui, text_box

def kill_existing_flask():
    """杀死已存在的Flask进程"""
    try:
        current_pid = os.getpid()
        
        for proc in psutil.process_iter(['pid', 'name', 'connections']):
            try:
                # 跳过当前进程
                if proc.pid == current_pid:
                    continue
                    
                for conn in proc.connections():
                    if conn.laddr.port == 5000:
                        print(f"发现端口5000被进程占用 (PID: {proc.pid})")
                        proc.kill()
                        print(f"已终止进程 {proc.pid}")
                        time.sleep(1)  # 等待进程完全终止
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                continue
    except Exception as e:
        print(f"清理进程时出错: {e}")

def run_flask():
    """运行Flask服务器"""
    try:
        # 先尝试释放端口
        try:
            import socket
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            sock.bind(('0.0.0.0', 5000))
            sock.close()
        except Exception as e:
            print(f"端口绑定测试失败: {e}")
            # 尝试结束占用端口的进程
            try:
                import os
                os.system("fuser -k 5000/tcp")
                print("已尝试释放端口5000")
                time.sleep(2)
            except:
                print("无法释放端口，请手动结束占用端口的进程")
                return False

        ip = get_ip_address()
        server_info = """
╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://{ip}:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000                               ║
║                                                ║
║  请在浏览器中访问以上地址来输入范文          ║
║                                                ║
╚════════════════════════════════════════════════╝
""".format(ip=ip)
        
        print("\n" + server_info, flush=True)
        
        app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
        return True
    except Exception as e:
        print("\n" + "="*50)
        print("【服务器启动失败】")
        print(f"错误信息: {str(e)}")
        print("="*50 + "\n")
        return False

def get_ip_address():
    """获取本机IP地址"""
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "127.0.0.1"

class AsrWsClient:
    def __init__(self, appid, token, cluster):
        self.appid = appid
        self.token = token
        self.cluster = cluster
        self.ws_url = "wss://openspeech.bytedance.com/api/v2/asr"
        self.success_code = 1000
        self.uid = "streaming_asr_demo"
        self.workflow = "audio_in,resample,partition,vad,fe,decode,itn,nlu_punctuate"
        self.show_language = False
        self.show_utterances = True
        self.result_type = "single"
        self.format = "raw"
        self.rate = 16000
        self.language = "zh-CN"
        self.bits = 16
        self.channel = 1
        self.codec = "raw"
        self.auth_method = "token"

    def construct_request(self, reqid):
        return {
            'app': {
                'appid': self.appid,
                'cluster': self.cluster,
                'token': self.token,
            },
            'user': {
                'uid': self.uid
            },
            'request': {
                'reqid': reqid,
                'nbest': 1,
                'workflow': self.workflow,
                'show_language': self.show_language,
                'show_utterances': self.show_utterances,
                'result_type': self.result_type,
                'sequence': 1
            },
            'audio': {
                'format': self.format,
                'rate': self.rate,
                'language': self.language,
                'bits': self.bits,
                'channel': self.channel,
                'codec': self.codec
            }
        }

    def token_auth(self):
        return {'Authorization': f'Bearer; {self.token}'}

    async def process_microphone(self):
        """实时麦克风录音并识别"""
        global is_recording
        is_recording = True
        
        reqid = str(uuid.uuid4())
        request_params = self.construct_request(reqid)
        
        payload_bytes = str.encode(json.dumps(request_params))
        payload_bytes = gzip.compress(payload_bytes)
        full_request = bytearray(generate_full_default_header())
        full_request.extend(len(payload_bytes).to_bytes(4, 'big'))
        full_request.extend(payload_bytes)

        print("建立WebSocket连接...")
        async with websockets.connect(
            self.ws_url, 
            extra_headers=self.token_auth(), 
            max_size=1000000000
        ) as ws:
            await ws.send(full_request)
            response = await ws.recv()
            result = parse_response(response)
            print(f"初始化响应: {result}")
            
            if 'payload_msg' in result and result['payload_msg']['code'] == self.success_code:
                print("初始化成功")
                print("录音任务已启动")
                chunk_size = 9600
                
                with sd.InputStream(channels=1, samplerate=16000, dtype=np.int16, blocksize=chunk_size) as stream:
                    print("开始录音...")
                    while is_recording:
                        audio_data, overflowed = stream.read(chunk_size)
                        if overflowed:
                            print("警告：音频缓冲区溢出")
                            
                        audio_bytes = audio_data.tobytes()
                        compressed_audio = gzip.compress(audio_bytes)
                        
                        audio_request = bytearray(generate_audio_default_header())
                        audio_request.extend(len(compressed_audio).to_bytes(4, 'big'))
                        audio_request.extend(compressed_audio)
                        
                        await ws.send(audio_request)
                        response = await ws.recv()
                        result = parse_response(response)
                        
                        if 'payload_msg' in result and 'result' in result['payload_msg']:
                            utterances = result['payload_msg']['result'][0].get('utterances', [])
                            for utterance in utterances:
                                if not utterance['definite']:
                                    print(f"\r[识别中...] {utterance['text']}", end='', flush=True)
                                else:
                                    print(f"\n[最终结果] {utterance['text']}")
                                    update_recognition_text(utterance['text'])

# 添加其他必要的函数
def generate_full_default_header():
    return generate_header()

def generate_header(
    version=PROTOCOL_VERSION,
    message_type=CLIENT_FULL_REQUEST,
    message_type_specific_flags=NO_SEQUENCE,
    serial_method=JSON,
    compression_type=GZIP,
    reserved_data=0x00,
    extension_header=bytes()
):
    """生成请求头"""
    header = bytearray()
    header_size = int(len(extension_header) / 4) + 1
    header.append((version << 4) | header_size)
    header.append((message_type << 4) | message_type_specific_flags)
    header.append((serial_method << 4) | compression_type)
    header.append(reserved_data)
    header.extend(extension_header)
    return header

def generate_audio_default_header():
    """生成音频数据请求头"""
    return generate_header(message_type=CLIENT_AUDIO_ONLY_REQUEST)

def parse_response(res):
    """解析响应"""
    try:
        protocol_version = res[0] >> 4
        header_size = res[0] & 0x0f
        message_type = res[1] >> 4
        message_type_specific_flags = res[1] & 0x0f
        serialization_method = res[2] >> 4
        message_compression = res[2] & 0x0f
        reserved = res[3]
        header_extensions = res[4:header_size * 4]
        payload = res[header_size * 4:]
        result = {}
        payload_msg = None
        payload_size = 0
        
        if message_type == SERVER_FULL_RESPONSE:
            payload_size = int.from_bytes(payload[:4], "big", signed=True)
            payload_msg = payload[4:]
        elif message_type == SERVER_ACK:
            seq = int.from_bytes(payload[:4], "big", signed=True)
            result['seq'] = seq
            if len(payload) >= 8:
                payload_size = int.from_bytes(payload[4:8], "big", signed=False)
                payload_msg = payload[8:]
        elif message_type == SERVER_ERROR_RESPONSE:
            code = int.from_bytes(payload[:4], "big", signed=False)
            result['code'] = code
            payload_size = int.from_bytes(payload[4:8], "big", signed=False)
            payload_msg = payload[8:]
            
        if payload_msg is None:
            return result
            
        if message_compression == GZIP:
            payload_msg = gzip.decompress(payload_msg)
            
        if serialization_method == JSON:
            payload_msg = json.loads(str(payload_msg, "utf-8"))
        elif serialization_method != NO_SERIALIZATION:
            payload_msg = str(payload_msg, "utf-8")
            
        result['payload_msg'] = payload_msg
        result['payload_size'] = payload_size
        return result
    except Exception as e:
        return {"error": f"Failed to parse response: {str(e)}"}

def update_recognition_text(new_text, is_correction=False):
    """更新识别结果并保持滚动条在底部"""
    try:
        prefix = "[纠正结果] " if is_correction else ""
        all_texts.append(prefix + new_text)
        
        full_text = "\n".join(all_texts)
        text_box.config(text=full_text)
        text_box.text.see(END)
        
        if gui.master.winfo_exists():
            gui.update()
            
    except Exception as e:
        print(f"更新文本错误: {e}")

@app.errorhandler(404)
def not_found(error):
    return jsonify({"status": "error", "message": "接口不存在"}), 404

@app.errorhandler(500)
def server_error(error):
    return jsonify({"status": "error", "message": "服务器内部错误"}), 500

def cleanup():
    """程序退出时的清理工作"""
    global is_recording, loop
    try:
        is_recording = False
        if loop and not loop.is_closed():
            loop.close()
    except Exception as e:
        print(f"清理时出错: {e}")

if __name__ == '__main__':
    try:
        print("\n=== 程序启动顺序 ===")
        print("0. 初始化API密钥")
        print("1. 启动Web服务器")
        print("2. 初始化GUI界面")
        print("3. 启动语音识别\n")
        
        # 首先初始化API密钥
        if not init_api_keys():
            print("API密钥初始化失败，程序退出")
            sys.exit(1)
            
        # 1. 先启动Web服务器
        print("正在启动Web服务器...\n")
        flask_thread = threading.Thread(target=run_flask)
        flask_thread.daemon = True
        flask_thread.start()
        
        # 等待确保服务器启动
        time.sleep(5)  # 增加等待时间
        
        # 检查服务器是否成功启动
        try:
            response = requests.get('http://localhost:5000', timeout=3)
            if response.status_code != 200:
                raise Exception("服务器响应异常")
        except Exception as e:
            print(f"服务器启动检查失败: {e}")
            print("请确保端口5000未被占用")
            sys.exit(1)
        
        # 2. 初始化GUI
        print("\n正在初始化GUI...\n")
        gui, text_box = init_gui()
        
        # 3. 创建客户端实例并启动语音识别
        print("\n正在启动语音识别...\n")
        client = AsrWsClient(appid, token, cluster)
        
        # 4. 创建事件循环
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        recognition_task = loop.create_task(client.process_microphone())
        
        # 5. 主循环
        while True:
            try:
                loop.run_until_complete(asyncio.sleep(0.1))
                gui.update()
                
            except KeyboardInterrupt:
                print("\n程序被用户中断")
                break
            except Exception as e:
                print(f"\n循环中出现错误: {e}")
                break
                
    except Exception as e:
        print(f"\n程序异常: {e}")
    finally:
        try:
            if 'loop' in locals() and loop and not loop.is_closed():
                loop.close()
        except Exception as e:
            print(f"关闭事件循环时出错: {e}")
        print("程序已正常退出")

# 设置API密钥
os.environ['DASHSCOPE_API_KEY'] = 'sk-7c04ee6f9432492bb344baa7a5c0162f'


=== 程序启动顺序 ===
0. 初始化API密钥
1. 启动Web服务器
2. 初始化GUI界面
3. 启动语音识别

当前 DashScope API Key: sk-7c04ee6f9432492bb344baa7a5c0162f
正在启动Web服务器...



╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://172.16.5.104:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000                               ║
║                                                ║
║  请在浏览器中访问以上地址来输入范文          ║
║                                                ║
╚════════════════════════════════════════════════╝

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.16.5.104:5000/ (Press CTRL+C to quit)
10.1.2.101 - - [31/Oct/2024 16:41:57] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:41:57] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:41:57] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:41:57] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:41:58] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:41:59] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:42:00] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:42:01] "GET /get_score HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2024 16:42:01] "GET / HTTP/1.1" 200 -



正在初始化GUI...



10.1.2.101 - - [31/Oct/2024 16:42:02] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:42:03] "GET /get_score HTTP/1.1" 200 -



正在启动语音识别...

建立WebSocket连接...
初始化响应: {'payload_msg': {'addition': {'duration': '0', 'logid': '20241031164203A6A0EBA94EA7E22ABDEF'}, 'code': 1000, 'message': 'Success', 'reqid': '4667f602-f62a-41b4-8d48-f1a7df509e81', 'sequence': 1}, 'payload_size': 158}
初始化成功
录音任务已启动
开始录音...


...

10.1.2.101 - - [31/Oct/2024 16:43:05] "GET /get_score HTTP/1.1" 200 -


获取到的朗读文本：再别康桥！
徐志摩。
轻轻的。
我走了。
正如我。
轻轻的来。
我轻轻的招手。
[纠正结果] 再别康桥。
徐志摩
轻轻的，
我走了，
正如我，
轻轻的来；
我轻轻的招手，

[错误] 请先在网页端输入范文


10.1.2.101 - - [31/Oct/2024 16:43:06] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:43:07] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:43:08] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:43:09] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:43:10] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:43:11] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:43:12] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:43:13] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:43:14] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:43:14] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:43:15] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:43:16] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:43:18] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:43:19] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/O

10.1.2.101 - - [31/Oct/2024 16:45:04] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:45:05] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:45:05] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:45:07] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:45:08] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:45:09] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:45:10] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:45:11] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:45:12] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:45:13] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:45:14] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:45:15] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:45:16] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:45:17] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/O


程序被用户中断
程序已正常退出


In [ ]:
修正了纠正按钮报错的问题，文本打分按钮任然无效

In [1]:
from flask import Flask, request, send_file, jsonify
from difflib import SequenceMatcher
import json
import uuid
import gzip
import asyncio
import websockets
import numpy as np
import sounddevice as sd
import nest_asyncio
from unihiker import GUI
import time
from tkinter import END
import requests
import os
import dashscope
import threading
import socket
import psutil
import sys

# Flask 应用
app = Flask(__name__)

# 共享数据存储
class SharedData:
    def __init__(self):
        self.speech_recognition_results = []
        self.latest_text = ""
        self.target_text = ""
        self.latest_score = None

shared_data = SharedData()

# Web 服务器路由
@app.route('/')
def index():
    return send_file('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    try:
        data = request.get_json()
        text = data.get('text', '')
        if not text:
            return jsonify({"status": "error", "message": "范文不能为空"})
            
        shared_data.target_text = text
        print(f"已保存范文: {text}")  # 添加调试信息
        return jsonify({"status": "success", "message": "范文提交成功"})
    except Exception as e:
        return jsonify({"status": "error", "message": str(e)}), 500

@app.route('/get_score')
def get_score():
    if shared_data.latest_score:
        score_data = shared_data.latest_score
        # 返回后清除分数，避免重复获取
        shared_data.latest_score = None
        return jsonify({"status": "success", "data": score_data})
    return jsonify({"status": "waiting"})

# 相似度计算函数
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

def calculate_completeness(target_text, speech_text):
    target_words = set(target_text.split())
    speech_words = set(speech_text.split())
    common_words = target_words.intersection(speech_words)
    return len(common_words) / len(target_words) if target_words else 0

def get_similarity_comment(similarity):
    if similarity >= 0.9: return "非常接近"
    elif similarity >= 0.7: return "比较接近"
    elif similarity >= 0.5: return "部分接近"
    else: return "差异较大"

def get_completeness_comment(completeness):
    if completeness >= 0.9: return "内容完整"
    elif completeness >= 0.7: return "大部分完整"
    elif completeness >= 0.5: return "部分完整"
    else: return "内容缺失"

# 配置参数
appid = "4166554764"    # 项目的 appid
token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"    # 项目的 token
cluster = "volcengine_streaming_common"  # 请求的集群

# 协议常量
PROTOCOL_VERSION = 0b0001
DEFAULT_HEADER_SIZE = 0b0001

PROTOCOL_VERSION_BITS = 4
HEADER_BITS = 4
MESSAGE_TYPE_BITS = 4
MESSAGE_TYPE_SPECIFIC_FLAGS_BITS = 4
MESSAGE_SERIALIZATION_BITS = 4
MESSAGE_COMPRESSION_BITS = 4
RESERVED_BITS = 8

# Message Type:
CLIENT_FULL_REQUEST = 0b0001
CLIENT_AUDIO_ONLY_REQUEST = 0b0010
SERVER_FULL_RESPONSE = 0b1001
SERVER_ACK = 0b1011
SERVER_ERROR_RESPONSE = 0b1111

# Message Type Specific Flags
NO_SEQUENCE = 0b0000
POS_SEQUENCE = 0b0001
NEG_SEQUENCE = 0b0010
NEG_SEQUENCE_1 = 0b0011

# Message Serialization
NO_SERIALIZATION = 0b0000
JSON = 0b0001
THRIFT = 0b0011
CUSTOM_TYPE = 0b1111

# Message Compression
NO_COMPRESSION = 0b0000
GZIP = 0b0001
CUSTOM_COMPRESSION = 0b1111

# 初始化 nest_asyncio
nest_asyncio.apply()

# 全局变量
is_recording = True
all_texts = []
gui = None
loop = None  # 添加全局 loop 变量

# 在程序最开始处（所有 import 语句之后）设置 API key
def init_api_keys():
    """初始化API密钥"""
    try:
        # 语音识别配置
        global appid, token, cluster
        appid = "4166554764"
        token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"
        cluster = "volcengine_streaming_common"
        
        # 千问API配置
        dashscope_key = 'sk-7c04ee6f9432492bb344baa7a5c0162f'
        os.environ['DASHSCOPE_API_KEY'] = dashscope_key
        dashscope.api_key = dashscope_key
        
        # 验证千问API key是否设置成功
        print(f"当前 DashScope API Key: {dashscope.api_key}")
        return True
        
    except Exception as e:
        print(f"API密钥初始化失败: {e}")
        return False

def call_qwen_api(text):
    """调用千问API进行文本纠错"""
    try:
        if not dashscope.api_key:
            print("[错误] DashScope API Key未设置")
            return None
            
        print(f"使用的API Key: {dashscope.api_key}")
        
        messages = [
            {
                'role': 'system',
                'content': '你是一个专业的文本纠错助手。请对输入的文本进行标点符号和错别字的修正，保持原文的意思不变。'
            },
            {
                'role': 'user',
                'content': f'请修正以下文本的标点符号和错别字：\n{text}'
            }
        ]
        
        response = dashscope.Generation.call(
            model='qwen-plus',
            messages=messages,
            result_format='message',
            api_key=dashscope.api_key  # 显式传递API key
        )
        
        if response.status_code == 200:
            corrected_text = response.output.choices[0].message.content.strip()
            print(f"[纠正结果] {corrected_text}")
            return corrected_text
        else:
            error_msg = f"API调用失败: {response.code} - {response.message}"
            print(f"[错误] {error_msg}")
            return None
            
    except Exception as e:
        error_msg = f"纠错API调用失败: {str(e)}"
        print(f"[错误] {error_msg}")
        return None

def on_correction_click():
    """处理纠正按钮点击事件"""
    global is_recording
    try:
        print("纠正按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的当前文本：{current_text}")
        
        if current_text.strip():
            # 3. 调用API进行纠正
            print("开始调用API进行纠正...")
            corrected_text = call_qwen_api(current_text)
            print(f"API返回的纠正结果：{corrected_text}")
            
            # 4. 显示纠正结果
            update_recognition_text(corrected_text, is_correction=True)
            print("更新显示完成")
        else:
            print("文本框为空，不进行API调用")
            
    except Exception as e:
        error_msg = f"纠正文本错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg, is_correction=True)

def on_score_click():
    """处理打分按钮点击事件"""
    global is_recording
    try:
        print("打分按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的朗读文本：{current_text}")
        
        # 3. 获取范文
        target_text = shared_data.target_text
        if not target_text:
            error_msg = "[错误] 请先在网页端输入范文"
            print(error_msg)
            update_recognition_text(error_msg)
            return
            
        if not current_text.strip():
            error_msg = "[错误] 请先进行语音识别"
            print(error_msg)
            update_recognition_text(error_msg)
            return
        
        print("开始调用API进行评分...")
        # 4. 构造提示词
        prompt = f"""
请对比以下两段文本，从准确度、完整度、流畅度三维度进行评分（满100分），并给出详细分析：

范文：
{target_text}

朗读文本：
{current_text}

请按以下格式输出：
准确度：XX分
完整度：XX分
流畅度：XX分
总分：XX分

详细分析：
1. 准确度分析：...
2. 完整度分析：...
3. 流畅度分析：...
4. 改进建议：...
"""
        
        # 5. 调用API进行评分
        response = dashscope.Generation.call(
            api_key=os.getenv('DASHSCOPE_API_KEY'),
            model="qwen-plus",
            messages=[
                {'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': prompt}
            ],
            result_format='message'
        )
        
        if response.status_code == 200:
            result = response.output.choices[0].message.content
            print(f"评分结果：{result}")
            
            # 更新UNIHIKER显示
            update_recognition_text(f"[评分结果]\n{result}")
            
            # 更新共享数据供网页显示
            shared_data.latest_score = {
                "target_text": target_text,
                "speech_text": current_text,
                "score_result": result
            }
            
        else:
            error_msg = f"评分失败: {response.message}"
            print(error_msg)
            update_recognition_text(f"[错误] {error_msg}")
            
    except Exception as e:
        error_msg = f"评分错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg)

def init_gui():
    """初始化GUI，只调用一次"""
    global gui, text_box
    
    if gui is not None:
        print("GUI 已经初始化，跳过重复初始化")
        return gui, text_box
        
    # 初始化GUI
    gui = GUI()
    
    # 创建标题
    title = gui.draw_text(
        x=120,
        y=20,
        text="背诵助手",
        font_size=20,
        origin='center'
    )
    
    # 创建文本框
    text_box = gui.add_text_box(
        x=120,
        y=140,
        w=220,
        h=180,
        origin='center',
        font_size=12
    )
    
    # 创建纠正按钮 - 左边
    correction_button = gui.add_button(
        x=70,  # 移到左边
        y=290,  # 两个按钮在同一水平线上
        w=100,  # 减小按钮宽度
        h=36,
        text="纠正文本",
        origin='center',
        onclick=on_correction_click
    )
    
    # 创建打分按钮 - 右边
    score_button = gui.add_button(
        x=180,  # 移到右边
        y=290,  # 与纠正按钮在同一水平线上
        w=100,  # 减小按钮宽度
        h=36,
        text="文本打分",
        origin='center',
        onclick=on_score_click
    )
    
    return gui, text_box

def kill_existing_flask():
    """杀死已存在的Flask进程"""
    try:
        current_pid = os.getpid()
        
        for proc in psutil.process_iter(['pid', 'name', 'connections']):
            try:
                # 跳过当前进程
                if proc.pid == current_pid:
                    continue
                    
                for conn in proc.connections():
                    if conn.laddr.port == 5000:
                        print(f"发现端口5000被进程占用 (PID: {proc.pid})")
                        proc.kill()
                        print(f"已终止进程 {proc.pid}")
                        time.sleep(1)  # 等待进程完全终止
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                continue
    except Exception as e:
        print(f"清理进程时出错: {e}")

def run_flask():
    """运行Flask服务器"""
    try:
        # 先尝试释放端口
        try:
            import socket
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            sock.bind(('0.0.0.0', 5000))
            sock.close()
        except Exception as e:
            print(f"端口绑定测试失败: {e}")
            # 尝试结束占用端口的进程
            try:
                import os
                os.system("fuser -k 5000/tcp")
                print("已尝试释放端口5000")
                time.sleep(2)
            except:
                print("无法释放端口，请手动结束占用端口的进程")
                return False

        ip = get_ip_address()
        server_info = """
╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://{ip}:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000                               ║
║                                                ║
║  请在浏览器中访问以上地址来输入范文          ║
║                                                ║
╚════════════════════════════════════════════════╝
""".format(ip=ip)
        
        print("\n" + server_info, flush=True)
        
        app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
        return True
    except Exception as e:
        print("\n" + "="*50)
        print("【服务器启动失败】")
        print(f"错误信息: {str(e)}")
        print("="*50 + "\n")
        return False

def get_ip_address():
    """获取本机IP地址"""
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "127.0.0.1"

class AsrWsClient:
    def __init__(self, appid, token, cluster):
        self.appid = appid
        self.token = token
        self.cluster = cluster
        self.ws_url = "wss://openspeech.bytedance.com/api/v2/asr"
        self.success_code = 1000
        self.uid = "streaming_asr_demo"
        self.workflow = "audio_in,resample,partition,vad,fe,decode,itn,nlu_punctuate"
        self.show_language = False
        self.show_utterances = True
        self.result_type = "single"
        self.format = "raw"
        self.rate = 16000
        self.language = "zh-CN"
        self.bits = 16
        self.channel = 1
        self.codec = "raw"
        self.auth_method = "token"

    def construct_request(self, reqid):
        return {
            'app': {
                'appid': self.appid,
                'cluster': self.cluster,
                'token': self.token,
            },
            'user': {
                'uid': self.uid
            },
            'request': {
                'reqid': reqid,
                'nbest': 1,
                'workflow': self.workflow,
                'show_language': self.show_language,
                'show_utterances': self.show_utterances,
                'result_type': self.result_type,
                'sequence': 1
            },
            'audio': {
                'format': self.format,
                'rate': self.rate,
                'language': self.language,
                'bits': self.bits,
                'channel': self.channel,
                'codec': self.codec
            }
        }

    def token_auth(self):
        return {'Authorization': f'Bearer; {self.token}'}

    async def process_microphone(self):
        """实时麦克风录音并识别"""
        global is_recording
        is_recording = True
        
        reqid = str(uuid.uuid4())
        request_params = self.construct_request(reqid)
        
        payload_bytes = str.encode(json.dumps(request_params))
        payload_bytes = gzip.compress(payload_bytes)
        full_request = bytearray(generate_full_default_header())
        full_request.extend(len(payload_bytes).to_bytes(4, 'big'))
        full_request.extend(payload_bytes)

        print("建立WebSocket连接...")
        async with websockets.connect(
            self.ws_url, 
            extra_headers=self.token_auth(), 
            max_size=1000000000
        ) as ws:
            await ws.send(full_request)
            response = await ws.recv()
            result = parse_response(response)
            print(f"初始化响应: {result}")
            
            if 'payload_msg' in result and result['payload_msg']['code'] == self.success_code:
                print("初始化成功")
                print("录音任务已启动")
                chunk_size = 9600
                
                with sd.InputStream(channels=1, samplerate=16000, dtype=np.int16, blocksize=chunk_size) as stream:
                    print("开始录音...")
                    while is_recording:
                        audio_data, overflowed = stream.read(chunk_size)
                        if overflowed:
                            print("警告：音频缓冲区溢出")
                            
                        audio_bytes = audio_data.tobytes()
                        compressed_audio = gzip.compress(audio_bytes)
                        
                        audio_request = bytearray(generate_audio_default_header())
                        audio_request.extend(len(compressed_audio).to_bytes(4, 'big'))
                        audio_request.extend(compressed_audio)
                        
                        await ws.send(audio_request)
                        response = await ws.recv()
                        result = parse_response(response)
                        
                        if 'payload_msg' in result and 'result' in result['payload_msg']:
                            utterances = result['payload_msg']['result'][0].get('utterances', [])
                            for utterance in utterances:
                                if not utterance['definite']:
                                    print(f"\r[识别中...] {utterance['text']}", end='', flush=True)
                                else:
                                    print(f"\n[最终结果] {utterance['text']}")
                                    update_recognition_text(utterance['text'])

# 添加其他必要的函数
def generate_full_default_header():
    return generate_header()

def generate_header(
    version=PROTOCOL_VERSION,
    message_type=CLIENT_FULL_REQUEST,
    message_type_specific_flags=NO_SEQUENCE,
    serial_method=JSON,
    compression_type=GZIP,
    reserved_data=0x00,
    extension_header=bytes()
):
    """生成请求头"""
    header = bytearray()
    header_size = int(len(extension_header) / 4) + 1
    header.append((version << 4) | header_size)
    header.append((message_type << 4) | message_type_specific_flags)
    header.append((serial_method << 4) | compression_type)
    header.append(reserved_data)
    header.extend(extension_header)
    return header

def generate_audio_default_header():
    """生成音频数据请求头"""
    return generate_header(message_type=CLIENT_AUDIO_ONLY_REQUEST)

def parse_response(res):
    """解析响应"""
    try:
        protocol_version = res[0] >> 4
        header_size = res[0] & 0x0f
        message_type = res[1] >> 4
        message_type_specific_flags = res[1] & 0x0f
        serialization_method = res[2] >> 4
        message_compression = res[2] & 0x0f
        reserved = res[3]
        header_extensions = res[4:header_size * 4]
        payload = res[header_size * 4:]
        result = {}
        payload_msg = None
        payload_size = 0
        
        if message_type == SERVER_FULL_RESPONSE:
            payload_size = int.from_bytes(payload[:4], "big", signed=True)
            payload_msg = payload[4:]
        elif message_type == SERVER_ACK:
            seq = int.from_bytes(payload[:4], "big", signed=True)
            result['seq'] = seq
            if len(payload) >= 8:
                payload_size = int.from_bytes(payload[4:8], "big", signed=False)
                payload_msg = payload[8:]
        elif message_type == SERVER_ERROR_RESPONSE:
            code = int.from_bytes(payload[:4], "big", signed=False)
            result['code'] = code
            payload_size = int.from_bytes(payload[4:8], "big", signed=False)
            payload_msg = payload[8:]
            
        if payload_msg is None:
            return result
            
        if message_compression == GZIP:
            payload_msg = gzip.decompress(payload_msg)
            
        if serialization_method == JSON:
            payload_msg = json.loads(str(payload_msg, "utf-8"))
        elif serialization_method != NO_SERIALIZATION:
            payload_msg = str(payload_msg, "utf-8")
            
        result['payload_msg'] = payload_msg
        result['payload_size'] = payload_size
        return result
    except Exception as e:
        return {"error": f"Failed to parse response: {str(e)}"}

def update_recognition_text(new_text, is_correction=False):
    """更新识别结果并保持滚动条在底部"""
    try:
        prefix = "[纠正结果] " if is_correction else ""
        all_texts.append(prefix + new_text)
        
        full_text = "\n".join(all_texts)
        text_box.config(text=full_text)
        text_box.text.see(END)
        
        if gui.master.winfo_exists():
            gui.update()
            
    except Exception as e:
        print(f"更新文本错误: {e}")

@app.errorhandler(404)
def not_found(error):
    return jsonify({"status": "error", "message": "接口不存在"}), 404

@app.errorhandler(500)
def server_error(error):
    return jsonify({"status": "error", "message": "服务器内部错误"}), 500

def cleanup():
    """程序退出时的清理工作"""
    global is_recording, loop
    try:
        is_recording = False
        if loop and not loop.is_closed():
            loop.close()
    except Exception as e:
        print(f"清理时出错: {e}")

if __name__ == '__main__':
    try:
        print("\n=== 程序启动顺序 ===")
        print("0. 初始化API密钥")
        print("1. 启动Web服务器")
        print("2. 初始化GUI界面")
        print("3. 启动语音识别\n")
        
        # 首先初始化API密钥
        if not init_api_keys():
            print("API密钥初始化失败，程序退出")
            sys.exit(1)
            
        # 1. 先启动Web服务器
        print("正在启动Web服务器...\n")
        flask_thread = threading.Thread(target=run_flask)
        flask_thread.daemon = True
        flask_thread.start()
        
        # 等待确保服务器启动
        time.sleep(5)  # 增加等待时间
        
        # 检查服务器是否成功启动
        try:
            response = requests.get('http://localhost:5000', timeout=3)
            if response.status_code != 200:
                raise Exception("服务器响应异常")
        except Exception as e:
            print(f"服务器启动检查失败: {e}")
            print("请确保端口5000未被占用")
            sys.exit(1)
        
        # 2. 初始化GUI
        print("\n正在初始化GUI...\n")
        gui, text_box = init_gui()
        
        # 3. 创建客户端实例并启动语音识别
        print("\n正在启动语音识别...\n")
        client = AsrWsClient(appid, token, cluster)
        
        # 4. 创建事件循环
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        recognition_task = loop.create_task(client.process_microphone())
        
        # 5. 主循环
        while True:
            try:
                loop.run_until_complete(asyncio.sleep(0.1))
                gui.update()
                
            except KeyboardInterrupt:
                print("\n程序被用户中断")
                break
            except Exception as e:
                print(f"\n循环中出现错误: {e}")
                break
                
    except Exception as e:
        print(f"\n程序异常: {e}")
    finally:
        try:
            if 'loop' in locals() and loop and not loop.is_closed():
                loop.close()
        except Exception as e:
            print(f"关闭事件循环时出错: {e}")
        print("程序已正常退出")

# 设置API密钥
os.environ['DASHSCOPE_API_KEY'] = 'sk-7c04ee6f9432492bb344baa7a5c0162f'


=== 程序启动顺序 ===
0. 初始化API密钥
1. 启动Web服务器
2. 初始化GUI界面
3. 启动语音识别

当前 DashScope API Key: sk-7c04ee6f9432492bb344baa7a5c0162f
正在启动Web服务器...



╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://172.16.5.104:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000                               ║
║                                                ║
║  请在浏览器中访问以上地址来输入范文          ║
║                                                ║
╚════════════════════════════════════════════════╝

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.16.5.104:5000/ (Press CTRL+C to quit)
10.1.2.101 - - [31/Oct/2024 16:47:40] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:47:40] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:47:40] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:47:41] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:47:42] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:47:43] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:47:44] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:47:45] "GET /get_score HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2024 16:47:45] "GET / HTTP/1.1" 200 -



正在初始化GUI...



10.1.2.101 - - [31/Oct/2024 16:47:46] "GET /get_score HTTP/1.1" 200 -



正在启动语音识别...

建立WebSocket连接...
初始化响应: {'payload_msg': {'addition': {'duration': '0', 'logid': '2024103116474698296968BEDB4F2C1FF9'}, 'code': 1000, 'message': 'Success', 'reqid': 'bfeedb14-a28a-4aff-8592-9a60f3f3fcae', 'sequence': 1}, 'payload_size': 160}
初始化成功
录音任务已启动
开始录音...


...


[最终结果] 那榆荫下的一潭，
获取到的当前文本：昂河的柔波里。
甘心做一条水草！

开始调用API进行纠正...
使用的API Key: sk-7c04ee6f9432492bb344baa7a5c0162f


10.1.2.101 - - [31/Oct/2024 16:48:57] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:48:58] "GET /get_score HTTP/1.1" 200 -


[纠正结果] 修正后的文本如下：

昂河的柔波里，
甘心做一条水草！
API返回的纠正结果：修正后的文本如下：

昂河的柔波里，
甘心做一条水草！
更新显示完成


10.1.2.101 - - [31/Oct/2024 16:48:59] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:49:00] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:49:01] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:49:02] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:49:03] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:49:04] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:49:05] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:49:06] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:49:07] "GET /get_score HTTP/1.1" 200 -



程序被用户中断
程序已正常退出


In [ ]:
from flask import Flask, request, send_file, jsonify
from difflib import SequenceMatcher
import json
import uuid
import gzip
import asyncio
import websockets
import numpy as np
import sounddevice as sd
import nest_asyncio
from unihiker import GUI
import time
from tkinter import END
import requests
import os
import dashscope
import threading
import socket
import psutil
import sys

# Flask 应用
app = Flask(__name__)

# 共享数据存储
class SharedData:
    def __init__(self):
        self.speech_recognition_results = []
        self.latest_text = ""
        self.target_text = ""
        self.latest_score = None

shared_data = SharedData()

# Web 服务器路由
@app.route('/')
def index():
    return send_file('index.html')

@app.route('/submit', methods=['POST'])
def submit():
    try:
        print("收到提交请求")  # 调试信息
        data = request.get_json()
        print(f"接收到的数据: {data}")  # 调试信息
        
        if not data:
            print("未接收到数据")
            return jsonify({"status": "error", "message": "未接收到数据"}), 400
            
        text = data.get('text', '').strip()
        print(f"提取的文本: {text}")  # 调试信息
        
        if not text:
            print("文本为空")
            return jsonify({"status": "error", "message": "范文不能为空"}), 400
            
        shared_data.target_text = text
        print(f"已保存范文: {shared_data.target_text}")  # 调试信息
        
        return jsonify({
            "status": "success", 
            "message": "范文提交成功",
            "text_length": len(text)
        })
        
    except Exception as e:
        print(f"提交处理错误: {str(e)}")  # 调试信息
        return jsonify({"status": "error", "message": str(e)}), 500

@app.route('/get_score')
def get_score():
    if shared_data.latest_score:
        score_data = shared_data.latest_score
        # 返回后清除分数，避免重复获取
        shared_data.latest_score = None
        return jsonify({"status": "success", "data": score_data})
    return jsonify({"status": "waiting"})

# 相似度计算函数
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1, text2).ratio()

def calculate_completeness(target_text, speech_text):
    target_words = set(target_text.split())
    speech_words = set(speech_text.split())
    common_words = target_words.intersection(speech_words)
    return len(common_words) / len(target_words) if target_words else 0

def get_similarity_comment(similarity):
    if similarity >= 0.9: return "非常接近"
    elif similarity >= 0.7: return "比较接近"
    elif similarity >= 0.5: return "部分接近"
    else: return "差异较大"

def get_completeness_comment(completeness):
    if completeness >= 0.9: return "内容完整"
    elif completeness >= 0.7: return "大部分完整"
    elif completeness >= 0.5: return "部分完整"
    else: return "内容缺失"

# 配置参数
appid = "4166554764"    # 项目的 appid
token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"    # 项目的 token
cluster = "volcengine_streaming_common"  # 请求的集群

# 协议常量
PROTOCOL_VERSION = 0b0001
DEFAULT_HEADER_SIZE = 0b0001

PROTOCOL_VERSION_BITS = 4
HEADER_BITS = 4
MESSAGE_TYPE_BITS = 4
MESSAGE_TYPE_SPECIFIC_FLAGS_BITS = 4
MESSAGE_SERIALIZATION_BITS = 4
MESSAGE_COMPRESSION_BITS = 4
RESERVED_BITS = 8

# Message Type:
CLIENT_FULL_REQUEST = 0b0001
CLIENT_AUDIO_ONLY_REQUEST = 0b0010
SERVER_FULL_RESPONSE = 0b1001
SERVER_ACK = 0b1011
SERVER_ERROR_RESPONSE = 0b1111

# Message Type Specific Flags
NO_SEQUENCE = 0b0000
POS_SEQUENCE = 0b0001
NEG_SEQUENCE = 0b0010
NEG_SEQUENCE_1 = 0b0011

# Message Serialization
NO_SERIALIZATION = 0b0000
JSON = 0b0001
THRIFT = 0b0011
CUSTOM_TYPE = 0b1111

# Message Compression
NO_COMPRESSION = 0b0000
GZIP = 0b0001
CUSTOM_COMPRESSION = 0b1111

# 初始化 nest_asyncio
nest_asyncio.apply()

# 全局变量
is_recording = True
all_texts = []
gui = None
loop = None  # 添加全局 loop 变量

# 在程序最开始处（所有 import 语句之后）设置 API key
def init_api_keys():
    """初始化API密钥"""
    try:
        # 语音识别配置
        global appid, token, cluster
        appid = "4166554764"
        token = "ggmUTHHMXio-nJlKMkRvqEgkcWyfDK0K"
        cluster = "volcengine_streaming_common"
        
        # 千问API配置
        dashscope_key = 'sk-7c04ee6f9432492bb344baa7a5c0162f'
        os.environ['DASHSCOPE_API_KEY'] = dashscope_key
        dashscope.api_key = dashscope_key
        
        # 验证千问API key是否设置成功
        print(f"当前 DashScope API Key: {dashscope.api_key}")
        return True
        
    except Exception as e:
        print(f"API密钥初始化失败: {e}")
        return False

def call_qwen_api(text):
    """调用千问API进行文本纠错"""
    try:
        if not dashscope.api_key:
            print("[错误] DashScope API Key未设置")
            return None
            
        print(f"使用的API Key: {dashscope.api_key}")
        
        messages = [
            {
                'role': 'system',
                'content': '你是一个专业的文本纠错助手。请对输入的文本进行标点符号和错别字的修正，保持原文的意思不变。'
            },
            {
                'role': 'user',
                'content': f'请修正以下文本的标点符号和错别字：\n{text}'
            }
        ]
        
        response = dashscope.Generation.call(
            model='qwen-plus',
            messages=messages,
            result_format='message',
            api_key=dashscope.api_key  # 显式传递API key
        )
        
        if response.status_code == 200:
            corrected_text = response.output.choices[0].message.content.strip()
            print(f"[纠正结果] {corrected_text}")
            return corrected_text
        else:
            error_msg = f"API调用失败: {response.code} - {response.message}"
            print(f"[错误] {error_msg}")
            return None
            
    except Exception as e:
        error_msg = f"纠错API调用失败: {str(e)}"
        print(f"[错误] {error_msg}")
        return None

def on_correction_click():
    """处理纠正按钮点击事件"""
    global is_recording
    try:
        print("纠正按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的当前文本：{current_text}")
        
        if current_text.strip():
            # 3. 调用API进行纠正
            print("开始调用API进行纠正...")
            corrected_text = call_qwen_api(current_text)
            print(f"API返回的纠正结果：{corrected_text}")
            
            # 4. 显示纠正结果
            update_recognition_text(corrected_text, is_correction=True)
            print("更新显示完成")
        else:
            print("文本框为空，不进行API调用")
            
    except Exception as e:
        error_msg = f"纠正文本错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg, is_correction=True)

def on_score_click():
    """处理打分按钮点击事件"""
    global is_recording
    try:
        print("打分按钮被点击")
        
        # 1. 停止语音识别
        is_recording = False
        print("正在停止语音识别...")
        time.sleep(1)
        
        # 2. 获取当前文本
        current_text = text_box.text.get("1.0", END)
        print(f"获取到的朗读文本：{current_text}")
        
        # 3. 获取范文
        target_text = shared_data.target_text
        if not target_text:
            error_msg = "[错误] 请先在网页端输入范文"
            print(error_msg)
            update_recognition_text(error_msg)
            return
            
        if not current_text.strip():
            error_msg = "[错误] 请先进行语音识别"
            print(error_msg)
            update_recognition_text(error_msg)
            return
        
        print("开始调用API进行评分...")
        # 4. 构造提示词
        prompt = f"""
请对比以下两段文本，从准确度、完整度、流畅度三维度进行评分（满100分），并给出详细分析：

范文：
{target_text}

朗读文本：
{current_text}

请按以下格式输出：
准确度：XX分
完整度：XX分
流畅度：XX分
总分：XX分

详细分析：
1. 准确度分析：...
2. 完整度分析：...
3. 流畅度分析：...
4. 改进建议：...
"""
        
        # 5. 调用API进行评分
        response = dashscope.Generation.call(
            api_key=os.getenv('DASHSCOPE_API_KEY'),
            model="qwen-plus",
            messages=[
                {'role': 'system', 'content': 'You are a helpful assistant.'},
                {'role': 'user', 'content': prompt}
            ],
            result_format='message'
        )
        
        if response.status_code == 200:
            result = response.output.choices[0].message.content
            print(f"评分结果：{result}")
            
            # 更新UNIHIKER显示
            update_recognition_text(f"[评分结果]\n{result}")
            
            # 更新共享数据供网页显示
            shared_data.latest_score = {
                "target_text": target_text,
                "speech_text": current_text,
                "score_result": result
            }
            
        else:
            error_msg = f"评分失败: {response.message}"
            print(error_msg)
            update_recognition_text(f"[错误] {error_msg}")
            
    except Exception as e:
        error_msg = f"评分错误: {str(e)}"
        print(error_msg)
        update_recognition_text(error_msg)

def init_gui():
    """初始化GUI，只调用一次"""
    global gui, text_box
    
    if gui is not None:
        print("GUI 已经初始化，跳过重复初始化")
        return gui, text_box
        
    # 初始化GUI
    gui = GUI()
    
    # 创建标题
    title = gui.draw_text(
        x=120,
        y=20,
        text="背诵助手",
        font_size=20,
        origin='center'
    )
    
    # 创建文本框
    text_box = gui.add_text_box(
        x=120,
        y=140,
        w=220,
        h=180,
        origin='center',
        font_size=12
    )
    
    # 创建纠正按钮 - 左边
    correction_button = gui.add_button(
        x=70,  # 移到左边
        y=290,  # 两个按钮在同一水平线上
        w=100,  # 减小按钮宽度
        h=36,
        text="纠正文本",
        origin='center',
        onclick=on_correction_click
    )
    
    # 创建打分按钮 - 右边
    score_button = gui.add_button(
        x=180,  # 移到右边
        y=290,  # 与纠正按钮在同一水平线上
        w=100,  # 减小按钮宽度
        h=36,
        text="文本打分",
        origin='center',
        onclick=on_score_click
    )
    
    return gui, text_box

def kill_existing_flask():
    """杀死已存在的Flask进程"""
    try:
        current_pid = os.getpid()
        
        for proc in psutil.process_iter(['pid', 'name', 'connections']):
            try:
                # 跳过当前进程
                if proc.pid == current_pid:
                    continue
                    
                for conn in proc.connections():
                    if conn.laddr.port == 5000:
                        print(f"发现端口5000被进程占用 (PID: {proc.pid})")
                        proc.kill()
                        print(f"已终止进程 {proc.pid}")
                        time.sleep(1)  # 等待进程完全终止
            except (psutil.NoSuchProcess, psutil.AccessDenied):
                continue
    except Exception as e:
        print(f"清理进程时出错: {e}")

def run_flask():
    """运行Flask服务器"""
    try:
        # 先尝试释放端口
        try:
            import socket
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
            sock.bind(('0.0.0.0', 5000))
            sock.close()
        except Exception as e:
            print(f"端口绑定测试失败: {e}")
            # 尝试结束占用端口的进程
            try:
                import os
                os.system("fuser -k 5000/tcp")
                print("已尝试释放端口5000")
                time.sleep(2)
            except:
                print("无法释放端口，请手动结束占用端口的进程")
                return False

        ip = get_ip_address()
        server_info = """
╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://{ip}:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000                               ║
║                                                ║
║  请在浏览器中访问以上地址来输入范文          ║
║                                                ║
╚════════════════════════════════════════════════╝
""".format(ip=ip)
        
        print("\n" + server_info, flush=True)
        
        app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)
        return True
    except Exception as e:
        print("\n" + "="*50)
        print("【服务器启动失败】")
        print(f"错误信息: {str(e)}")
        print("="*50 + "\n")
        return False

def get_ip_address():
    """获取本机IP地址"""
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip = s.getsockname()[0]
        s.close()
        return ip
    except:
        return "127.0.0.1"

class AsrWsClient:
    def __init__(self, appid, token, cluster):
        self.appid = appid
        self.token = token
        self.cluster = cluster
        self.ws_url = "wss://openspeech.bytedance.com/api/v2/asr"
        self.success_code = 1000
        self.uid = "streaming_asr_demo"
        self.workflow = "audio_in,resample,partition,vad,fe,decode,itn,nlu_punctuate"
        self.show_language = False
        self.show_utterances = True
        self.result_type = "single"
        self.format = "raw"
        self.rate = 16000
        self.language = "zh-CN"
        self.bits = 16
        self.channel = 1
        self.codec = "raw"
        self.auth_method = "token"

    def construct_request(self, reqid):
        return {
            'app': {
                'appid': self.appid,
                'cluster': self.cluster,
                'token': self.token,
            },
            'user': {
                'uid': self.uid
            },
            'request': {
                'reqid': reqid,
                'nbest': 1,
                'workflow': self.workflow,
                'show_language': self.show_language,
                'show_utterances': self.show_utterances,
                'result_type': self.result_type,
                'sequence': 1
            },
            'audio': {
                'format': self.format,
                'rate': self.rate,
                'language': self.language,
                'bits': self.bits,
                'channel': self.channel,
                'codec': self.codec
            }
        }

    def token_auth(self):
        return {'Authorization': f'Bearer; {self.token}'}

    async def process_microphone(self):
        """实时麦克风录音并识别"""
        global is_recording
        is_recording = True
        
        reqid = str(uuid.uuid4())
        request_params = self.construct_request(reqid)
        
        payload_bytes = str.encode(json.dumps(request_params))
        payload_bytes = gzip.compress(payload_bytes)
        full_request = bytearray(generate_full_default_header())
        full_request.extend(len(payload_bytes).to_bytes(4, 'big'))
        full_request.extend(payload_bytes)

        print("建立WebSocket连接...")
        async with websockets.connect(
            self.ws_url, 
            extra_headers=self.token_auth(), 
            max_size=1000000000
        ) as ws:
            await ws.send(full_request)
            response = await ws.recv()
            result = parse_response(response)
            print(f"初始化响应: {result}")
            
            if 'payload_msg' in result and result['payload_msg']['code'] == self.success_code:
                print("初始化成功")
                print("录音任务已启动")
                chunk_size = 9600
                
                with sd.InputStream(channels=1, samplerate=16000, dtype=np.int16, blocksize=chunk_size) as stream:
                    print("开始录音...")
                    while is_recording:
                        audio_data, overflowed = stream.read(chunk_size)
                        if overflowed:
                            print("警告：音频缓冲区溢出")
                            
                        audio_bytes = audio_data.tobytes()
                        compressed_audio = gzip.compress(audio_bytes)
                        
                        audio_request = bytearray(generate_audio_default_header())
                        audio_request.extend(len(compressed_audio).to_bytes(4, 'big'))
                        audio_request.extend(compressed_audio)
                        
                        await ws.send(audio_request)
                        response = await ws.recv()
                        result = parse_response(response)
                        
                        if 'payload_msg' in result and 'result' in result['payload_msg']:
                            utterances = result['payload_msg']['result'][0].get('utterances', [])
                            for utterance in utterances:
                                if not utterance['definite']:
                                    print(f"\r[识别中...] {utterance['text']}", end='', flush=True)
                                else:
                                    print(f"\n[最终结果] {utterance['text']}")
                                    update_recognition_text(utterance['text'])

# 添加其他必要的函数
def generate_full_default_header():
    return generate_header()

def generate_header(
    version=PROTOCOL_VERSION,
    message_type=CLIENT_FULL_REQUEST,
    message_type_specific_flags=NO_SEQUENCE,
    serial_method=JSON,
    compression_type=GZIP,
    reserved_data=0x00,
    extension_header=bytes()
):
    """生成请求头"""
    header = bytearray()
    header_size = int(len(extension_header) / 4) + 1
    header.append((version << 4) | header_size)
    header.append((message_type << 4) | message_type_specific_flags)
    header.append((serial_method << 4) | compression_type)
    header.append(reserved_data)
    header.extend(extension_header)
    return header

def generate_audio_default_header():
    """生成音频数据请求头"""
    return generate_header(message_type=CLIENT_AUDIO_ONLY_REQUEST)

def parse_response(res):
    """解析响应"""
    try:
        protocol_version = res[0] >> 4
        header_size = res[0] & 0x0f
        message_type = res[1] >> 4
        message_type_specific_flags = res[1] & 0x0f
        serialization_method = res[2] >> 4
        message_compression = res[2] & 0x0f
        reserved = res[3]
        header_extensions = res[4:header_size * 4]
        payload = res[header_size * 4:]
        result = {}
        payload_msg = None
        payload_size = 0
        
        if message_type == SERVER_FULL_RESPONSE:
            payload_size = int.from_bytes(payload[:4], "big", signed=True)
            payload_msg = payload[4:]
        elif message_type == SERVER_ACK:
            seq = int.from_bytes(payload[:4], "big", signed=True)
            result['seq'] = seq
            if len(payload) >= 8:
                payload_size = int.from_bytes(payload[4:8], "big", signed=False)
                payload_msg = payload[8:]
        elif message_type == SERVER_ERROR_RESPONSE:
            code = int.from_bytes(payload[:4], "big", signed=False)
            result['code'] = code
            payload_size = int.from_bytes(payload[4:8], "big", signed=False)
            payload_msg = payload[8:]
            
        if payload_msg is None:
            return result
            
        if message_compression == GZIP:
            payload_msg = gzip.decompress(payload_msg)
            
        if serialization_method == JSON:
            payload_msg = json.loads(str(payload_msg, "utf-8"))
        elif serialization_method != NO_SERIALIZATION:
            payload_msg = str(payload_msg, "utf-8")
            
        result['payload_msg'] = payload_msg
        result['payload_size'] = payload_size
        return result
    except Exception as e:
        return {"error": f"Failed to parse response: {str(e)}"}

def update_recognition_text(new_text, is_correction=False):
    """更新识别结果并保持滚动条在底部"""
    try:
        prefix = "[纠正结果] " if is_correction else ""
        all_texts.append(prefix + new_text)
        
        full_text = "\n".join(all_texts)
        text_box.config(text=full_text)
        text_box.text.see(END)
        
        if gui.master.winfo_exists():
            gui.update()
            
    except Exception as e:
        print(f"更新文本错误: {e}")

@app.errorhandler(404)
def not_found(error):
    return jsonify({"status": "error", "message": "接口不存在"}), 404

@app.errorhandler(500)
def server_error(error):
    return jsonify({"status": "error", "message": "服务器内部错误"}), 500

def cleanup():
    """程序退出时的清理工作"""
    global is_recording, loop
    try:
        is_recording = False
        if loop and not loop.is_closed():
            loop.close()
    except Exception as e:
        print(f"清理时出错: {e}")

if __name__ == '__main__':
    try:
        print("\n=== 程序启动顺序 ===")
        print("0. 初始化API密钥")
        print("1. 启动Web服务器")
        print("2. 初始化GUI界面")
        print("3. 启动语音识别\n")
        
        # 首先初始化API密钥
        if not init_api_keys():
            print("API密钥初始化失败，程序退出")
            sys.exit(1)
            
        # 1. 先启动Web服务器
        print("正在启动Web服务器...\n")
        flask_thread = threading.Thread(target=run_flask)
        flask_thread.daemon = True
        flask_thread.start()
        
        # 等待确保服务器启动
        time.sleep(5)  # 增加等待时间
        
        # 检查服务器是否成功启动
        try:
            response = requests.get('http://localhost:5000', timeout=3)
            if response.status_code != 200:
                raise Exception("服务器响应异常")
        except Exception as e:
            print(f"服务器启动检查失败: {e}")
            print("请确保端口5000未被占用")
            sys.exit(1)
        
        # 2. 初始化GUI
        print("\n正在初始化GUI...\n")
        gui, text_box = init_gui()
        
        # 3. 创建客户端实例并启动语音识别
        print("\n正在启动语音识别...\n")
        client = AsrWsClient(appid, token, cluster)
        
        # 4. 创建事件循环
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        recognition_task = loop.create_task(client.process_microphone())
        
        # 5. 主循环
        while True:
            try:
                loop.run_until_complete(asyncio.sleep(0.1))
                gui.update()
                
            except KeyboardInterrupt:
                print("\n程序被用户中断")
                break
            except Exception as e:
                print(f"\n循环中出现错误: {e}")
                break
                
    except Exception as e:
        print(f"\n程序异常: {e}")
    finally:
        try:
            if 'loop' in locals() and loop and not loop.is_closed():
                loop.close()
        except Exception as e:
            print(f"关闭事件循环时出错: {e}")
        print("程序已正常退出")

# 设置API密钥
os.environ['DASHSCOPE_API_KEY'] = 'sk-7c04ee6f9432492bb344baa7a5c0162f'


=== 程序启动顺序 ===
0. 初始化API密钥
1. 启动Web服务器
2. 初始化GUI界面
3. 启动语音识别

当前 DashScope API Key: sk-7c04ee6f9432492bb344baa7a5c0162f
正在启动Web服务器...



╔════════════════════════════════════════════════╗
║             Web服务器启动信息                  ║
╠════════════════════════════════════════════════╣
║                                                ║
║  状态: 正在启动...                            ║
║  访问地址: http://172.16.5.104:5000                   ║
║  本地地址: http://localhost:5000              ║
║  监听端口: 5000                               ║
║                                                ║
║  请在浏览器中访问以上地址来输入范文          ║
║                                                ║
╚════════════════════════════════════════════════╝

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.16.5.104:5000/ (Press CTRL+C to quit)
10.1.2.101 - - [31/Oct/2024 16:53:08] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:53:08] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:53:08] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:53:09] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:53:10] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:53:11] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:53:12] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:53:13] "GET /get_score HTTP/1.1" 200 -
127.0.0.1 - - [31/Oct/2024 16:53:13] "GET / HTTP/1.1" 200 -



正在初始化GUI...



10.1.2.101 - - [31/Oct/2024 16:53:14] "GET /get_score HTTP/1.1" 200 -



正在启动语音识别...

建立WebSocket连接...
初始化响应: {'payload_msg': {'addition': {'duration': '0', 'logid': '20241031165314679BD5E2A932552815CD'}, 'code': 1000, 'message': 'Success', 'reqid': '253fc01a-2db5-4503-b896-d76915658c96', 'sequence': 1}, 'payload_size': 158}
初始化成功
录音任务已启动
开始录音...


...

获取到的朗读文本：对呀！
一支长高。
再别康桥！
许志摩。
轻轻的。
走了！
正如我轻轻的来。
我轻轻的招手。
作别。
西天的云彩！
[纠正结果] 对呀！
一支长高。
《再别康桥》
——许志摩
轻轻的，
走了！
正如我轻轻的来；
我轻轻的招手，
作别西天的云彩。

开始调用API进行评分...


10.1.2.101 - - [31/Oct/2024 16:54:48] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:54:49] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:54:50] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:54:51] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:54:52] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:54:53] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:54:54] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:54:55] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:54:56] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:54:57] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:54:58] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:54:59] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:00] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:01] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/O

评分结果：**准确度：40分**
**完整度：30分**
**流畅度：35分**
**总分：35分**

**详细分析：**

1. **准确度分析：**
   - 朗读文本在内容上与原文存在较大的偏差，多处出现了断句和遗漏。例如，“对呀！”、“一支长高。”这两句话在原文中完全不存在，且与诗歌的内容无关，严重影响了文本的准确性。
   - 此外，朗读文本中“许志摩”应为作者的名字，但在原文中并未出现，而是在标题下方以破折号引出，这一点在朗读文本中也未能正确处理。
   - 原文中的许多关键诗句，如“那河畔的金柳，是夕阳中的新娘；波光里的艳影，在我的心头荡漾。”等均未在朗读文本中出现，导致文本信息严重缺失。
   - 朗读文本中出现的标点符号使用也不规范，如多次使用感叹号，而原文中并未如此频繁地使用感叹号，这进一步影响了文本的准确性。

2. **完整度分析：**
   - 朗读文本仅包含了原文前四行的内容，且在表述上存在明显的遗漏和错误。例如，“那河畔的金柳，是夕阳中的新娘；波光里的艳影，在我的心头荡漾。”等后续内容均未出现，使得文本内容极其不完整。
   - 朗读文本未能涵盖原文的主要情感表达和意境描绘，导致读者无法全面理解诗歌的内涵和情感。

3. **流畅度分析：**
   - 朗读文本的断句和停顿显得生硬，缺乏连贯性。例如，“轻轻的。走了！”这样的断句方式使得句子失去了原有的韵律感，影响了朗读的流畅度。
   - 朗读文本中的标点符号使用不当，如多次使用感叹号，而原文中并未如此频繁地使用，这使得朗读时的语气和节奏变得突兀，影响了整体的流畅度。
   - 朗读文本在表达上显得支离破碎，缺乏连贯性和逻辑性，使得读者难以跟随文本的思路和情感变化。

4. **改进建议：**
   - **准确度方面：** 严格按照原文内容进行朗读，避免添加或删减原文中的任何内容，确保文本的准确性。
   - **完整度方面：** 全面涵盖原文的所有内容，特别是那些关键的诗句和情感表达，确保文本的完整性。
   - **流畅度方面：** 注意断句和停顿的合理使用，遵循原文的韵律和节奏，使朗读更加自然流畅。同时，正确使用标点符号，避免不必要的停顿和语气变化，保持朗读的连贯性。


10.1.2.101 - - [31/Oct/2024 16:55:17] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:18] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:19] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:20] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:21] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:22] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:23] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:24] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:25] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:26] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:27] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:28] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:29] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:55:30] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/O

10.1.2.101 - - [31/Oct/2024 16:57:15] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:57:16] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:57:17] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:57:18] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:57:19] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:57:20] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:57:21] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:57:22] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:57:23] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:57:24] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:57:25] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:57:26] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:57:27] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/Oct/2024 16:57:28] "GET /get_score HTTP/1.1" 200 -
10.1.2.101 - - [31/O

In [ ]:
基本实现了所有共功能，仅仅没有实现，个别学生背诵成绩呈现。